In [ ]:
# Cài đặt các thư viện cần thiết
!pip install python-dotenv
!pip install neo4j
!pip install llama-index
!pip install llama-index-graph-stores-neo4j
!pip install llama-index-llms-openai
!pip install llama-index-vector-stores-neo4jvector
!pip install openai
!pip install pandas numpy scikit-learn mlxtend
!pip install nltk

In [ ]:
from llama_index.core.retrievers import CustomPGRetriever, VectorContextRetriever, TextToCypherRetriever
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI

In [ ]:
import os
import logging
import csv
import json
import re
from datetime import datetime
from dotenv import load_dotenv
import nest_asyncio
from google.colab import userdata
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from neo4j import GraphDatabase
import heapq
from typing import List, Dict, Any
from mlxtend.frequent_patterns import apriori, association_rules
from llama_index.core import (
PropertyGraphIndex,
VectorStoreIndex,
SimpleDirectoryReader,
StorageContext,
Settings,
Document
)
from llama_index.llms.openai import OpenAI
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore
from llama_index.vector_stores.neo4jvector import Neo4jVectorStore
from llama_index.core.indices.property_graph import TextToCypherRetriever
from llama_index.core.response_synthesizers import TreeSummarize
from llama_index.core.prompts import PromptTemplate
import openai
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from google.colab import drive

# Hàm mở rộng tag ngữ nghĩa
from nltk.corpus import wordnet
import nltk
nltk.download('wordnet')
import pandas as pd
#drive.mount('/content/drive')

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import time

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
llm = OpenAI(model="gpt-3.5-turbo", temperature=0.3)

# Config

In [ ]:
# Tạo thư mục log trước khi thiết lập logging
log_dir = '/content/drive/MyDrive/log'
os.makedirs(log_dir, exist_ok=True)
log_file = os.path.join(log_dir, 'learning_path_system.log')

# Xóa cấu hình logging cũ và thiết lập lại
logger = logging.getLogger('design_learning_path')
logger.handlers = []  # Xóa tất cả handler cũ của logger cụ thể
logger.setLevel(logging.INFO)
logger.propagate = False  # Ngăn log gửi lên root logger

# Tạo file handler
file_handler = logging.FileHandler(log_file)
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))

# Thêm handler vào logger
logger.addHandler(file_handler)

# Kiểm tra logger
logger.info("Logger đã được cấu hình")
logger.handlers[0].flush()  # Flush để ghi ngay lập tức

In [ ]:


# Hàm hiển thị nội dung tệp log
def display_log_content(log_file: str):
    """Hiển thị nội dung tệp log."""
    try:
        if os.path.exists(log_file):
            with open(log_file, 'r', encoding='utf-8') as f:
                print("Nội dung tệp log:")
                print(f.read())
        else:
            print(f"Không tìm thấy tệp log tại: {log_file}")
    except Exception as e:
        print(f"Lỗi khi đọc tệp log: {str(e)}")

# Định nghĩa lớp Config với các hằng số toàn cục
class Config:
    # Đường dẫn tệp
    STUDENT_FILE = "/content/drive/MyDrive/LP_data_based/student.csv"
    NODES_FILE = "/content/drive/MyDrive/LP_data_based/nodes.csv"

    IMPORT_NODES_FILE = "nodes.csv"
    RELATIONSHIPS_FILE = "/content/drive/MyDrive/LP_data_based/relationships.csv"
    IMPORT_RELATIONSHIPS_FILE = "relationships.csv"
    QUIZ_OUTPUT_DIR = "/content/drive/MyDrive/Output_CSV_KGLP"
    PROPERTY_GRAPH_STORAGE_DIR = "/content/drive/MyDrive/property_graph_storage"

    # Cấu hình mặc định
    DEFAULT_GOAL_NODE_PREFIX = "concept:advanced_"
    DEFAULT_CONTEXT_EXAMPLE = "e_learning"

    # Thuộc tính nút (bắt buộc)
    PROPERTY_DOCUMENT_ID = "Document_ID"
    PROPERTY_ID = "Node_ID"
    PROPERTY_SOURCE_ID = "Source_Node_ID"
    PROPERTY_TARGET_ID = "Target_Node_ID"
    PROPERTY_NODE_LABEL = "Node_Label"
    PROPERTY_SANITIZED_CONCEPT = "Sanitized_Concept"
    PROPERTY_CONTEXT = "Context"
    PROPERTY_DEFINITION = "Definition"
    PROPERTY_EXAMPLE = "Example"
    PROPERTY_LEARNING_OBJECTIVE = "Learning_Objective"
    PROPERTY_SKILL_LEVEL = "Skill_Level"
    PROPERTY_TIME_ESTIMATE = "Time_Estimate"
    PROPERTY_DIFFICULTY = "Difficulty"
    PROPERTY_PRIORITY = "Priority"
    PROPERTY_PREREQUISITES = "Prerequisites"
    PROPERTY_SEMANTIC_TAGS = "Semantic_Tags"
    PROPERTY_FOCUSED_SEMANTIC_TAGS = "Focused_Semantic_Tags"

    # Thuộc tính nút (tùy chọn)
    PROPERTY_COMMON_ERRORS = "Common_Errors"
    PROPERTY_LEARNING_STYLE_PREFERENCE = "Learning_Style_Preference"

    # Loại quan hệ
    RELATIONSHIP_REQUIRES = "REQUIRES"
    RELATIONSHIP_IS_PREREQUISITE_OF = "IS_PREREQUISITE_OF"
    RELATIONSHIP_NEXT = "NEXT"
    RELATIONSHIP_REMEDIATES = "REMEDIATES"
    RELATIONSHIP_HAS_ALTERNATIVE_PATH = "HAS_ALTERNATIVE_PATH"
    RELATIONSHIP_SIMILAR_TO = "SIMILAR_TO"
    RELATIONSHIP_IS_SUBCONCEPT_OF = "IS_SUBCONCEPT_OF"

    # Cấu hình bài kiểm tra
    QUIZ_NUM_QUESTIONS = 20
    QUIZ_DISTRIBUTION = {"basic": 0.5, "intermediate": 0.3, "advanced": 0.2}
    QUIZ_PASSING_SCORE = 70

    # Tham số A* algorithm
    ASTAR_DIFFICULTY_FILTER = ["STANDARD", "ADVANCED"]
    ASTAR_MAX_TIME_ESTIMATE = 60
    ASTAR_SKILL_LEVELS_LOW = ["Remember", "Understand"]
    ASTAR_SKILL_LEVELS_HIGH = ["Apply", "Analyze", "Evaluate", "Create"]
    ASTAR_CURRENT_LEVEL_THRESHOLD = 50
    ASTAR_HEURISTIC_WEIGHTS = {
        "priority": 1.0,
        "difficulty_standard": 1.0,
        "difficulty_advanced": 2.0,
        "skill_level": {
            "Remember": 1.0,
            "Understand": 2.0,
            "Apply": 3.0,
            "Analyze": 4.0,
            "Evaluate": 5.0,
            "Create": 6.0,
            "default": 0.5
        },
        "time_estimate": 1.0 / 60
    }
    ASTAR_SKILL_LEVEL_SCORES = {
        "Remember": 1,
        "Understand": 2,
        "Apply": 3,
        "Analyze": 4,
        "Evaluate": 5,
        "Create": 6,
        "default": 0
    }
    ASTAR_CANDIDATE_LIMIT = 10
    ASTAR_CURRENT_LEVEL_WEIGHT = 0.4
    ASTAR_ASSESSMENT_WEIGHT = 0.6

    # Tham số nút đích
    GOAL_NODE_SUGGESTION_LIMIT = 3
    GOAL_NODE_DEFAULT_DIFFICULTY = "ADVANCED"

    # Hàm lưu dữ liệu học tập
    LEARNING_PATHS_FILE = "/content/drive/MyDrive/LP_data_based/learning_paths.csv"
    LEARNING_DATA_FILE = "/content/drive/MyDrive/LP_data_based/learning_data.csv"
    TEACHER_NOTIFICATION_FILE = "/content/drive/MyDrive/Output_CSV_KGLP/teacher_notifications.csv"


    # Ngưỡng tương đồng tối thiểu dùng cho collaborative_filtering
    MIN_SIMILARITY_THRESHOLD = 0.5

    # Learning style
    LEARNING_STYLE_VARK = ["visual", "auditory", "reading_writing", "kinesthetic"]
    DEFAULT_LEARNING_STYLE = "reading_writing"

    # Danh sách thuộc tính bắt buộc
    required_props = [
        PROPERTY_ID,
        PROPERTY_SANITIZED_CONCEPT,
        PROPERTY_CONTEXT,
        PROPERTY_DEFINITION,
        PROPERTY_EXAMPLE,
        PROPERTY_LEARNING_OBJECTIVE,
        PROPERTY_SKILL_LEVEL,
        PROPERTY_TIME_ESTIMATE,
        PROPERTY_DIFFICULTY,
        PROPERTY_PRIORITY,
        PROPERTY_PREREQUISITES,
        PROPERTY_SEMANTIC_TAGS,
        PROPERTY_FOCUSED_SEMANTIC_TAGS
    ]

    # Danh sách thuộc tính tùy chọn từ prompt SPR Generator
    optional_props = [
        "Numerical_Data",
        "Learning_Style_Preference",
        "Reference",
        "Learning_Tips",
        "Detailed_Example",
        "Real_World_Application",
        "Common_Errors",
        "Instruction_Content_Template",
        "MCQ_Template",
        "Correct_Answer",
        "Remediation_Path",
        "Source_Section",
        "Multimedia_Resources",
        "Node_Category"
    ]


# Cấu hình Neo4j
NEO4J_CONFIG = {
    "url": userdata.get("NEO4J_URL"),
    "username": userdata.get("NEO4J_USER"),
    "password": userdata.get("NEO4J_PASSWORD"),
}

# Kết nối tới Neo4j
try:
    driver = GraphDatabase.driver(
        NEO4J_CONFIG["url"],
        auth=(NEO4J_CONFIG["username"], NEO4J_CONFIG["password"])
    )
    logger.info("Kết nối tới Neo4j thành công")
    logger.handlers[0].flush()
except Exception as e:
    logger.error(f"Lỗi kết nối Neo4j: {str(e)}")
    logger.handlers[0].flush()
    raise

# Khởi tạo Neo4j Graph Store
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore

graph_store = Neo4jPropertyGraphStore(
    username=NEO4J_CONFIG["username"],
    password=NEO4J_CONFIG["password"],
    url=NEO4J_CONFIG["url"],
    database="neo4j"
)
logger.info("Khởi tạo Neo4j thành công")
logger.handlers[0].flush()

# Cấu hình OpenAI LLM
openai.api_key = userdata.get("OPENAI_API_KEY")

# Set up embedding model and LLM
embed_model = OpenAIEmbedding(model_name="text-embedding-3-small")

Settings.llm = OpenAI(
    model="gpt-3.5-turbo",
    temperature=0.3,
    system_prompt="Bạn là chuyên gia lộ trình học tập chuyên về giáo dục."
)
logger.info("Cấu hình OpenAI LLM thành công")
logger.handlers[0].flush()

kg_index = None
# Khởi tạo PropertyGraphIndex
try:
    if os.path.exists(Config.PROPERTY_GRAPH_STORAGE_DIR):
        kg_index = PropertyGraphIndex.from_existing(
            property_graph_store=graph_store,
            persist_dir=Config.PROPERTY_GRAPH_STORAGE_DIR
        )

        logger.info("Tải PropertyGraphIndex từ nơi lưu trữ thành công")
    else:
        kg_index = PropertyGraphIndex.from_existing(
            property_graph_store=graph_store,
            embed_kg_nodes=True,
            include_embeddings=True
        )
    kg_index.storage_context.persist(persist_dir=Config.PROPERTY_GRAPH_STORAGE_DIR)
    logger.info("Khởi tạo và lưu PropertyGraphIndex thành công")
    logger.handlers[0].flush()
except Exception as e:
    logger.error(f"Lỗi khởi tạo PropertyGraphIndex: {str(e)}")
    logger.handlers[0].flush()
    raise



# Kiểm tra kết nối Neo4j
def test_neo4j_connection():
    with driver.session() as session:
        result = session.run("RETURN 1")
        return result.single()[0] == 1

if test_neo4j_connection():
    logger.info("Kiểm tra kết nối Neo4j: Thành công")
    logger.handlers[0].flush()
    display_log_content(log_file)
else:
    logger.error("Kiểm tra kết nối Neo4j: Thất bại")
    logger.handlers[0].flush()
    display_log_content(log_file)
    raise Exception("Không thể xác minh kết nối Neo4j")



# Thêm biến toàn cục cho start node
global_start_node = None

Connection to 52.77.3.235:11831 closed with incomplete handshake response)
Connection to 52.77.3.235:11831 closed with incomplete handshake response)
Connection to 52.77.3.235:11831 closed with incomplete handshake response)
Connection to 52.77.3.235:11831 closed with incomplete handshake response)
Connection to 52.77.3.235:11831 closed with incomplete handshake response)


ServiceUnavailable: Couldn't connect to 0.tcp.ap.ngrok.io:11831 (resolved to ('3.1.16.19:11831',)):
Connection to 3.1.16.19:11831 closed with incomplete handshake response

In [ ]:
# Define custom retriever
class MyCustomRetriever(CustomPGRetriever):
    def init(self, **kwargs):
        self.vector_retriever = VectorContextRetriever(
            graph_store=graph_store,
            embed_model=embed_model,
            similarity_top_k=5
        )
        self.cypher_retriever = TextToCypherRetriever(
            graph_store=graph_store,
            llm=llm
        )
    def custom_retrieve(self, query):
        vector_results = self.vector_retriever.retrieve(query)
        cypher_results = self.cypher_retriever.retrieve(query)
        # Combine results (e.g., rerank or merge)
        combined_results = vector_results + cypher_results
        return combined_results

# jaccard_similarity

In [ ]:
# Hàm tính độ tương đồng Jaccard
def jaccard_similarity(tags1: List[str], tags2: List[str]) -> float:
    """Tính độ tương đồng Jaccard giữa hai danh sách tag.

    Args:
        tags1 (List[str]): Danh sách tag thứ nhất.
        tags2 (List[str]): Danh sách tag thứ hai.

    Returns:
        float: Độ tương đồng Jaccard (0.0 đến 1.0).
    """
    try:
        set1, set2 = set(tags1), set(tags2)
        intersection = len(set1.intersection(set2))
        union = len(set1.union(set2))
        return intersection / union if union > 0 else 0.0
    except Exception as e:
        logger.error(f"Lỗi trong jaccard_similarity: {str(e)}")
        return 0.0


# initialize_vector_index

In [ ]:
from llama_index.core import StorageContext, load_index_from_storage, VectorStoreIndex, Document
import os
import logging

# Khai báo biến toàn cục và hằng số
vector_index = None
DOCUMENT_STORAGE_DIR = "/content/drive/MyDrive/document_storage"
VECTOR_INDEX_STORAGE_DIR = "/content/drive/MyDrive/vector_index_storage"
logger = logging.getLogger('design_learning_path')

def initialize_vector_index() -> dict:
    """Khởi tạo VectorStoreIndex từ các nút trong đồ thị tri thức.

    Returns:
        dict: Dictionary chứa 'status', 'error_message', và 'vector_index'.
    """
    global vector_index
    try:
        # Bước 1: Kiểm tra sự tồn tại của thư mục lưu trữ
        if os.path.exists(VECTOR_INDEX_STORAGE_DIR):
            try:
                storage_context = StorageContext.from_defaults(persist_dir=VECTOR_INDEX_STORAGE_DIR)
                vector_index = load_index_from_storage(storage_context)
                logger.info("Tải VectorStoreIndex từ nơi lưu trữ thành công")
                return {
                    "status": "success",
                    "error_message": None,
                    "vector_index": vector_index
                }
            except FileNotFoundError:
                logger.warning("File lưu trữ không tồn tại, tạo VectorStoreIndex mới")
        else:
            logger.info("Thư mục lưu trữ không tồn tại, tạo VectorStoreIndex mới")

        # Bước 2: Tải tài liệu từ thư mục lưu trữ hoặc từ Neo4j
        documents = []
        if os.path.exists(DOCUMENT_STORAGE_DIR):
            reader = SimpleDirectoryReader(DOCUMENT_STORAGE_DIR)
            documents = reader.load_data()
            logger.info(f"Tải {len(documents)} tài liệu từ {DOCUMENT_STORAGE_DIR}")
        else:
            # Nếu không có thư mục lưu trữ tài liệu, truy vấn từ Neo4j
            query = f"""
            MATCH (n)
            RETURN n.{Config.PROPERTY_ID} AS id, n.{Config.PROPERTY_SANITIZED_CONCEPT} AS sanitized_concept,
                   n.{Config.PROPERTY_CONTEXT} AS context, n.{Config.PROPERTY_LEARNING_OBJECTIVE} AS learning_objective,
                   n.{Config.PROPERTY_SKILL_LEVEL} AS skill_level, n.{Config.PROPERTY_DIFFICULTY} AS difficulty
            """
            nodes = execute_cypher_query(driver, query)
            for node in nodes:
                text = f"Khái niệm: {node.get('sanitized_concept', 'Không có khái niệm')}, Ngữ cảnh: {node.get('context', 'Không có ngữ cảnh')}, Mục tiêu học tập: {node.get('learning_objective', 'Không có mục tiêu học tập')}, Trình độ kỹ năng: {node.get('skill_level', 'Không có trình độ kỹ năng')}, Độ khó: {node.get('difficulty', 'Không có độ khó')}"
                doc = Document(text=text, metadata={"node_id": node["id"]})
                documents.append(doc)
            # Lưu tài liệu vào thư mục để tái sử dụng
            os.makedirs(DOCUMENT_STORAGE_DIR, exist_ok=True)
            for i, doc in enumerate(documents):
                with open(os.path.join(DOCUMENT_STORAGE_DIR, f"doc_{i}.txt"), "w", encoding="utf-8") as f:
                    f.write(doc.text)
            logger.info(f"Lưu {len(documents)} tài liệu vào {DOCUMENT_STORAGE_DIR}")

        # Bước 3: Kiểm tra xem có tài liệu để tạo VectorStoreIndex không
        if not documents:
            logger.warning("Không có tài liệu để tạo VectorStoreIndex")
            return {
                "status": "success",
                "error_message": "Không có tài liệu để tạo VectorStoreIndex",
                "vector_index": None
            }

        # Bước 4: Tạo VectorStoreIndex từ tài liệu và lưu lại

        vector_index = VectorStoreIndex.from_documents(documents)
        vector_index.storage_context.persist(persist_dir=VECTOR_INDEX_STORAGE_DIR)

        logger.info("Khởi tạo và lưu VectorStoreIndex thành công")
        return {
            "status": "success",
            "error_message": None,
            "vector_index": vector_index
        }

    except Exception as e:
        logger.error(f"Lỗi trong initialize_vector_index: {str(e)}")
        return {
            "status": "error",
            "error_message": str(e),
            "vector_index": None
        }

# merge_properties

In [ ]:
# Hàm hợp nhất thuộc tính
def merge_properties(existing_props: Dict[str, Any], new_props: Dict[str, Any]) -> Dict[str, Any]:
    """Hợp nhất thuộc tính của hai nút, ưu tiên giá trị mới nếu hợp lệ.

    Args:
        existing_props (Dict[str, Any]): Thuộc tính hiện tại.
        new_props (Dict[str, Any]): Thuộc tính mới.

    Returns:
        Dict[str, Any]: Thuộc tính đã hợp nhất.
    """
    try:
        merged_props = existing_props.copy()
        for key, new_value in new_props.items():
            if key not in merged_props or merged_props[key] == "Not Available":
                merged_props[key] = new_value
            elif new_value != "Not Available":
                if key in [Config.PROPERTY_SEMANTIC_TAGS, Config.PROPERTY_FOCUSED_SEMANTIC_TAGS, Config.PROPERTY_PREREQUISITES]:
                    existing_list = merged_props[key].split(";") if isinstance(merged_props[key], str) else merged_props[key]
                    new_list = new_value.split(";") if isinstance(new_value, str) else new_value
                    merged_list = sorted(list(set(existing_list + new_list)))
                    merged_props[key] = ";".join(merged_list)
                else:
                    merged_props[key] = new_value
        return merged_props
    except Exception as e:
        logger.error(f"Lỗi trong merge_properties: {str(e)}")
        return existing_props




# execute_cypher_query

In [ ]:
# Hàm thực thi truy vấn Cypher
def execute_cypher_query(driver, query: str, params: Dict = None) -> List[Dict]:
    """Thực thi truy vấn Cypher trên Neo4j.

    Args:
        driver: Đối tượng driver Neo4j.
        query (str): Câu truy vấn Cypher.
        params (Dict, optional): Tham số cho truy vấn.

    Returns:
        List[Dict]: Kết quả truy vấn dưới dạng danh sách dictionary.
	Raises:
        Exception: Nếu truy vấn thất bại.
    """
    try:
        with driver.session() as session:
            result = session.run(query, params or {})
            return [record.data() for record in result]
    except Exception as e:
        logger.error(f"Lỗi thực thi truy vấn Cypher: {str(e)}")
        raise



# clean_json_response

In [ ]:
# Hàm làm sạch phản hồi JSON
def clean_json_response(response_str: str) -> str:
    """Làm sạch chuỗi JSON để đảm bảo định dạng hợp lệ.

    Args:
        response_str (str): Chuỗi JSON gốc.

    Returns:
        str: Chuỗi JSON đã được làm sạch.

    Raises:
        ValueError: Nếu không tìm thấy đối tượng JSON hợp lệ.
    """
    try:
        start = min(response_str.find('{'), response_str.find('['))
        if start == -1:
            raise ValueError("Không tìm thấy đối tượng JSON trong phản hồi")
        response_str = response_str[start:]
        end = max(response_str.rfind('}'), response_str.rfind(']')) + 1
        if end == 0:
            raise ValueError("Không tìm thấy đối tượng JSON trong phản hồi")
        response_str = response_str[:end]
        response_str = re.sub(r',\s*}', '}', response_str)
        response_str = re.sub(r',\s*]', ']', response_str)
        return response_str
    except Exception as e:
        logger.error(f"Lỗi làm sạch JSON: {str(e)}")
        raise

# load_student_profile

In [ ]:
def load_student_profile(student_id: str) -> Dict:
    """Tải hồ sơ học sinh từ Neo4j hoặc trả về mặc định nếu không tìm thấy.

    Args:
        student_id (str): ID của học sinh.

    Returns:
        Dict: Hồ sơ học sinh với các thuộc tính như student_id, learning_history, v.v.
    """
    try:
        query = """
        MATCH (s:Student {StudentID: $student_id})
        RETURN s
        """
        result = execute_cypher_query(driver, query, {"student_id": student_id})
        if result:
            student_data = result[0]["s"]
            return {
                "student_id": student_data["StudentID"],
                "learning_history": student_data.get("learning_history", "").split(",") if student_data.get("learning_history") else [],
                "current_level": float(student_data.get("current_level", 0)),
                "performance_details": student_data.get("performance_details", "").split(";") if student_data.get("performance_details") else [],
                "learning_style_preference": student_data.get("learning_style_preference", Config.DEFAULT_LEARNING_STYLE),
                "preferred_difficulty": student_data.get("preferred_difficulty", "STANDARD"),
                "time_availability": float(student_data.get("time_availability", 60)),
                "learning_speed": calculate_learning_speed(student_data.get("performance_details", "").split(";")),
                "topic_preference": extract_topic_preference(student_data.get("learning_history", "").split(",")),
                "long_term_goal": student_data.get("long_term_goal", "Not Specified")
            }
        else:
            logger.warning(f"Không tìm thấy hồ sơ cho student_id: {student_id}")
            return {
                "student_id": student_id,
                "learning_history": [],
                "current_level": 0,
                "performance_details": [],
                "learning_style_preference": Config.DEFAULT_LEARNING_STYLE,
                "preferred_difficulty": "STANDARD",
                "time_availability": 60,
                "learning_speed": 0,
                "topic_preference": "Unknown",
                "long_term_goal": "Not Specified"
            }
    except Exception as e:
        logger.error(f"Lỗi tải hồ sơ học sinh: {str(e)}")
        return {
            "student_id": student_id,
            "learning_history": [],
            "current_level": 0,
            "performance_details": [],
            "learning_style_preference": Config.DEFAULT_LEARNING_STYLE,
            "preferred_difficulty": "STANDARD",
            "time_availability": 60,
            "learning_speed": 0,
            "topic_preference": "Unknown",
            "long_term_goal": "Not Specified"
        }

# calculate_learning_speed

In [ ]:
# Hàm tính tốc độ học tập
def calculate_learning_speed(performance_details: List[str]) -> float:
    """Tính tốc độ học tập trung bình dựa trên thời gian dành cho các nút.

    Args:
        performance_details (List[str]): Danh sách chi tiết hiệu suất, định dạng 'node_id:score:time_spent:skill_level' hoặc 'node_id:score:time_spent'.

    Returns:
        float: Tốc độ học tập trung bình (phút/nút).
    """
    total_time = 0
    num_nodes = 0
    for detail in performance_details:
        try:
            parts = detail.split(":")
            if len(parts) == 4:  # Định dạng node_id:score:time_spent:skill_level
                time_spent = parts[2]
            elif len(parts) == 3:  # Định dạng node_id:score:time_spent
                time_spent = parts[2]
            else:
                raise ValueError(f"Định dạng không hợp lệ calculate_learning_speed: {detail}")
            total_time += int(time_spent)
            num_nodes += 1
        except (ValueError, IndexError) as e:
            logger.warning(f"Lỗi xử lý chi tiết hiệu suất: {detail}. Bỏ qua. ({str(e)})")
            continue
    return total_time / num_nodes if num_nodes > 0 else 0

# extract_topic_preference

In [ ]:
# Hàm xác định sở thích chủ đề
def extract_topic_preference(learning_history: List[str]) -> str:
    """Xác định sở thích chủ đề dựa trên lịch sử học tập.

    Args:
        learning_history (List[str]): Danh sách ID của các nút đã học.

    Returns:
        str: Chủ đề phổ biến nhất hoặc 'Unknown' nếu không có dữ liệu.
    """
    contexts = []
    for node_id in learning_history:
        query = f"MATCH (n {{{Config.PROPERTY_ID}: '{node_id}'}}) RETURN n.{Config.PROPERTY_CONTEXT} AS context"
        try:
            result = execute_cypher_query(driver, query)
            if result:
                contexts.append(result[0]["context"])
        except Exception as e:
            logger.warning(f"Lỗi truy vấn context cho node {node_id}: {str(e)}")
    if contexts:
        return Counter(contexts).most_common(1)[0][0]
    return "Unknown"


# check_and_load_kg

In [ ]:
from typing import Dict
import os
from neo4j import GraphDatabase
from llama_index.core import Document

DOCUMENT_STORAGE_DIR = "/content/drive/MyDrive/document_storage"

def check_and_load_kg(driver, node_file: str = Config.NODES_FILE, import_node_file: str = Config.IMPORT_NODES_FILE, import_relationship_file: str = Config.IMPORT_RELATIONSHIPS_FILE, similarity_threshold: float = 0.7) -> Dict:
    """Tải và hợp nhất đồ thị tri thức từ file CSV, sử dụng Cypher LOAD CSV để tải nút và hợp nhất dựa trên độ tương đồng Jaccard.

    Args:
        driver: Đối tượng driver Neo4j.
        node_file (str): Đường dẫn đến file nodes.csv (không sử dụng trong hàm này).
        import_node_file (str): Đường dẫn đến file nodes.csv trong thư mục import của Neo4j.
        import_relationship_file (str): Đường dẫn đến file relationships.csv trong thư mục import của Neo4j.
        similarity_threshold (float): Ngưỡng tương đồng Jaccard để hợp nhất nút.

    Returns:
        Dict: Trạng thái tải {'status': 'success', 'error_message': None, 'updated': bool}.
    """
    try:
        with driver.session() as session:
            result = session.run("MATCH (n) RETURN count(n) as node_count")
            node_count = result.single()["node_count"] if result.peek() else 0
            updated = False

            if node_count == 0:
                logger.info("Không tìm thấy đồ thị tri thức. Tải dữ liệu từ file CSV...")

            load_nodes_query = f"""
            LOAD CSV WITH HEADERS FROM 'file:///{import_node_file}' AS row
            MERGE (n:KnowledgeNode {{
                {Config.PROPERTY_ID}: row.{Config.PROPERTY_ID},
                {Config.PROPERTY_SANITIZED_CONCEPT}: row.{Config.PROPERTY_SANITIZED_CONCEPT},
                {Config.PROPERTY_CONTEXT}: row.{Config.PROPERTY_CONTEXT}
            }})
            ON CREATE SET
                n.{Config.PROPERTY_SANITIZED_CONCEPT} = row.{Config.PROPERTY_SANITIZED_CONCEPT},
                n.{Config.PROPERTY_CONTEXT} = row.{Config.PROPERTY_CONTEXT},
                n.{Config.PROPERTY_DEFINITION} = row.{Config.PROPERTY_DEFINITION},
                n.{Config.PROPERTY_EXAMPLE} = row.{Config.PROPERTY_EXAMPLE},
                n.{Config.PROPERTY_LEARNING_OBJECTIVE} = row.{Config.PROPERTY_LEARNING_OBJECTIVE},
                n.{Config.PROPERTY_SKILL_LEVEL} = row.{Config.PROPERTY_SKILL_LEVEL},
                n.{Config.PROPERTY_TIME_ESTIMATE} = toFloat(row.{Config.PROPERTY_TIME_ESTIMATE}),
                n.{Config.PROPERTY_DIFFICULTY} = row.{Config.PROPERTY_DIFFICULTY},
                n.{Config.PROPERTY_PRIORITY} = toInteger(row.{Config.PROPERTY_PRIORITY}),
                n.{Config.PROPERTY_PREREQUISITES} = row.{Config.PROPERTY_PREREQUISITES},
                n.{Config.PROPERTY_SEMANTIC_TAGS} = row.{Config.PROPERTY_SEMANTIC_TAGS},
                n.{Config.PROPERTY_FOCUSED_SEMANTIC_TAGS} = row.{Config.PROPERTY_FOCUSED_SEMANTIC_TAGS},
                n.{Config.PROPERTY_COMMON_ERRORS} = row.{Config.PROPERTY_COMMON_ERRORS},
                n.{Config.PROPERTY_LEARNING_STYLE_PREFERENCE} = row.{Config.PROPERTY_LEARNING_STYLE_PREFERENCE}
            ON MATCH SET
                n.{Config.PROPERTY_SANITIZED_CONCEPT} = row.{Config.PROPERTY_SANITIZED_CONCEPT},
                n.{Config.PROPERTY_CONTEXT} = row.{Config.PROPERTY_CONTEXT},
                n.{Config.PROPERTY_DEFINITION} = row.{Config.PROPERTY_DEFINITION},
                n.{Config.PROPERTY_EXAMPLE} = row.{Config.PROPERTY_EXAMPLE},
                n.{Config.PROPERTY_LEARNING_OBJECTIVE} = row.{Config.PROPERTY_LEARNING_OBJECTIVE},
                n.{Config.PROPERTY_SKILL_LEVEL} = row.{Config.PROPERTY_SKILL_LEVEL},
                n.{Config.PROPERTY_TIME_ESTIMATE} = toFloat(row.{Config.PROPERTY_TIME_ESTIMATE}),
                n.{Config.PROPERTY_DIFFICULTY} = row.{Config.PROPERTY_DIFFICULTY},
                n.{Config.PROPERTY_PRIORITY} = toInteger(row.{Config.PROPERTY_PRIORITY}),
                n.{Config.PROPERTY_PREREQUISITES} = row.{Config.PROPERTY_PREREQUISITES},
                n.{Config.PROPERTY_SEMANTIC_TAGS} = row.{Config.PROPERTY_SEMANTIC_TAGS},
                n.{Config.PROPERTY_FOCUSED_SEMANTIC_TAGS} = row.{Config.PROPERTY_FOCUSED_SEMANTIC_TAGS},
                n.{Config.PROPERTY_COMMON_ERRORS} = row.{Config.PROPERTY_COMMON_ERRORS},
                n.{Config.PROPERTY_LEARNING_STYLE_PREFERENCE} = row.{Config.PROPERTY_LEARNING_STYLE_PREFERENCE}
            """
            session.run(load_nodes_query)
            logger.info("Tải các nút từ CSV thành công")

            # Thực hiện hợp nhất nút dựa trên semantic_tags và Context để tránh hợp nhất không mong muốn
            nodes_query = f"""
            MATCH (n:KnowledgeNode)
            RETURN n.{Config.PROPERTY_ID} AS node_id, n.{Config.PROPERTY_SEMANTIC_TAGS} AS semantic_tags, n.{Config.PROPERTY_CONTEXT} AS context, properties(n) AS props
            """
            nodes = session.run(nodes_query)
            nodes_list = list(nodes)

            for i, node in enumerate(nodes_list):
                node_id = node["node_id"]

                semantic_tags = node["semantic_tags"].split(";")
                context = node["context"]
                node_props = node["props"]

                if context and context.strip():
                    existing_nodes_query = f"""
                    MATCH (n:KnowledgeNode)
                    WHERE n.{Config.PROPERTY_ID} <> $node_id AND n.{Config.PROPERTY_CONTEXT} = $context
                    RETURN n.{Config.PROPERTY_ID} AS id, n.{Config.PROPERTY_SEMANTIC_TAGS} AS tags, properties(n) AS props
                    """
                    params = {"node_id": node_id, "context": context}
                else:
                    existing_nodes_query = f"""
                    MATCH (n:KnowledgeNode)
                    WHERE n.{Config.PROPERTY_ID} <> $node_id
                    RETURN n.{Config.PROPERTY_ID} AS id, n.{Config.PROPERTY_SEMANTIC_TAGS} AS tags, properties(n) AS props
                    """
                    params = {"node_id": node_id}

                existing_nodes = session.run(existing_nodes_query, params)
                best_match = None
                best_similarity = 0.0
                for existing_node in existing_nodes:
                    existing_tags = existing_node["tags"].split(";")
                    similarity = jaccard_similarity(semantic_tags, existing_tags)
                    if similarity > best_similarity and similarity >= similarity_threshold:
                        best_similarity = similarity
                        best_match = existing_node

                if best_match:
                    merged_props = merge_properties(best_match["props"], node_props)
                    session.run(
                        f"MATCH (n {{{Config.PROPERTY_ID}: $existing_node_id}}) SET n += $merged_props",
                        existing_node_id=best_match["id"],
                        merged_props=merged_props
                    )

                    # Chuyển các mối quan hệ outgoing
                    outgoing_rels = session.run(
                        f"""
                        MATCH (n {{{Config.PROPERTY_ID}: $new_node_id}})-[r]->(target)
                        RETURN type(r) AS rel_type, r.Weight AS weight, r.Dependency AS dependency, target.{Config.PROPERTY_ID} AS target_id
                        """,
                        new_node_id=node_id
                    )
                    for rel in outgoing_rels:
                        rel_type = rel["rel_type"]
                        session.run(
                            f"""
                            MATCH (existing {{{Config.PROPERTY_ID}: $existing_node_id}})
                            MATCH (target {{{Config.PROPERTY_ID}: $target_id}})
                            MERGE (existing)-[r:{rel_type} {{Weight: $weight, Dependency: $dependency}}]->(target)
                            """,
                            existing_node_id=best_match["id"],
                            target_id=rel["target_id"],
                            weight=rel["weight"],
                            dependency=rel["dependency"]
                        )
                    session.run(
                        f"MATCH (n {{{Config.PROPERTY_ID}: $new_node_id}})-[r]->() DELETE r",
                        new_node_id=node_id
                    )

                    # Chuyển các mối quan hệ incoming
                    incoming_rels = session.run(
                        f"""
                        MATCH (source)-[r]->(n {{{Config.PROPERTY_ID}: $new_node_id}})
                        RETURN type(r) AS rel_type, r.Weight AS weight, r.Dependency AS dependency, source.{Config.PROPERTY_ID} AS source_id
                        """,
                        new_node_id=node_id
                    )
                    for rel in incoming_rels:
                        rel_type = rel["rel_type"]
                        session.run(
                            f"""
                            MATCH (source {{{Config.PROPERTY_ID}: $source_id}})
                            MATCH (existing {{{Config.PROPERTY_ID}: $existing_node_id}})
                            MERGE (source)-[r:{rel_type} {{Weight: $weight, Dependency: $dependency}}]->(existing)
                            """,
                            source_id=rel["source_id"],
                            existing_node_id=best_match["id"],
                            weight=rel["weight"],
                            dependency=rel["dependency"]
                        )
                    session.run(
                        f"MATCH ()-[r]->(n {{{Config.PROPERTY_ID}: $new_node_id}}) DELETE r",
                        new_node_id=node_id
                    )

                    # Xóa nút cũ sau khi hợp nhất
                    session.run(
                        f"MATCH (n {{{Config.PROPERTY_ID}: $node_id}}) DETACH DELETE n",
                        node_id=node_id
                    )
                else:
                    updated = True

            # Tải mối quan hệ từ file CSV
            load_rels_query = f"""
            LOAD CSV WITH HEADERS FROM 'file:///{import_relationship_file}' AS row
            MATCH (source {{Node_ID: row.Source_Node_ID}})
            MATCH (target {{Node_ID: row.Target_Node_ID}})
            CALL apoc.merge.relationship(source, row.Relationship_Type, {{}}, {{Weight: toFloat(row.Weight), Dependency: toFloat(row.Dependency)}}, target) YIELD rel
            RETURN rel
            """
            session.run(load_rels_query)
            logger.info("Tải mối quan hệ từ CSV thành công")

            # Kiểm tra và xóa các mối quan hệ tự tham chiếu (self-loop) để tránh chu trình
            remove_self_loops_query = """
            MATCH (n)-[r]->(n)
            DELETE r
            """
            session.run(remove_self_loops_query)
            logger.info("Đã xóa các mối quan hệ tự tham chiếu để ngăn chu trình")

        if updated:
            os.makedirs(DOCUMENT_STORAGE_DIR, exist_ok=True)
            query = f"""
            MATCH (n)
            RETURN n.{Config.PROPERTY_ID} AS id, n.{Config.PROPERTY_SANITIZED_CONCEPT} AS sanitized_concept,
                    n.{Config.PROPERTY_CONTEXT} AS context, n.{Config.PROPERTY_LEARNING_OBJECTIVE} AS learning_objective,
                    n.{Config.PROPERTY_SKILL_LEVEL} AS skill_level, n.{Config.PROPERTY_DIFFICULTY} AS difficulty
            """
            nodes = execute_cypher_query(driver, query)
            for i, node in enumerate(nodes):
                text = f"Khái niệm: {node.get('sanitized_concept', 'Không có khái niệm')}, Ngữ cảnh: {node.get('context', 'Không có ngữ cảnh')}, Mục tiêu học tập: {node.get('learning_objective', 'Không có mục tiêu học tập')}, Trình độ kỹ năng: {node.get('skill_level', 'Không có trình độ kỹ năng')}, Độ khó: {node.get('difficulty', 'Không có độ khó')}"
                with open(os.path.join(DOCUMENT_STORAGE_DIR, f"doc_{i}.txt"), "w", encoding="utf-8") as f:
                    f.write(text)
            logger.info(f"Cập nhật {len(nodes)} tài liệu trong {DOCUMENT_STORAGE_DIR}")
            initialize_vector_index()

        global kg_index
        kg_index = PropertyGraphIndex.from_existing(
            property_graph_store=graph_store,
            embed_kg_nodes=True,
            include_embeddings=True
        )
        kg_index.storage_context.persist(persist_dir=Config.PROPERTY_GRAPH_STORAGE_DIR)
        logger.info("Cập nhật PropertyGraphIndex thành công")

        logger.info("Cập nhật đồ thị tri thức, tài liệu, và PropertyGraphIndex thành công")
        return {"status": "success", "error_message": None, "updated": updated}

    except Exception as e:
        logger.error(f"Lỗi trong check_and_load_kg: {str(e)}")
        return {"status": "error", "error_message": str(e), "updated": False}


# check_and_load_students

In [ ]:
def check_and_load_students(driver, student_file: str = Config.STUDENT_FILE) -> Dict:
    """Tải và cập nhật nút Student từ file CSV vào đồ thị Neo4j, ưu tiên dữ liệu có LastUpdated lớn nhất.

    Args:
        driver: Đối tượng driver Neo4j.
        student_file (str): Đường dẫn đến file CSV chứa thông tin học sinh.

    Returns:
        Dict: Trạng thái tải {'status': 'success', 'error_message': None, 'updated': bool}.
    """
    try:
        with driver.session() as session:
            # Đọc dữ liệu từ CSV
            students_df = pd.read_csv(student_file, quoting=csv.QUOTE_ALL)
            students_dict = students_df.to_dict("records")

            updated = False
            for student in students_dict:
                student_id = student["StudentID"]
                last_updated_csv_str = student.get("LastUpdated", None)

                # Chuyển đổi LastUpdated từ CSV sang định dạng datetime
                if last_updated_csv_str:
                    try:
                        last_updated_csv = datetime.fromisoformat(last_updated_csv_str)
                    except ValueError:
                        logger.warning(f"Định dạng LastUpdated không hợp lệ trong CSV cho student_id: {student_id}")
                        last_updated_csv = datetime.now()
                else:
                    last_updated_csv = datetime.now()


                # Kiểm tra nút Student đã tồn tại
                existing_node = session.run(
                    f"MATCH (s:Student {{StudentID: $student_id}}) RETURN s",
                    student_id=student_id
                ).single()

                if existing_node:
                    existing_student = dict(existing_node["s"])
                    last_updated_db_str = existing_student.get("LastUpdated", None)

                    # Chuyển đổi LastUpdated từ đồ thị sang datetime
                    if last_updated_db_str:
                        try:
                            last_updated_db = datetime.fromisoformat(last_updated_db_str)
                        except ValueError:
                            logger.warning(f"Định dạng LastUpdated không hợp lệ trong đồ thị cho student_id: {student_id}")
                            last_updated_db = None
                    else:
                        last_updated_db = None


                    # So sánh LastUpdated
                    if last_updated_csv and (not last_updated_db or last_updated_csv > last_updated_db):
                        # Cập nhật thuộc tính với dữ liệu mới nhất từ CSV
                        student_props = {
                            "learning_history": student.get("Learning_History", ""),
                            "current_level": float(student.get("Current_Level", 0)),
                            "performance_details": student.get("Performance_Details", ""),
                            "learning_style_preference": student.get("Learning_Style_Preference", Config.DEFAULT_LEARNING_STYLE),
                            "preferred_difficulty": student.get("Preferred_Difficulty", "STANDARD"),
                            "time_availability": float(student.get("Time_Availability", 60)),
                            "long_term_goal": student.get("Long_Term_Goal", "Not Specified"),
                            "LastUpdated": last_updated_csv.isoformat()
                        }
                        session.run(
                            f"MATCH (s:Student {{StudentID: $student_id}}) SET s += $props",
                            student_id=student_id,
                            props=student_props
                        )
                        updated = True
                    else:
                        # Giữ nguyên dữ liệu trong đồ thị
                        logger.info(f"Giữ nguyên dữ liệu cho student_id: {student_id} do LastUpdated không mới hơn")
                else:
                    # Tạo nút Student mới
                    student_props = {
                        "StudentID": student_id,
                        "learning_history": student.get("Learning_History", ""),
                        "current_level": float(student.get("Current_Level", 0)),
                        "performance_details": student.get("Performance_Details", ""),
                        "learning_style_preference": student.get("Learning_Style_Preference", Config.DEFAULT_LEARNING_STYLE),
                        "preferred_difficulty": student.get("Preferred_Difficulty", "STANDARD"),
                        "time_availability": float(student.get("Time_Availability", 60)),
                        "long_term_goal": student.get("Long_Term_Goal", "Not Specified"),
                        "LastUpdated": last_updated_csv.isoformat() if last_updated_csv else "Unknown"
                    }
                    session.run(
                        "CREATE (s:Student $props)",
                        props=student_props
                    )
                    updated = True

            logger.info("Cập nhật nút Student thành công")
            return {"status": "success", "error_message": None, "updated": updated}

    except Exception as e:
        logger.error(f"Lỗi trong check_and_load_students: {str(e)}")
        return {"status": "error", "error_message": str(e), "updated": False}

In [ ]:
def check_graph_before():
    """Kiểm tra các nút và quan hệ trong đồ thị trước khi tải."""
    # Kiểm tra nút
    query_nodes = """
    MATCH (n)
    RETURN n.Node_ID, n.Sanitized_Concept, n.Semantic_Tags
    """
    try:
        nodes = execute_cypher_query(driver, query_nodes)
        print("Nodes in graph before loading:")
        for node in nodes:
            print(node)
    except Exception as e:
        logger.error(f"Lỗi khi kiểm tra nút trước khi tải: {str(e)}")
        raise

    # Kiểm tra mối quan hệ
    query_rels = """
    MATCH (source)-[r]->(target)
    RETURN source.Node_ID AS source_id, type(r) AS rel_type, target.Node_ID AS target_id,
           coalesce(r.Weight, 0.0) AS weight, coalesce(r.Dependency, 0.0) AS dependency
    """
    try:
        relationships = execute_cypher_query(driver, query_rels)
        print("Relationships in graph before loading:")
        for rel in relationships:
            print(rel)
    except Exception as e:
        logger.error(f"Lỗi khi kiểm tra quan hệ trước khi tải: {str(e)}")
        raise

def check_graph_after():
    """Kiểm tra các nút và quan hệ trong đồ thị sau khi tải."""
    # Kiểm tra nút
    query_nodes = """
    MATCH (n)
    RETURN n.Node_ID, n.Sanitized_Concept, n.Semantic_Tags
    """
    try:
        nodes = execute_cypher_query(driver, query_nodes)
        print("Nodes in graph after loading:")
        for node in nodes:
            print(node)
    except Exception as e:
        logger.error(f"Lỗi khi kiểm tra nút sau khi tải: {str(e)}")
        raise

    # Kiểm tra mối quan hệ
    query_rels = """
    MATCH (source)-[r]->(target)
    RETURN source.Node_ID AS source_id, type(r) AS rel_type, target.Node_ID AS target_id,
           coalesce(r.Weight, 0.0) AS weight, coalesce(r.Dependency, 0.0) AS dependency
    """
    try:
        relationships = execute_cypher_query(driver, query_rels)
        print("Relationships in graph after loading:")
        for rel in relationships:
            print(rel)
    except Exception as e:
        logger.error(f"Lỗi khi kiểm tra quan hệ sau khi tải: {str(e)}")
        raise

def verify_graph(driver):
    """Kiểm tra các nút và mối quan hệ trong đồ thị Neo4j."""
    try:
        # Kiểm tra số lượng nút
        result = driver.session().run("MATCH (n) RETURN count(n) AS node_count")
        node_count = result.single()["node_count"]
        logger.info(f"Số lượng nút trong đồ thị: {node_count}")

        # Kiểm tra số lượng mối quan hệ
        result = driver.session().run("MATCH ()-[r]->() RETURN count(r) AS relationship_count")
        relationship_count = result.single()["relationship_count"]
        logger.info(f"Số lượng mối quan hệ trong đồ thị: {relationship_count}")
        print(f"Kiểm tra thành công.\nSố lượng nút trong đồ thị: {node_count}\nSố lượng mối quan hệ trong đồ thị: {relationship_count}")
    except Exception as e:
        logger.error(f"Lỗi khi kiểm tra đồ thị: {str(e)}")
        raise

In [ ]:
# # Tải đồ thị tri thức
# try:
#     check_graph_before()
#     result = check_and_load_kg(driver)
#     if result["status"] == "success":
#         logger.info("Tải đồ thị tri thức thành công")
#         verify_graph(driver)
#         check_graph_after()
#     else:
#         logger.error(f"Tải đồ thị tri thức thất bại: {result['error_message']}")
#         raise Exception(result["error_message"])
# except Exception as e:
#     logger.error(f"Lỗi khởi tạo đồ thị tri thức: {str(e)}")
#     raise

In [ ]:
# check_and_load_kg(driver)

In [ ]:
 verify_graph(driver)

Kiểm tra thành công.
Số lượng nút trong đồ thị: 143
Số lượng mối quan hệ trong đồ thị: 167


# expand_tags

In [ ]:
def expand_tags(tags: List[str]) -> List[str]:
    """Mở rộng danh sách tag với từ đồng nghĩa từ WordNet và chuẩn hóa thành lowercase với dấu '_'.

    Args:
        tags (List[str]): Danh sách tag ngữ nghĩa.

    Returns:
        List[str]: Danh sách tag đã được mở rộng và chuẩn hóa.
    """
    try:
        expanded = set()
        for tag in tags:
            # Chuẩn hóa tag ban đầu thành lowercase và giữ dấu '_'
            normalized_tag = tag.lower()
            expanded.add(normalized_tag)

            # Tách các từ trong tag nếu có dấu '_'
            words = normalized_tag.split('_')

            for word in words:
                for syn in wordnet.synsets(word):
                    for lemma in syn.lemmas():
                        # Chuẩn hóa lemma thành lowercase và thay khoảng trắng bằng '_'
                        lemma_name = lemma.name().lower().replace(' ', '_')
                        expanded.add(lemma_name)
        return list(expanded)
    except Exception as e:
        logger.error(f"Lỗi trong expand_tags: {str(e)}")
        return tags

# update_heuristic_based_on_assessment

In [ ]:
# Hàm cập nhật heuristic dựa trên kết quả bài kiểm tra
def update_heuristic_based_on_assessment(assessment_history: Dict) -> None:
    """Cập nhật trọng số heuristic dựa trên kết quả bài kiểm tra.

    Args:
        assessment_history (Dict): Lịch sử bài kiểm tra với avg_score.
    """
    try:
        if assessment_history["avg_score"] < 50:
            Config.ASTAR_HEURISTIC_WEIGHTS["difficulty_standard"] += 0.05
            logger.info("Tăng trọng số difficulty_standard do điểm trung bình thấp")
    except Exception as e:
        logger.error(f"Lỗi trong update_heuristic_based_on_assessment: {str(e)}")

# generate_goal_tags

In [ ]:
import re
# Hàm chuẩn hóa tag thành snake_case
def to_snake_case(tag):
    # Loại bỏ khoảng trắng thừa
    tag = tag.strip()
    # Chuyển thành lowercase
    tag = tag.lower()
    # Thay khoảng trắng, dấu gạch nối, hoặc ký tự đặc biệt bằng '_'
    tag = re.sub(r'[\s-]+', '_', tag)
    # Loại bỏ các ký tự không phải chữ, số, hoặc '_'
    tag = re.sub(r'[^a-z0-9_]', '', tag)
    # Loại bỏ nhiều dấu '_' liên tiếp
    tag = re.sub(r'_+', '_', tag)
    # Loại bỏ dấu '_' ở đầu hoặc cuối
    tag = tag.strip('_')
    return tag if tag else 'unknown'  # Trả về 'unknown' nếu tag rỗng

In [ ]:
# def generate_goal_tags(student_goal, context, llm,num):
#     """
#     Sử dụng LLM để tạo danh sách các tag liên quan đến student_goal và context.

#     Args:
#         student_goal (str): Mục tiêu học tập của học sinh.
#         context (str): Ngữ cảnh bổ sung (nếu có).
#         llm: Đối tượng LLM từ llama_index hoặc OpenAI để tạo nội dung.

#     Returns:
#         list: Danh sách các tag dưới dạng chuỗi.
#     """
#     prompt = (
#         f"Dựa trên mục tiêu học tập của học sinh: '{student_goal}' "
#         f"và ngữ cảnh: '{context if context else 'không có'}', "
#         "hãy tạo một danh sách các tag liên quan đến mục tiêu này. "
#         "Tag nên là các từ khóa mô tả các khái niệm hoặc chủ đề chính, "
#         "phù hợp để tìm kiếm các nội dung học tập liên quan. "
#         f"Hãy tạo chính xác {num} tag, bao gồm:\n\n"
#         "- **Core Tags**: Từ đồng nghĩa và ý nghĩa cốt lõi của mục tiêu học tập.\n"
#         "- **Contextual Tags**: Các khái niệm và ứng dụng liên quan đến mục tiêu và ngữ cảnh.\n"
#         "- **Extended Tags**: Các liên kết lý thuyết và liên ngành rộng hơn.\n\n"
#         f"Nếu không đủ {num} tag, hãy mở rộng bằng cách sử dụng ontologies giáo dục như WordNet hoặc DBpedia. "
#         f"Nếu có nhiều hơn {num} tag, hãy ưu tiên các tag liên quan nhất đến mục tiêu và ngữ cảnh, "
#         "và loại bỏ các tag chung chung hoặc không liên quan.\n\n"
#         "Cuối cùng, hãy sắp xếp các tag theo thứ tự alphabet và trả về dưới dạng danh sách phân cách bởi dấu chấm phẩy (;)."
#     )


#     # Gọi LLM để tạo phản hồi
#     response = llm.complete(prompt).text

#     # Chuyển đổi phản hồi thành danh sách tag
#     tags = [to_snake_case(tag) for tag in response.split(";")]
#     return tags

In [ ]:
def generate_goal_tags(student_goal, context, llm, num=30):
    """
    Sử dụng LLM để tạo danh sách các tag liên quan đến student_goal và context.

    Args:
        student_goal (str): Mục tiêu học tập của học sinh.
        context (str): Ngữ cảnh bổ sung (nếu có).
        llm: Đối tượng LLM từ llama_index hoặc OpenAI để tạo nội dung.

    Returns:
        list: Danh sách các tag dưới dạng chuỗi.
    """
    prompt = (
        f"Dựa trên mục tiêu học tập của học sinh: '{student_goal}' "
        f"và ngữ cảnh: '{context if context else 'không có'}', "
        "hãy tạo một danh sách các tag liên quan đến mục tiêu này. "
        "Tag nên là các từ khóa hoặc cụm từ ngắn gọn mô tả các khái niệm hoặc chủ đề chính, "
        "phù hợp để tìm kiếm các nội dung học tập liên quan. "
        "Hãy tập trung vào việc tạo ra các tag liên quan nhất và hữu ích nhất, "
        "bao gồm:\n\n"
        "- **Core Tags**: Từ đồng nghĩa và ý nghĩa cốt lõi của mục tiêu học tập.\n"
        "- **Contextual Tags**: Các khái niệm và ứng dụng liên quan đến mục tiêu và ngữ cảnh.\n"
        "- **Extended Tags**: Các liên kết lý thuyết và liên ngành rộng hơn.\n\n"
        "Nếu cần thiết, hãy sử dụng ontologies giáo dục như WordNet hoặc DBpedia để bổ sung ý nghĩa cho danh sách tag. "
        "Hãy ưu tiên các tag cụ thể, có tính ứng dụng cao và liên quan chặt chẽ đến mục tiêu và ngữ cảnh, "
        "đồng thời loại bỏ các tag quá chung chung, không cần thiết hoặc không liên quan.\n\n"
        "**Lưu ý quan trọng**:\n"
        "- Mỗi tag phải là một phần tử riêng biệt, ngắn gọn và dễ hiểu (ví dụ: 'business_intelligence', 'data_analysis').\n"
        "- Không ghép nhiều tag thành một chuỗi dài.\n"
        "- Kiểm tra và loại bỏ các tag trùng lặp hoặc không cần thiết.\n\n"
        "Cuối cùng, hãy sắp xếp các tag theo thứ tự alphabet và trả về dưới dạng danh sách phân cách bởi dấu chấm phẩy (;). "
        "Ví dụ: 'business_intelligence; data_analysis; machine_learning'."
    )

    # Gọi LLM để tạo phản hồi
    response = llm.complete(prompt).text

    # Chuyển đổi phản hồi thành danh sách tag
    tags = [to_snake_case(tag.strip()) for tag in response.split(";")]
    return tags

In [ ]:
generate_goal_tags(student_goal='Business Intelligence Systems Master', context='',llm=Settings.llm, num=30)

['business_intelligence',
 'data_analysis',
 'decision_support_systems',
 'data_visualization',
 'data_mining',
 'predictive_analytics',
 'big_data',
 'business_analytics',
 'information_systems',
 'data_management',
 'reporting_tools',
 'data_integration',
 'data_warehousing',
 'data_modeling',
 'machine_learning',
 'artificial_intelligence',
 'data_driven_decision_making',
 'data_quality',
 'data_security',
 'data_architecture',
 'data_processing',
 'data_storage',
 'data_analytics',
 'data_science',
 'business_strategy',
 'competitive_analysis',
 'market_research',
 'data_driven_insights',
 'performance_metrics']

# determine_start_node

In [ ]:
init_vector_result = initialize_vector_index()

In [ ]:
display_log_content(log_file)

In [ ]:
determine_start_node(student_id="stu001", level="Remember", context="", student_goal="Business Intelligence Systems advanced level")

In [ ]:
def determine_start_node(student_id: str, level: str, context: str, student_goal: str = None, student_file: str = "students.csv", skip_quiz: bool = False) -> Dict:
    """Xác định điểm bắt đầu cho học sinh dựa trên trình độ, ngữ cảnh và mục tiêu học tập.

    Args:
        student_id (str): ID của học sinh.
        level (str): Trình độ hiện tại (e.g., 'Remember', 'Understand').
        context (str): Ngữ cảnh học tập (e.g., 'e_learning').
        student_goal (str, optional): Mục tiêu học tập (e.g., 'sql basics').
        student_file (str): Đường dẫn đến file hồ sơ học sinh.
        skip_quiz (bool): Bỏ qua bài kiểm tra nếu True.

    Returns:
        Dict: {'start_node': str, 'status': str, 'error_message': str or None}
    """
    global vector_index, driver, Settings, Config
    global_start_node = None

    try:
        # Validate inputs
        if not student_goal and not context:
            raise ValueError("Phải cung cấp ít nhất một trong student_goal hoặc context")
        query_txt = f"Mục tiêu học tập: {student_goal or context}"
        if level not in Config.ASTAR_SKILL_LEVELS_LOW + Config.ASTAR_SKILL_LEVELS_HIGH:
            raise ValueError(f"Trình độ không hợp lệ: {level}")

        # Load student profile
        profile = load_student_profile(student_id)
        current_level = profile.get("current_level", 0)
        learned_nodes = profile.get("learning_history", [])

        # Logic mới cho skill_levels
        if current_level <= Config.ASTAR_CURRENT_LEVEL_THRESHOLD:
            skill_levels = Config.ASTAR_SKILL_LEVELS_LOW
        else:
            # Lấy tất cả cấp độ từ LOW và HIGH
            all_levels = Config.ASTAR_SKILL_LEVELS_LOW + Config.ASTAR_SKILL_LEVELS_HIGH
            skill_weights = Config.ASTAR_HEURISTIC_WEIGHTS["skill_level"]
            level_weight = skill_weights.get(level, skill_weights["default"])
            # Lọc các cấp độ có trọng số nhỏ hơn hoặc bằng level đầu vào
            skill_levels = [
                skill for skill in all_levels
                if skill_weights.get(skill, skill_weights["default"]) <= level_weight
            ]

        # Ensure skill_levels is defined
        if not skill_levels:
            logger.error("Skill levels not defined. Check Config.ASTAR_SKILL_LEVELS_LOW and Config.ASTAR_SKILL_LEVELS_HIGH.")
            return {"status": "error", "error_message": "Skill levels not defined"}

        # Step 1: Tìm kiếm ngữ nghĩa với VectorStoreIndex
        potential_nodes = []
        if vector_index:
            query_engine = vector_index.as_query_engine(similarity_top_k=20)
            query_txt = f"Mục tiêu học tập: {student_goal or context}. Trình độ: {level}. Ngữ cảnh: {context or 'không có'}"
            response = query_engine.query(query_txt)
            potential_nodes = [
                doc.metadata.get("node_id")
                for doc in response.source_nodes
                if doc.metadata.get("node_id") and doc.metadata.get("node_id") != "Không có ID"
                and doc.metadata.get("skill_level") in skill_levels
            ]
            logger.info(f"Potential nodes from vector index: {potential_nodes}")
        else:
            logger.warning("Vector index not initialized.")

        # Step 2: Truy vấn đồ thị với Cypher query trực tiếp
        candidates = []
        goal_tags = generate_goal_tags(student_goal=student_goal, context=context, llm=Settings.llm, num=30)
        logger.info(f"goal_tags: {goal_tags}")
        goal_tags_str = ', '.join([f"'{tag}'" for tag in goal_tags])
        logger.info(f"goal_tags_str: {goal_tags_str}")

        # Xây dựng truy vấn Cypher linh hoạt dựa trên potential_nodes
        if potential_nodes:
            node_list = ', '.join([f"'{node}'" for node in potential_nodes])
            cypher_query = f"""
            MATCH (n:KnowledgeNode)
            WHERE n.Node_ID IN [{node_list}]
            AND (
                ANY(tag IN {goal_tags} WHERE tag IN split(n.Semantic_Tags, ';'))
                OR ANY(tag IN {goal_tags} WHERE tag IN split(n.Sanitized_Concept, ';'))
                OR n.Learning_Objective CONTAINS '{student_goal or ''}'
            )
            AND n.Skill_Level IN {skill_levels}
            """
        else:
            cypher_query = f"""
            MATCH (n:KnowledgeNode)
            WHERE (
                ANY(tag IN {goal_tags} WHERE tag IN split(n.Semantic_Tags, ';'))
                OR ANY(tag IN {goal_tags} WHERE tag IN split(n.Sanitized_Concept, ';'))
                OR n.Learning_Objective CONTAINS '{student_goal or ''}'
            )
            AND n.Skill_Level IN {skill_levels}
            """

        # Thêm điều kiện context nếu context không rỗng
        if context:
            cypher_query += f" AND n.Context = '{context}'"

        cypher_query += """
        AND EXISTS ((n)-[:IS_PREREQUISITE_OF|IS_SUBCONCEPT_OF|NEXT]->())
        RETURN n.Node_ID AS id,
               n.Sanitized_Concept AS sanitized_concept,
               n.Learning_Objective AS learning_objective,
               n.Skill_Level AS skill_level,
               n.Priority AS priority,
               n.Time_Estimate AS time_estimate,
               n.Difficulty AS difficulty,
               n.Semantic_Tags AS semantic_tags
        """

        # Thực hiện truy vấn Cypher
        result = execute_cypher_query(driver=driver, query=cypher_query)
        logger.info(f"result: {result}")
        candidates = [record for record in result]
        logger.info(f"candidates_1: {candidates}")


        #Step 3: Nếu không có candidates, lấy nút đi vào
        if not candidates:
            logger.info("No candidates with outgoing relationships. Querying incoming nodes.")
            incoming_query = f"""
            MATCH (n:KnowledgeNode)<-[:IS_PREREQUISITE_OF|IS_SUBCONCEPT_OF|NEXT]-(incoming:KnowledgeNode)
            WHERE n.Skill_Level IN $skill_levels
            AND EXISTS ((incoming)-[:IS_PREREQUISITE_OF|IS_SUBCONCEPT_OF|NEXT]->())
            """
            if context:
                incoming_query += f" AND n.Context = '{context}'"
            incoming_query += """
            RETURN incoming.Node_ID AS id,
                   incoming.Sanitized_Concept AS sanitized_concept,
                   incoming.Learning_Objective AS learning_objective,
                   incoming.Skill_Level AS skill_level,
                   incoming.Priority AS priority,
                   incoming.Time_Estimate AS time_estimate,
                   incoming.Difficulty AS difficulty,
                   incoming.Semantic_Tags AS semantic_tags
            """
            result = execute_cypher_query(driver=driver, query=incoming_query, params={"skill_levels": skill_levels})
            candidates = [record for record in result]


        # Step 4: Fallback nếu không có candidates
        if not candidates:
            # Fallback 1: Lấy nodes theo level và context (nếu có)
            query = f"""
            MATCH (n:KnowledgeNode)
            WHERE n.Skill_Level IN {skill_levels}
            AND EXISTS ((n)-[:IS_PREREQUISITE_OF|IS_SUBCONCEPT_OF|NEXT]->())
            """
            if context:
                query += f" AND n.Context = '{context}'"
            query += """
            RETURN n.Node_ID AS id,
                  n.Sanitized_Concept AS sanitized_concept,
                  n.Learning_Objective AS learning_objective,
                  n.Skill_Level AS skill_level,
                  n.Priority AS priority,
                  n.Time_Estimate AS time_estimate,
                  n.Difficulty AS difficulty,
                  n.Semantic_Tags AS semantic_tags
            ORDER BY n.Priority ASC
            """
            candidates = execute_cypher_query(driver, query, params={"level": level})
            if candidates:
                logger.info(f"candidates_in_Fallback_1: {candidates}")
            if not candidates:
                # Fallback 2: Lấy nodes theo context (nếu có)
                query = """
                MATCH (n:KnowledgeNode)
                WHERE EXISTS ((n)-[:IS_PREREQUISITE_OF|IS_SUBCONCEPT_OF|NEXT]->())
                """
                if context:
                    query += f" AND n.Context = '{context}'"
                query += f"""
                RETURN n.Node_ID AS id,
                      n.Sanitized_Concept AS sanitized_concept,
                      n.Learning_Objective AS learning_objective,
                      n.Skill_Level AS skill_level,
                      n.Priority AS priority,
                      n.Time_Estimate AS time_estimate,
                      n.Difficulty AS difficulty,
                      n.Semantic_Tags AS semantic_tags
                WHERE n.Skill_Level IN {skill_levels}
                """
                # ORDER BY n.Priority ASC
                # LIMIT 10
                candidates = execute_cypher_query(driver, query)
                if candidates:
                    logger.info(f"candidates_in_Fallback_2: {candidates}")

        # Step 4: Lọc bổ sung với Jaccard similarity
        if candidates:
            similarities = []
            for node in candidates:
                semantic_tags = node.get("semantic_tags", "").split(";")
                similarity = jaccard_similarity(goal_tags, semantic_tags)
                node["similarity"] = similarity
                similarities.append(similarity)

            threshold = 0.7
            filtered_candidates = [node for node in candidates if node["similarity"] >= threshold]

            if not filtered_candidates:
                threshold_DYM = max(min(max(similarities) * 0.8, 0.7), 0.1) if similarities else 0.3
                filtered_candidates = [node for node in candidates if node["similarity"] >= threshold_DYM]

                if not filtered_candidates:
                    logger.error("Không có nút nào trong ngưỡng. Chọn top 5 nút có similarity cao nhất")
                    filtered_candidates = sorted(candidates, key=lambda x: x["similarity"], reverse=True)[:5]
            logger.info(f"similarities: {similarities}")

        else:
            # Fallback chung nếu không có candidates
            query = f"""
            MATCH (n:KnowledgeNode)
            WHERE EXISTS ((n)-[:IS_PREREQUISITE_OF|IS_SUBCONCEPT_OF|NEXT]->())
            AND n.Skill_Level IN {skill_levels}
            """
            if context:
                query += f" AND n.Context = '{context}'"
            else:
                query += " AND true"

            query += """
            RETURN n.{id_prop} AS id,
                  n.{concept_prop} AS sanitized_concept,
                  n.{objective_prop} AS learning_objective,
                  n.{skill_prop} AS skill_level,
                  n.{priority_prop} AS priority,
                  n.{time_prop} AS time_estimate,
                  n.{difficulty_prop} AS difficulty,
                  n.{tags_prop} AS semantic_tags
            ORDER BY n.{priority_prop} ASC
            LIMIT 10
            """.format(
                id_prop=Config.PROPERTY_ID,
                concept_prop=Config.PROPERTY_SANITIZED_CONCEPT,
                objective_prop=Config.PROPERTY_LEARNING_OBJECTIVE,
                skill_prop=Config.PROPERTY_SKILL_LEVEL,
                priority_prop=Config.PROPERTY_PRIORITY,
                time_prop=Config.PROPERTY_TIME_ESTIMATE,
                difficulty_prop=Config.PROPERTY_DIFFICULTY,
                tags_prop=Config.PROPERTY_SEMANTIC_TAGS,
                skill_levels=skill_levels
            )
            filtered_candidates = execute_cypher_query(driver, query)

        filtered_nodes = [node for node in filtered_candidates if node["id"] not in learned_nodes]
        candidates = filtered_nodes

        if not candidates:
            logger.info("No candidates found after filtering learned nodes. Using priority-based fallback.")
            query = f"""
            MATCH (n:KnowledgeNode)
            WHERE n.Skill_Level IN {skill_levels}
            AND EXISTS ((n)-[:IS_PREREQUISITE_OF|IS_SUBCONCEPT_OF|NEXT]->())
            """
            if context:
                query += f" AND n.Context = '{context}'"
            query += """
            RETURN n.Node_ID AS id,
                  n.Sanitized_Concept AS sanitized_concept,
                  n.Learning_Objective AS learning_objective,
                  n.Skill_Level AS skill_level,
                  n.Priority AS priority,
                  n.Time_Estimate AS time_estimate,
                  n.Difficulty AS difficulty,
                  n.Semantic_Tags AS semantic_tags
            ORDER BY n.Priority ASC
            LIMIT 5
            """
            candidates = execute_cypher_query(driver, query, params={"level": level})
            candidates = [node for node in candidates if node["id"] not in learned_nodes]
            if not candidates:
                logger.error("No candidate nodes found after all fallbacks")
                return {"status": "error", "error_message": "Không tìm thấy nút ứng viên nào"}

        # Skip quiz if requested
        if skip_quiz:
            best_start_node = candidates[0]["id"]
			      # Kiểm tra sự tồn tại của nút
            check_query = f"MATCH (n {{{Config.PROPERTY_ID}: $node_id}}) RETURN n"
            result = execute_cypher_query(driver, check_query, params={"node_id": best_start_node})
            if not result:
                logger.error(f"Nút '{best_start_node}' không tồn tại trong đồ thị")
                return {"status": "error", "error_message": f"Nút '{best_start_node}' không tồn tại trong đồ thị"}
            return {"start_node": best_start_node, "status": "success", "error_message": None}

        # Generate quiz
        prompt = PromptTemplate(f"""
        Based on: {candidates}
        Generate exactly 15 multiple-choice questions (4 options each) for {student_goal} in context {context}.
        Each question should be in the following JSON format:
        {{
            "text": "Question text",
            "options": ["Option 1", "Option 2", "Option 3", "Option 4"],
            "correct_index": 0-3,
            "related_concept": "Sanitized concept or concept",
            "difficulty": "STANDARD|ADVANCED"
        }}
        Ensure questions cover a broad range of concepts and vary in difficulty 80% STANDARD, 20% ADVANCED.
        Return only the JSON list of questions.
        """)
        candidate_str = "\n".join([f"{c['id']}: {c['learning_objective']}" for c in candidates])
        quiz_json = Settings.llm.complete(prompt.format(candidates=candidate_str, goal=student_goal or "general knowledge", context=context or "general"))
        questions = json.loads(clean_json_response(quiz_json.text))
        if len(questions) != 15:
            questions = questions[:15] if len(questions) > 15 else questions + [{"text": "Placeholder", "options": ["A", "B", "C", "D"], "correct_index": 0, "related_concept": "general", "difficulty": "STANDARD"}] * (15 - len(questions))

        # Display quiz with options
        print('Hãy thực hiện bài kiểm tra đầu vào để xác định lộ trình học tập (Chọn 1 trong 4 đáp án A B C D). . .')
        answers = []
        for i, q in enumerate(questions):
            print(f"\nCâu {i+1} ({q['difficulty']}): {q['text']}")
            for idx, option in enumerate(q['options']):
                print(f"{chr(65 + idx)}. {option}") # Display as A, B, C, D
            while True:
                try:
                    answer = input("Chọn đáp án (A B C D): ").strip().upper()
                    if answer in ['A', 'B', 'C', 'D']:
                        answer_idx = ord(answer) - 65
                        break
                    print("Đầu vào không hợp lệ: Vui lòng chọn A, B, C hoặc D.")
                except KeyboardInterrupt:
                    print("Đầu vào bị gián đoạn. Chọn đáp án mặc định: A.")
                    answer_idx = 0
                    break
            answers.append(answer_idx)


        # Calculate score
        assessment_score = sum(1 for i, ans in enumerate(answers) if ans == questions[i]["correct_index"]) / 15 * 100
        logger.info(f"assessment_score: {assessment_score}")

        # Select best start node
        node_weights = {q["related_concept"]: ({"STANDARD": 1.0, "ADVANCED": 2.0}[q["difficulty"]] if answers[i] != q["correct_index"] else 0) for i, q in enumerate(questions)}
        bloom_levels = Config.ASTAR_SKILL_LEVELS_LOW + Config.ASTAR_SKILL_LEVELS_HIGH
        student_level_idx = bloom_levels.index(level)

        combined_score = Config.ASTAR_CURRENT_LEVEL_WEIGHT * current_level + Config.ASTAR_ASSESSMENT_WEIGHT * assessment_score
        logger.info(f"combined_score: {combined_score}")
        node_candidates = [
            {
                "id": c["id"],
                "combined": abs((Config.ASTAR_SKILL_LEVEL_SCORES.get(c["skill_level"], Config.ASTAR_SKILL_LEVEL_SCORES["default"])/6 * 25
                                + (c.get("priority") or 0)/5 * 10)
                                - Config.ASTAR_CURRENT_LEVEL_WEIGHT * current_level
                                - Config.ASTAR_ASSESSMENT_WEIGHT * assessment_score),
                "min_gap": abs(bloom_levels.index(skill_level) - student_level_idx) if (skill_level := (c.get("skill_level") if c.get("skill_level") in bloom_levels else "Understand")) in bloom_levels else 0,
                "priority": c.get("priority") or 0,
                "time_estimate": c.get("time_estimate") or 30,
                "difficulty": c.get("difficulty", "STANDARD")
            }
            for c in candidates
        ]

        logger.info(f"node_candidates: {node_candidates}")

        if not node_candidates:
            logger.error("No node candidates generated")
            if candidates:
                best_start_node = candidates[0]["id"]
            else:
                return {"status": "error", "error_message": "Không có nút nào để chọn làm điểm bắt đầu"}
        else:
            node_candidates.sort(key=lambda x: (x["combined"], x["min_gap"], -x["priority"], x["time_estimate"]))
            best_start_node = node_candidates[0]["id"]
            logger.info(f"Best start node selected: {best_start_node}")

        # Kiểm tra sự tồn tại của best_start_node
        check_query = f"MATCH (n {{{Config.PROPERTY_ID}: $node_id}}) RETURN n"
        result = execute_cypher_query(driver, check_query, params={"node_id": best_start_node})
        if not result:
            logger.error(f"Nút '{best_start_node}' không tồn tại trong đồ thị")
            return {"status": "error", "error_message": f"Nút '{best_start_node}' không tồn tại trong đồ thị"}

        # Lưu kết quả bài kiểm tra để sử dụng cho feedback loop
        assessment_history = {"avg_score": assessment_score}
        update_heuristic_based_on_assessment(assessment_history)
        logger.info(f"Assessment history saved: {assessment_history}")

        global_start_node = best_start_node
        return {"start_node": best_start_node, "status": "success", "error_message": None}

    except Exception as e:
        logger.error(f"Error in determine_start_node: {str(e)}")
        return {"start_node": None, "status": "error", "error_message": str(e)}

In [ ]:
# def determine_start_node(student_id: str, level: str, context: str, student_goal: str = None, student_file: str = "students.csv", skip_quiz: bool = False) -> Dict:
#     """Xác định điểm bắt đầu cho học sinh dựa trên trình độ, ngữ cảnh và mục tiêu học tập.

#     Args:
#         student_id (str): ID của học sinh.
#         level (str): Trình độ hiện tại (e.g., 'Remember', 'Understand').
#         context (str): Ngữ cảnh học tập (e.g., 'e_learning').
#         student_goal (str, optional): Mục tiêu học tập (e.g., 'sql basics').
#         student_file (str): Đường dẫn đến file hồ sơ học sinh.
#         skip_quiz (bool): Bỏ qua bài kiểm tra nếu True.

#     Returns:
#         Dict: {'start_node': str, 'status': str, 'error_message': str or None}
#     """
#     global vector_index, driver, Settings, Config
#     global_start_node = None

#     try:
#         # Validate inputs
#         if not student_goal and not context:
#             raise ValueError("Phải cung cấp ít nhất một trong student_goal hoặc context")
#         query_txt = f"Mục tiêu học tập: {student_goal or context}"
#         if level not in Config.ASTAR_SKILL_LEVELS_LOW + Config.ASTAR_SKILL_LEVELS_HIGH:
#             raise ValueError(f"Trình độ không hợp lệ: {level}")

#         # Load student profile
#         profile = load_student_profile(student_id)
#         current_level = profile.get("current_level", 0)
#         learned_nodes = profile.get("learning_history", [])

#         # Logic mới cho skill_levels
#         if current_level <= Config.ASTAR_CURRENT_LEVEL_THRESHOLD:
#             skill_levels = Config.ASTAR_SKILL_LEVELS_LOW
#         else:
#             # Lấy tất cả cấp độ từ LOW và HIGH
#             all_levels = Config.ASTAR_SKILL_LEVELS_LOW + Config.ASTAR_SKILL_LEVELS_HIGH
#             skill_weights = Config.ASTAR_HEURISTIC_WEIGHTS["skill_level"]
#             level_weight = skill_weights.get(level, skill_weights["default"])
#             # Lọc các cấp độ có trọng số nhỏ hơn hoặc bằng level đầu vào
#             skill_levels = [
#                 skill for skill in all_levels
#                 if skill_weights.get(skill, skill_weights["default"]) <= level_weight
#             ]

#         # Ensure skill_levels is defined
#         if not skill_levels:
#             logger.error("Skill levels not defined. Check Config.ASTAR_SKILL_LEVELS_LOW and Config.ASTAR_SKILL_LEVELS_HIGH.")
#             return {"status": "error", "error_message": "Skill levels not defined"}

#         # Step 1: Tìm kiếm ngữ nghĩa với VectorStoreIndex
#         potential_nodes = []
#         if vector_index:
#             query_engine = vector_index.as_query_engine(similarity_top_k=20)
#             query_txt = f"Mục tiêu học tập: {student_goal or context}. Trình độ: {level}. Ngữ cảnh: {context or 'không có'}"
#             response = query_engine.query(query_txt)
#             potential_nodes = [
#                 doc.metadata.get("node_id")
#                 for doc in response.source_nodes
#                 if doc.metadata.get("node_id") and doc.metadata.get("node_id") != "Không có ID"
#                 and doc.metadata.get("skill_level") in skill_levels
#             ]
#             logger.info(f"Potential nodes from vector index: {potential_nodes}")
#         else:
#             logger.warning("Vector index not initialized.")

#         # Step 2: Truy vấn đồ thị với Cypher query trực tiếp
#         candidates = []
#         goal_tags = generate_goal_tags(student_goal=student_goal, context=context, llm=Settings.llm, num=30)
#         logger.info(f"goal_tags: {goal_tags}")
#         goal_tags_str = ', '.join([f"'{tag}'" for tag in goal_tags])
#         logger.info(f"goal_tags_str: {goal_tags_str}")

#         # Xây dựng truy vấn Cypher linh hoạt dựa trên potential_nodes
#         if potential_nodes:
#             node_list = ', '.join([f"'{node}'" for node in potential_nodes])
#             cypher_query = f"""
#             MATCH (n:KnowledgeNode)
#             WHERE n.Node_ID IN [{node_list}]
#             AND (
#                 ANY(tag IN [{goal_tags_str}] WHERE tag IN split(n.Semantic_Tags, ';'))
#                 OR ANY(tag IN [{goal_tags_str}] WHERE tag IN split(n.Sanitized_Concept, ';'))
#                 OR n.Learning_Objective CONTAINS '{student_goal or ''}'
#             )
#             AND n.Skill_Level IN {skill_levels}
#             """
#         else:
#             cypher_query = f"""
#             MATCH (n:KnowledgeNode)
#             WHERE (
#                 ANY(tag IN [{goal_tags_str}] WHERE tag IN split(n.Semantic_Tags, ';'))
#                 OR ANY(tag IN [{goal_tags_str}] WHERE tag IN split(n.Sanitized_Concept, ';'))
#                 OR n.Learning_Objective CONTAINS '{student_goal or ''}'
#             )
#             AND n.Skill_Level IN {skill_levels}
#             """

#         # Thêm điều kiện context nếu context không rỗng
#         if context:
#             cypher_query += f" AND n.Context = '{context}'"

#         cypher_query += """
#         AND EXISTS ((n)-[]->())
#         RETURN n.Node_ID AS id,
#                n.Sanitized_Concept AS sanitized_concept,
#                n.Learning_Objective AS learning_objective,
#                n.Skill_Level AS skill_level,
#                n.Priority AS priority,
#                n.Time_Estimate AS time_estimate,
#                n.Difficulty AS difficulty,
#                n.Semantic_Tags AS semantic_tags
#         """

#         # Thực hiện truy vấn Cypher
#         result = execute_cypher_query(driver=driver, query=cypher_query)
#         logger.info(f"result: {result}")
#         candidates = [record.data() for record in result]
#         logger.info(f"candidates_1: {candidates}")

#         # Step 3: Fallback nếu không có candidates
#         if not candidates:
#             # Fallback 1: Lấy nodes theo level và context (nếu có)
#             query = f"""
#             MATCH (n:KnowledgeNode)
#             WHERE n.Skill_Level IN {skill_levels}
#             AND EXISTS ((n)-[]->())
#             """
#             if context:
#                 query += f" AND n.Context = '{context}'"
#             query += """
#             RETURN n.Node_ID AS id,
#                   n.Sanitized_Concept AS sanitized_concept,
#                   n.Learning_Objective AS learning_objective,
#                   n.Skill_Level AS skill_level,
#                   n.Priority AS priority,
#                   n.Time_Estimate AS time_estimate,
#                   n.Difficulty AS difficulty,
#                   n.Semantic_Tags AS semantic_tags
#             ORDER BY n.Priority ASC
#             """
#             candidates = execute_cypher_query(driver, query, params={"level": level})
#             if candidates:
#                 logger.info(f"candidates_in_Fallback_1: {candidates}")
#             if not candidates:
#                 # Fallback 2: Lấy nodes theo context (nếu có)
#                 query = """
#                 MATCH (n:KnowledgeNode)
#                 WHERE EXISTS ((n)-[]->())
#                 """
#                 if context:
#                     query += f" AND n.Context = '{context}'"
#                 query += f"""
#                 RETURN n.Node_ID AS id,
#                       n.Sanitized_Concept AS sanitized_concept,
#                       n.Learning_Objective AS learning_objective,
#                       n.Skill_Level AS skill_level,
#                       n.Priority AS priority,
#                       n.Time_Estimate AS time_estimate,
#                       n.Difficulty AS difficulty,
#                       n.Semantic_Tags AS semantic_tags
#                 WHERE n.Skill_Level IN {skill_levels}
#                 """
#                 # ORDER BY n.Priority ASC
#                 # LIMIT 10
#                 candidates = execute_cypher_query(driver, query)
#                 if candidates:
#                     logger.info(f"candidates_in_Fallback_2: {candidates}")

#         # Step 4: Lọc bổ sung với Jaccard similarity
#         if candidates:
#             similarities = []
#             for node in candidates:
#                 semantic_tags = node.get("semantic_tags", "").split(";")
#                 similarity = jaccard_similarity(goal_tags, semantic_tags)
#                 node["similarity"] = similarity
#                 similarities.append(similarity)

#             threshold = 0.7
#             filtered_candidates = [node for node in candidates if node["similarity"] >= threshold]

#             if not filtered_candidates:
#                 threshold_DYM = max(min(max(similarities) * 0.8, 0.7), 0.1) if similarities else 0.3
#                 filtered_candidates = [node for node in candidates if node["similarity"] >= threshold_DYM]

#                 if not filtered_candidates:
#                     logger.error("Không có nút nào trong ngưỡng. Chọn top 5 nút có similarity cao nhất")
#                     filtered_candidates = sorted(candidates, key=lambda x: x["similarity"], reverse=True)[:5]
#             logger.info(f"similarities: {similarities}")

#         else:
#             # Fallback chung nếu không có candidates
#             query = f"""
#             MATCH (n:KnowledgeNode)
#             WHERE EXISTS ((n)-[]->())
#             AND n.Skill_Level IN {skill_levels}
#             """
#             if context:
#                 query += f" AND n.Context = '{context}'"
#             else:
#                 query += " AND true"

#             query += """
#             RETURN n.{id_prop} AS id,
#                   n.{concept_prop} AS sanitized_concept,
#                   n.{objective_prop} AS learning_objective,
#                   n.{skill_prop} AS skill_level,
#                   n.{priority_prop} AS priority,
#                   n.{time_prop} AS time_estimate,
#                   n.{difficulty_prop} AS difficulty,
#                   n.{tags_prop} AS semantic_tags
#             ORDER BY n.{priority_prop} ASC
#             LIMIT 10
#             """.format(
#                 id_prop=Config.PROPERTY_ID,
#                 concept_prop=Config.PROPERTY_SANITIZED_CONCEPT,
#                 objective_prop=Config.PROPERTY_LEARNING_OBJECTIVE,
#                 skill_prop=Config.PROPERTY_SKILL_LEVEL,
#                 priority_prop=Config.PROPERTY_PRIORITY,
#                 time_prop=Config.PROPERTY_TIME_ESTIMATE,
#                 difficulty_prop=Config.PROPERTY_DIFFICULTY,
#                 tags_prop=Config.PROPERTY_SEMANTIC_TAGS,
#                 skill_levels=skill_levels
#             )
#             filtered_candidates = execute_cypher_query(driver, query)

#         filtered_nodes = [node for node in filtered_candidates if node["id"] not in learned_nodes]
#         candidates = filtered_nodes

#         if not candidates:
#             logger.info("No candidates found after filtering learned nodes. Using priority-based fallback.")
#             query = f"""
#             MATCH (n:KnowledgeNode)
#             WHERE n.Skill_Level IN {skill_levels}
#             AND EXISTS ((n)-[]->())
#             """
#             if context:
#                 query += f" AND n.Context = '{context}'"
#             query += """
#             RETURN n.Node_ID AS id,
#                   n.Sanitized_Concept AS sanitized_concept,
#                   n.Learning_Objective AS learning_objective,
#                   n.Skill_Level AS skill_level,
#                   n.Priority AS priority,
#                   n.Time_Estimate AS time_estimate,
#                   n.Difficulty AS difficulty,
#                   n.Semantic_Tags AS semantic_tags
#             ORDER BY n.Priority ASC
#             LIMIT 5
#             """
#             candidates = execute_cypher_query(driver, query, params={"level": level})
#             candidates = [node for node in candidates if node["id"] not in learned_nodes]
#             if not candidates:
#                 logger.error("No candidate nodes found after all fallbacks")
#                 return {"status": "error", "error_message": "Không tìm thấy nút ứng viên nào"}

#         # Skip quiz if requested
#         if skip_quiz:
#             best_start_node = candidates[0]["id"]
# 			      # Kiểm tra sự tồn tại của nút
#             check_query = f"MATCH (n {{{Config.PROPERTY_ID}: $node_id}}) RETURN n"
#             result = execute_cypher_query(driver, check_query, params={"node_id": best_start_node})
#             if not result:
#                 logger.error(f"Nút '{best_start_node}' không tồn tại trong đồ thị")
#                 return {"status": "error", "error_message": f"Nút '{best_start_node}' không tồn tại trong đồ thị"}
#             return {"start_node": best_start_node, "status": "success", "error_message": None}

#         # Generate quiz
#         prompt = PromptTemplate(f"""
#         Based on: {candidates}
#         Generate exactly 15 multiple-choice questions (4 options each) for {student_goal} in context {context}.
#         Each question should be in the following JSON format:
#         {{
#             "text": "Question text",
#             "options": ["Option 1", "Option 2", "Option 3", "Option 4"],
#             "correct_index": 0-3,
#             "related_concept": "Sanitized concept or concept",
#             "difficulty": "STANDARD|ADVANCED"
#         }}
#         Ensure questions cover a broad range of concepts and vary in difficulty 80% STANDARD, 20% ADVANCED.
#         Return only the JSON list of questions.
#         """)
#         candidate_str = "\n".join([f"{c['id']}: {c['learning_objective']}" for c in candidates])
#         quiz_json = Settings.llm.complete(prompt.format(candidates=candidate_str, goal=student_goal or "general knowledge", context=context or "general"))
#         questions = json.loads(clean_json_response(quiz_json.text))
#         if len(questions) != 15:
#             questions = questions[:15] if len(questions) > 15 else questions + [{"text": "Placeholder", "options": ["A", "B", "C", "D"], "correct_index": 0, "related_concept": "general", "difficulty": "STANDARD"}] * (15 - len(questions))

#         # Display quiz with options
#         print('Hãy thực hiện bài kiểm tra đầu vào để xác định lộ trình học tập (Chọn 1 trong 4 đáp án A B C D). . .')
#         answers = []
#         for i, q in enumerate(questions):
#             print(f"\nCâu {i+1} ({q['difficulty']}): {q['text']}")
#             for idx, option in enumerate(q['options']):
#                 print(f"{chr(65 + idx)}. {option}") # Display as A, B, C, D
#             while True:
#                 try:
#                     answer = input("Chọn đáp án (A B C D): ").strip().upper()
#                     if answer in ['A', 'B', 'C', 'D']:
#                         answer_idx = ord(answer) - 65
#                         break
#                     print("Đầu vào không hợp lệ: Vui lòng chọn A, B, C hoặc D.")
#                 except KeyboardInterrupt:
#                     print("Đầu vào bị gián đoạn. Chọn đáp án mặc định: A.")
#                     answer_idx = 0
#                     break
#             answers.append(answer_idx)

#         # Calculate score
#         assessment_score = sum(1 for i, ans in enumerate(answers) if ans == questions[i]["correct_index"]) / 15 * 100
#         logger.info(f"assessment_score: {assessment_score}")

#         # Select best start node
#         node_weights = {q["related_concept"]: ({"STANDARD": 1.0, "ADVANCED": 2.0}[q["difficulty"]] if answers[i] != q["correct_index"] else 0) for i, q in enumerate(questions)}
#         bloom_levels = Config.ASTAR_SKILL_LEVELS_LOW + Config.ASTAR_SKILL_LEVELS_HIGH
#         student_level_idx = bloom_levels.index(level)

#         combined_score = Config.ASTAR_CURRENT_LEVEL_WEIGHT * current_level + Config.ASTAR_ASSESSMENT_WEIGHT * assessment_score
#         logger.info(f"combined_score: {combined_score}")
#         node_candidates = [
#             {
#                 "id": c["id"],
#                 "combined": abs((Config.ASTAR_SKILL_LEVEL_SCORES.get(c["skill_level"], Config.ASTAR_SKILL_LEVEL_SCORES["default"])/6 * 25
#                                 + (c.get("priority") or 0)/5 * 10)
#                                 - Config.ASTAR_CURRENT_LEVEL_WEIGHT * current_level
#                                 - Config.ASTAR_ASSESSMENT_WEIGHT * assessment_score),
#                 "min_gap": abs(bloom_levels.index(skill_level) - student_level_idx) if (skill_level := (c.get("skill_level") if c.get("skill_level") in bloom_levels else "Understand")) in bloom_levels else 0,
#                 "priority": c.get("priority") or 0,
#                 "time_estimate": c.get("time_estimate") or 30,
#                 "difficulty": c.get("difficulty", "STANDARD")
#             }
#             for c in candidates
#         ]

#         logger.info(f"node_candidates: {node_candidates}")

#         if not node_candidates:
#             logger.error("No node candidates generated")
#             if candidates:
#                 best_start_node = candidates[0]["id"]
#             else:
#                 return {"status": "error", "error_message": "Không có nút nào để chọn làm điểm bắt đầu"}
#         else:
#             node_candidates.sort(key=lambda x: (x["combined"], x["min_gap"], -x["priority"], x["time_estimate"]))
#             best_start_node = node_candidates[0]["id"]

#         # Kiểm tra sự tồn tại của best_start_node
#         check_query = f"MATCH (n {{{Config.PROPERTY_ID}: $node_id}}) RETURN n"
#         result = execute_cypher_query(driver, check_query, params={"node_id": best_start_node})
#         if not result:
#             logger.error(f"Nút '{best_start_node}' không tồn tại trong đồ thị")
#             return {"status": "error", "error_message": f"Nút '{best_start_node}' không tồn tại trong đồ thị"}

#         # Lưu kết quả bài kiểm tra để sử dụng cho feedback loop
#         assessment_history = {"avg_score": assessment_score}
#         update_heuristic_based_on_assessment(assessment_history)

#         global_start_node = best_start_node
#         return {"start_node": best_start_node, "status": "success", "error_message": None}

#     except Exception as e:
#         logger.error(f"Error in determine_start_node: {str(e)}")
#         return {"start_node": None, "status": "error", "error_message": str(e)}

# determine_goal_node

In [ ]:
def determine_goal_node(student_id: str, context: str, student_goal: str = None, start_node: str = None, student_file: str = Config.STUDENT_FILE) -> Dict:
    """Xác định điểm đích (goal node) dựa trên mục tiêu học tập, điểm bắt đầu và hồ sơ học sinh.

    Args:
        student_id (str): ID của học sinh.
        context (str): Ngữ cảnh học tập (e.g., 'math', 'programming').
        student_goal (str, optional): Mục tiêu học tập cụ thể của học sinh.
        start_node (str): ID của nút khởi đầu.
        student_file (str): Đường dẫn đến file hồ sơ học sinh.

    Returns:
        Dict: Dictionary chứa 'goal_node', 'status', và 'error_message'.
    """
    try:
        # Bước 1: Kiểm tra đầu vào
        if not student_id or not isinstance(student_id, str):
            raise ValueError("student_id phải là chuỗi không rỗng")
        if not start_node or not isinstance(start_node, str):
            raise ValueError("Nút khởi đầu phải là chuỗi không rỗng")

        # Bước 2: Tải hồ sơ học sinh
        profile = load_student_profile(student_id)
        if not profile:
            logger.warning(f"Không tìm thấy hồ sơ cho student_id '{student_id}'. Sử dụng giá trị mặc định.")
            current_level = 0
            learning_style_preference = Config.DEFAULT_LEARNING_STYLE
            skill_level = "Understand"
        else:
            current_level = profile.get("current_level", 0)
            learning_style_preference = profile.get("learning_style_preference", Config.DEFAULT_LEARNING_STYLE)
            skill_level = profile.get("skill_level", "Understand")

        # Bước 3: Xác định độ khó dựa trên trình độ hiện tại
        difficulty = Config.ASTAR_DIFFICULTY_FILTER[0] if current_level < Config.ASTAR_CURRENT_LEVEL_THRESHOLD else Config.ASTAR_DIFFICULTY_FILTER[1]

        # Bước 4: Truy vấn các nút liên quan từ điểm bắt đầu
        query = f"""
        MATCH (start {{{Config.PROPERTY_ID}: $start_node}})-[:{Config.RELATIONSHIP_NEXT}|{Config.RELATIONSHIP_REQUIRES}|{Config.RELATIONSHIP_IS_PREREQUISITE_OF}|{Config.RELATIONSHIP_HAS_ALTERNATIVE_PATH}|{Config.RELATIONSHIP_SIMILAR_TO}|{Config.RELATIONSHIP_IS_SUBCONCEPT_OF}*0..]->(related)
        WHERE related.{Config.PROPERTY_DIFFICULTY} = $difficulty
        AND NOT (related)-[:{Config.RELATIONSHIP_NEXT}|{Config.RELATIONSHIP_IS_PREREQUISITE_OF}|{Config.RELATIONSHIP_IS_SUBCONCEPT_OF}]->()
        AND NOT ()-[:{Config.RELATIONSHIP_REQUIRES}]->(related)
        RETURN related.{Config.PROPERTY_ID} AS id, related.{Config.PROPERTY_SANITIZED_CONCEPT} AS sanitized_concept,
              related.{Config.PROPERTY_LEARNING_OBJECTIVE} AS learning_objective, related.{Config.PROPERTY_SKILL_LEVEL} AS skill_level,
              related.{Config.PROPERTY_DIFFICULTY} AS difficulty, related.{Config.PROPERTY_SEMANTIC_TAGS} AS semantic_tags,
              related.{Config.PROPERTY_LEARNING_STYLE_PREFERENCE} AS learning_style
        """
        related_nodes = execute_cypher_query(driver, query, params={"start_node": start_node, "difficulty": difficulty})

        if not related_nodes:
            logger.warning("Không tìm thấy nút liên quan. Tìm kiếm trong toàn bộ đồ thị.")
            query = f"""
            MATCH (start {{{Config.PROPERTY_ID}: $start_node}})-[:{Config.RELATIONSHIP_NEXT}|{Config.RELATIONSHIP_REQUIRES}|{Config.RELATIONSHIP_IS_PREREQUISITE_OF}|{Config.RELATIONSHIP_HAS_ALTERNATIVE_PATH}|{Config.RELATIONSHIP_SIMILAR_TO}|{Config.RELATIONSHIP_IS_SUBCONCEPT_OF}*0..]->(related)
            WHERE NOT EXISTS {{
                MATCH (related)-[:{Config.RELATIONSHIP_NEXT}|{Config.RELATIONSHIP_IS_PREREQUISITE_OF}]->()
            }}
            RETURN related.{Config.PROPERTY_ID} AS id, related.{Config.PROPERTY_SANITIZED_CONCEPT} AS sanitized_concept,
                  related.{Config.PROPERTY_LEARNING_OBJECTIVE} AS learning_objective, related.{Config.PROPERTY_SKILL_LEVEL} AS skill_level,
                  related.{Config.PROPERTY_DIFFICULTY} AS difficulty, related.{Config.PROPERTY_SEMANTIC_TAGS} AS semantic_tags,
                  related.{Config.PROPERTY_LEARNING_STYLE_PREFERENCE} AS learning_style
            ORDER BY related.{Config.PROPERTY_PRIORITY} DESC
            LIMIT 10
            """
            related_nodes = execute_cypher_query(driver, query, params={"start_node": start_node, "difficulty": difficulty})

        if not related_nodes:
            raise ValueError("Không có nút nào trong đồ thị để chọn làm nút đích")

        # Bước 5: Tính độ tương đồng với mục tiêu học tập
        goal_tags = generate_goal_tags(student_goal=student_goal, context=context, llm=Settings.llm, num=30)
        bloom_levels = Config.ASTAR_SKILL_LEVELS_LOW + Config.ASTAR_SKILL_LEVELS_HIGH
        student_level_idx = bloom_levels.index(skill_level)
        potential_goals = []

        for node in related_nodes:
            node_level_idx = bloom_levels.index(node.get("skill_level", "Understand") if node.get("skill_level") in bloom_levels else "Understand")
            if abs(node_level_idx - student_level_idx) > 2:
                continue
            # Xử lý semantic_tags nếu không tồn tại
            semantic_tags_str = node.get("semantic_tags", "")
            semantic_tags = semantic_tags_str.split(";") if isinstance(semantic_tags_str, str) and semantic_tags_str else []
            expanded_tags = expand_tags(semantic_tags)
            similarity = jaccard_similarity(goal_tags, expanded_tags) if goal_tags else 0.0
            if similarity >= 0.3 and learning_style_preference in node.get("learning_style", Config.DEFAULT_LEARNING_STYLE):
                potential_goals.append((node, similarity))

        # Nếu không tìm thấy goal dựa trên similarity, dùng LLM để lọc từ related_nodes
        if not potential_goals and student_goal:
            logger.warning("Không tìm thấy goal dựa trên similarity. Sử dụng LLM để lọc từ related_nodes.")
            if not related_nodes:
                raise ValueError("Không có related_nodes để lọc")

            # Chuẩn bị dữ liệu cho LLM
            node_descriptions = [
                f"ID: {node['id']}, Khái niệm: {node['sanitized_concept']}, Mục tiêu học tập: {node['learning_objective']}, Thẻ ngữ nghĩa: {node['semantic_tags']}"
                for node in related_nodes
            ]
            prompt = f"Dựa trên mục tiêu học tập: '{student_goal}', hãy chọn nút phù hợp nhất từ danh sách sau:\n" + "\n".join(node_descriptions)
            llm_response = Settings.llm.complete(prompt).text.strip()

            # LM trả về ID của nút phù hợp nhất
            best_node_id = llm_response.split()[0]
            for node in related_nodes:
                if node["id"] == best_node_id:
                    potential_goals.append((node, 1.0))  # Gán similarity cao
                    logger.info(f"Bước 5: Tính độ tương đồng - Node {best_node_id} được chọn bởi LLM với similarity=1.0")
                    break

        potential_goals = sorted(potential_goals, key=lambda x: x[1], reverse=True)[:Config.GOAL_NODE_SUGGESTION_LIMIT]

        if not potential_goals:
            logger.warning("Không có nút nào đạt ngưỡng tương đồng. Chọn nút mặc định.")
            sorted_nodes = sorted(related_nodes, key=lambda x: x.get("priority", 0), reverse=True)

            if sorted_nodes:
                default_goal_node = sorted_nodes[0]["id"]
                logger.info(f"Bước 5: Tính độ tương đồng - Chọn default goal_node: {default_goal_node}")
                return {"goal_node": default_goal_node, "status": "success", "error_message": None}
            else:
                raise ValueError("Không có nút nào để chọn làm nút đích")

        logger.info("Bước 6: Tìm goal node tối ưu - Bắt đầu A* search")
        best_goal = a_star_search(start_node, potential_goals, driver)
        # Kiểm tra sự tồn tại của best_goal
        check_query = f"MATCH (n {{{Config.PROPERTY_ID}: $best_goal}}) RETURN n"
        if not execute_cypher_query(driver, check_query, params={"best_goal": best_goal}):
            raise ValueError(f"Goal node '{best_goal}' không tồn tại trong đồ thị")

        logger.info(f"Đã chọn goal_node: {best_goal}")
        return {"goal_node": best_goal, "status": "success", "error_message": None}

    except Exception as e:
        logger.error(f"Lỗi trong determine_goal_node: {str(e)}")
        return {"goal_node": None, "status": "error", "error_message": str(e)}

In [ ]:
#  Áp dụng thuật toán A* để chọn nút đích tối ưu
def get_heuristic(node_id, goal_similarities, driver):
    similarity = goal_similarities.get(node_id, 0.0)
    node_info_query = f"""
    MATCH (n {{{Config.PROPERTY_ID}: $node_id}})
    RETURN n.{Config.PROPERTY_DIFFICULTY} AS difficulty, n.{Config.PROPERTY_SKILL_LEVEL} AS skill_level,
          n.{Config.PROPERTY_PRIORITY} AS priority, n.{Config.PROPERTY_TIME_ESTIMATE} AS time_estimate
    """
    node_info = execute_cypher_query(driver, node_info_query, params={"node_id": node_id})
    if not node_info:
        return float('inf')
    difficulty_score = Config.ASTAR_HEURISTIC_WEIGHTS["difficulty_standard"] if node_info[0]["difficulty"] == "STANDARD" else Config.ASTAR_HEURISTIC_WEIGHTS["difficulty_advanced"]
    skill_level_score = Config.ASTAR_HEURISTIC_WEIGHTS["skill_level"].get(node_info[0]["skill_level"], Config.ASTAR_HEURISTIC_WEIGHTS["skill_level"]["default"])
    priority_score = node_info[0].get("priority", 0) * Config.ASTAR_HEURISTIC_WEIGHTS["priority"]
    time_estimate_score = node_info[0].get("time_estimate", 0) * Config.ASTAR_HEURISTIC_WEIGHTS["time_estimate"]
    h_score = (0.4 * (1 - similarity)) + (0.2 * difficulty_score / 3.0) + (0.2 * skill_level_score / 6.0) + (0.1 * priority_score / 5.0) + (0.1 * time_estimate_score)
    return h_score

In [ ]:
def a_star_search(start: str, potential_goals: List[tuple], driver) -> str:
    open_set = [(0, 0, start, [start])]
    closed_set = set()
    goal_nodes = {goal[0]["id"] for goal in potential_goals}
    goal_similarities = {goal[0]["id"]: goal[1] for goal in potential_goals}

    while open_set:
        f_score, g_score, current, path = heapq.heappop(open_set)
        if current in closed_set:
            continue
        closed_set.add(current)
        if current in goal_nodes:
            return current
        neighbors = execute_cypher_query(
            driver,
            f"""
            MATCH (current:KnowledgeNode {{{Config.PROPERTY_ID}: $current}})
            -[r:{Config.RELATIONSHIP_NEXT}|{Config.RELATIONSHIP_REQUIRES}|{Config.RELATIONSHIP_IS_PREREQUISITE_OF}|{Config.RELATIONSHIP_HAS_ALTERNATIVE_PATH}|{Config.RELATIONSHIP_SIMILAR_TO}]->(neighbor:KnowledgeNode)
            RETURN neighbor.{Config.PROPERTY_ID} AS id, r.Weight AS weight
            """,
            params={"current": current}
        )
        for record in neighbors:
            neighbor = record["id"]
            weight = record.get("weight", 1.0)
            if neighbor in closed_set:
                continue
            new_g_score = g_score + weight
            h_score = get_heuristic(neighbor, goal_similarities, driver)
            f_score = new_g_score + h_score
            heapq.heappush(open_set, (f_score, new_g_score, neighbor, path + [neighbor]))

    # Fallback: Kiểm tra từng goal_node trong potential_goals
    for goal, _ in potential_goals:
        check_query = f"MATCH (n {{{Config.PROPERTY_ID}: $node_id}}) RETURN n"
        if execute_cypher_query(driver, check_query, params={"node_id": goal["id"]}):
            return goal["id"]
    raise ValueError("Không tìm thấy goal_node hợp lệ trong potential_goals")

# get_goal_tags

In [ ]:
def get_goal_tags(driver, goal_node):
    """Lấy semantic_tags của goal_node từ đồ thị Neo4j."""

    query = f"""
    MATCH (goal:KnowledgeNode {{{Config.PROPERTY_ID}: $goal_node}})
    RETURN goal.{Config.PROPERTY_SEMANTIC_TAGS} AS semantic_tags
    """

    result = execute_cypher_query(driver, query, params={"goal_node": goal_node})
    if result and result[0]["semantic_tags"]:
        return result[0]["semantic_tags"].split(";")  # Giả định semantic_tags là chuỗi phân cách bằng dấu chấm phẩy
    else:
        raise ValueError(f"Không tìm thấy goal_node '{goal_node}' trong đồ thị ở bước get_goal_tags")

# calculate_and_store_similarity

In [ ]:
def calculate_and_store_similarity(driver, goal_tags: List[str]) -> None:
    """Tính và lưu similarity giữa goal_tags và semantic_tags của các nút."""
    query = """
    MATCH (n:KnowledgeNode)
    RETURN n.Node_ID AS id, n.Semantic_Tags AS semantic_tags
    """
    result = execute_cypher_query(driver, query)

    for node in result:
        node_id = node["id"]
        semantic_tags = node["semantic_tags"].split(";") if node["semantic_tags"] else []
        similarity = jaccard_similarity(goal_tags, semantic_tags)

        update_query = f"""
        MATCH (n:KnowledgeNode {{{Config.PROPERTY_ID}: $node_id}})
        SET n.similarity = $similarity
        """
        execute_cypher_query(driver, update_query, params={"node_id": node_id, "similarity": similarity})


# generate_learning_path

In [ ]:
import heapq
import json
def heuristic(node_id: str, driver, goal_tags: List[str], dynamic_weights: Dict, Config, context: str, heuristic_cache: Dict = None) -> float:
    if heuristic_cache is None:
        heuristic_cache = {}

    if node_id in heuristic_cache:
        return heuristic_cache[node_id]

    # Truy vấn Cypher với coalesce để xử lý giá trị thiếu
    query = f"""
    MATCH (n {{{Config.PROPERTY_ID}: $node_id}})
    RETURN coalesce(n.similarity, 0.0) AS similarity,
           coalesce(n.{Config.PROPERTY_PRIORITY}, 0.0) AS priority,
           coalesce(n.{Config.PROPERTY_DIFFICULTY}, 'STANDARD') AS difficulty,
           coalesce(n.{Config.PROPERTY_SKILL_LEVEL}, 'Understand') AS skill_level,
           coalesce(n.{Config.PROPERTY_TIME_ESTIMATE}, 30.0) AS time_estimate,
           coalesce(n.{Config.PROPERTY_CONTEXT}, '') AS context
    """
    result = execute_cypher_query(driver, query, params={"node_id": node_id})

    if not result:
        heuristic_cache[node_id] = 0.0
        return 0.0

    node = result[0]

    # Đảm bảo các giá trị là số hoặc chuỗi hợp lệ
    similarity = float(node.get("similarity", 0.0))
    priority = float(node.get("priority", 0.0))
    difficulty = node.get("difficulty", "STANDARD")
    skill_level = node.get("skill_level", "Understand")
    time_estimate = float(node.get("time_estimate", 30.0))
    context_match = 0.5 if node.get("context", "") == context else 0.0

    logger.debug(f"Node {node_id}: similarity={similarity}, priority={priority}, time_estimate={time_estimate}")

    # Kiểm tra dynamic_weights và gán giá trị mặc định nếu không hợp lệ
    if not isinstance(dynamic_weights, dict):
        print(f"dynamic_weights không phải dictionary cho node {node_id}, sử dụng mặc định")
        dynamic_weights = {
            'difficulty_standard': 0.0,
            'difficulty_advanced': 0.0,
            'skill_level': {'default': 0.0},
            'time_estimate': 0.0
        }

    # Gán giá trị với mặc định nếu thiếu
    difficulty_weight = float(dynamic_weights.get('difficulty_standard', 0.0) if difficulty == Config.ASTAR_DIFFICULTY_FILTER[0] else dynamic_weights.get('difficulty_advanced', 0.0))
    skill_level_weights = dynamic_weights.get('skill_level', {})
    if not isinstance(skill_level_weights, dict):
        skill_level_weights = {'default': 0.0}
    skill_weight = float(skill_level_weights.get(skill_level, skill_level_weights.get('default', 0.0)))
    weight_time_estimate = float(dynamic_weights.get('time_estimate', 0.0))

    # Log để debug
    print(f"Node {node_id}: similarity={similarity}, priority={priority}, difficulty={difficulty}, "
          f"skill_level={skill_level}, time_estimate={time_estimate}, context_match={context_match}")
    print(f"Dynamic weights: difficulty_weight={difficulty_weight}, skill_weight={skill_weight}, "
          f"weight_time_estimate={weight_time_estimate}")

    # Tính heuristic
    h = (0.4 * (1 - similarity) +
         priority * 0.1 / 5.0 +
         difficulty_weight * 0.2 / 3.0 +
         skill_weight * 0.2 / 6.0 +
         time_estimate * Config.ASTAR_HEURISTIC_WEIGHTS["time_estimate"] * weight_time_estimate -
         context_match)

    heuristic_cache[node_id] = h
    return h






In [ ]:
import json
import re
from typing import Dict
from llama_index.core.prompts import PromptTemplate
from llama_index.core.retrievers import TextToCypherRetriever
from llama_index.core.response_synthesizers import TreeSummarize



def generate_learning_path(student_id: str, level: str, start_node: str = None, context: str = None,
                          learning_style: str = None, student_goal: str = None,
                          student_file: str = Config.STUDENT_FILE, use_llm: bool = False) -> Dict:
    """Tạo lộ trình học tập từ điểm bắt đầu đến điểm đích với ngữ cảnh là tiêu chí ưu tiên phụ."""
    global global_start_node
    try:
        # Kiểm tra đầu vào
        if not student_id or not isinstance(student_id, str):
            raise ValueError("student_id phải là chuỗi không rỗng")
        if student_goal and not isinstance(student_goal, str):
            raise ValueError("student_goal phải là chuỗi không rỗng")

        # Xác định điểm bắt đầu
        if start_node is None:
            if global_start_node is None:
                start_node_result = determine_start_node(student_id, level, context, student_goal, student_file, skip_quiz=True)
                if start_node_result["status"] != "success":
                    raise ValueError(f"Lỗi xác định điểm bắt đầu: {start_node_result['error_message']}")
                start_node = start_node_result["start_node"]
            else:
                start_node = global_start_node

        # Kiểm tra sự tồn tại của start_node
        query = f"""
        MATCH (n {{{Config.PROPERTY_ID}: $node_id}})
        RETURN n.{Config.PROPERTY_ID} AS id
        """
        result_start = execute_cypher_query(driver, query, params={"node_id": start_node})
        if not result_start:
            raise ValueError(f"Nút khởi đầu '{start_node}' không tồn tại trong Knowledge Graph.")

        # Xác định điểm đích
        goal_node_result = determine_goal_node(student_id, context, student_goal, start_node, student_file)
        if goal_node_result["status"] != "success" or goal_node_result["goal_node"] is None:
            raise ValueError(f"Lỗi xác định điểm đích: {goal_node_result['error_message']}")
        goal_node = goal_node_result["goal_node"]

        result_goal = execute_cypher_query(driver, query, params={"node_id": goal_node})
        if not result_goal:
            raise ValueError(f"Nút đích '{goal_node}' không tồn tại trong Knowledge Graph")

        # Trường hợp start_node == goal_node
        if start_node == goal_node:
            logger.warning("start_node và goal_node trùng nhau, tìm các nút liên quan.")
            query = f"""
            MATCH (n {{{Config.PROPERTY_ID}: $node_id}})
            OPTIONAL MATCH (n)-[:{Config.RELATIONSHIP_NEXT}|{Config.RELATIONSHIP_REQUIRES}]->(related)
            RETURN n.{Config.PROPERTY_ID} AS id,
                   n.{Config.PROPERTY_SANITIZED_CONCEPT} AS sanitized_concept,
                   n.{Config.PROPERTY_CONTEXT} AS context,
                   n.{Config.PROPERTY_DEFINITION} AS definition,
                   n.{Config.PROPERTY_EXAMPLE} AS example,
                   n.{Config.PROPERTY_LEARNING_OBJECTIVE} AS learning_objective,
                   n.{Config.PROPERTY_SKILL_LEVEL} AS skill_level,
                   n.{Config.PROPERTY_DIFFICULTY} AS difficulty,
                   n.{Config.PROPERTY_TIME_ESTIMATE} AS time_estimate,
                   related.{Config.PROPERTY_ID} AS related_id,
                   related.{Config.PROPERTY_SANITIZED_CONCEPT} AS related_concept,
                   related.{Config.PROPERTY_CONTEXT} AS related_context,
                   related.{Config.PROPERTY_DEFINITION} AS related_definition,
                   related.{Config.PROPERTY_EXAMPLE} AS related_example,
                   related.{Config.PROPERTY_LEARNING_OBJECTIVE} AS related_objective,
                   related.{Config.PROPERTY_SKILL_LEVEL} AS related_skill_level,
                   related.{Config.PROPERTY_DIFFICULTY} AS related_difficulty,
                   related.{Config.PROPERTY_TIME_ESTIMATE} AS related_time_estimate
            LIMIT 5
            """
            result = execute_cypher_query(driver, query, params={"node_id": start_node})
            path = []
            step = 1

            if result:
                node = result[0]
                path.append({
                    "step": step,
                    "node_id": node["id"],
                    "sanitized_concept": node["sanitized_concept"] or "Unknown Concept",
                    "context": node["context"] or context or "general",
                    "definition": node["definition"] or "Không có định nghĩa",
                    "example": node["example"] or "Không có ví dụ",
                    "learning_objective": node["learning_objective"] or f"Học {node['sanitized_concept'] or 'Unknown Concept'}",
                    "skill_level": node["skill_level"] or level,
                    "difficulty": node["difficulty"] or "STANDARD",
                    "time_estimate": node["time_estimate"] or 30
                })
                step += 1

                for record in result:
                    if record["related_id"]:
                        path.append({
                            "step": step,
                            "node_id": record["related_id"],
                            "sanitized_concept": record["related_concept"] or "Unknown Concept",
                            "context": record["related_context"] or context or "general",
                            "definition": record["related_definition"] or "Không có định nghĩa",
                            "example": record["related_example"] or "Không có ví dụ",
                            "learning_objective": record["related_objective"] or f"Học {record['related_concept'] or 'Unknown Concept'}",
                            "skill_level": record["related_skill_level"] or level,
                            "difficulty": record["related_difficulty"] or "STANDARD",
                            "time_estimate": record["related_time_estimate"] or 30
                        })
                        step += 1

            if not path:
                logger.error("Không tìm thấy nút liên quan nào để xây dựng lộ trình")
                return {"path": [], "status": "error", "error_message": "Không tìm thấy nút liên quan nào để xây dựng lộ trình"}
            return {"path": path, "status": "success", "error_message": None}

        logger.warning(f"startnode trong generate learning path: {start_node}")
        logger.warning(f"goalnode trong generate learning path: {goal_node}")

        # Tải hồ sơ học sinh và thiết lập tham số
        profile = load_student_profile(student_id)
        skill_levels = Config.ASTAR_SKILL_LEVELS_LOW if profile.get("current_level", 0.0) < Config.ASTAR_CURRENT_LEVEL_THRESHOLD else Config.ASTAR_SKILL_LEVELS_HIGH
        learning_style_filter = f" AND n.{Config.PROPERTY_LEARNING_STYLE_PREFERENCE} CONTAINS $learning_style_preference" if profile.get("learning_style_preference") else ""

        # Tính trọng số động
        dynamic_weights = calculate_dynamic_weights()

        # Kiểm tra sự tồn tại của đường dẫn từ start_node đến goal_node
        path_check_query = f"""
        MATCH path = (start:KnowledgeNode {{{Config.PROPERTY_ID}: $start_id}})-[:NEXT|REQUIRES|IS_PREREQUISITE_OF|HAS_ALTERNATIVE_PATH|SIMILAR_TO|IS_SUBCONCEPT_OF*1..10]->(end:KnowledgeNode {{{Config.PROPERTY_ID}: $end_id}})
        RETURN path LIMIT 1
        """
        path_check_result = execute_cypher_query(driver, path_check_query, params={"start_id": start_node, "end_id": goal_node})

        if not path_check_result:
            logger.warning(f"Không có đường dẫn từ {start_node} đến {goal_node}. Fallback sang sử dụng LLM.")
            use_llm = True

        # Tạo lộ trình học tập
        if use_llm:
            context_filter = f"WHERE ALL(n IN nodes(path) WHERE n.Context = '{context}' OR '{context}' IS NULL)" if context else ""
            cypher_template = PromptTemplate(
                f"""
                #TASK:
                Generate a valid Cypher query to find a learning path from node with Node_ID '{start_node}' to node with Node_ID '{goal_node}' in a Neo4j graph database.
                #Instructions:
                The path should use relationships of types NEXT, REQUIRES, IS_PREREQUISITE_OF, HAS_ALTERNATIVE_PATH, SIMILAR_TO, or IS_SUBCONCEPT_OF, with a maximum path length of 10.
                Include an optional context filter for nodes with Context = '{context}' if context is provided.
                Return only the Cypher query string, without any explanation or additional text.
                Example:
                MATCH path = (start:KnowledgeNode {{Node_ID: '{start_node}'}})-[:NEXT|REQUIRES|IS_PREREQUISITE_OF|HAS_ALTERNATIVE_PATH|SIMILAR_TO|IS_SUBCONCEPT_OF*1..10]->(end:KnowledgeNode {{Node_ID: '{goal_node}'}})
                {context_filter}
                RETURN [node IN nodes(path) | node.Node_ID] AS path
                """
            )
            cypher_retriever = TextToCypherRetriever(
                graph_store=kg_index.property_graph_store,
                llm=Settings.llm,
                text_to_cypher_template=cypher_template,
                response_template="Generated Cypher query:\n(query)\n\nCypher Response:\n{response}",
                cypher_validator=None,
                allowed_output_field=["text", "label", "type"]
            )
            query_engine = kg_index.as_query_engine(sub_retrievers=[cypher_retriever], response_synthesizer=TreeSummarize())
            llm_response = query_engine.query(
                f"Create learning path from `{start_node}` to `{goal_node}` with optional context `{context}`"
            )
            logger.info(f"LLM Cypher response: {llm_response}")

            path = []
            if hasattr(llm_response, 'response'):
                cypher_query = extract_cypher_query(str(llm_response.response))
                if cypher_query:
                    try:
                        result = execute_cypher_query(driver, cypher_query)
                        if result and 'path' in result[0]:
                            path = result[0]['path']
                        else:
                            logger.warning("Truy vấn Cypher không trả về lộ trình hợp lệ.")
                    except Exception as e:
                        logger.warning(f"Thực thi truy vấn Cypher thất bại: {str(e)}")
                else:
                    logger.warning("Không trích xuất được truy vấn Cypher hợp lệ từ phản hồi LLM.")
            else:
                logger.warning("LLM response không hợp lệ (không có thuộc tính response).")

            if not path:
                logger.info("Cypher không tạo được lộ trình. Sử dụng LLM để tự sinh lộ trình học tập.")
                fallback_prompt = PromptTemplate(
                    f"""
                    Dựa trên điểm bắt đầu '{start_node}' và mục tiêu học tập '{student_goal}',
                    hãy tạo một lộ trình học tập *bằng tiếng Việt* gồm 3 bước tiến bộ từ cơ bản đến nâng cao, theo định dạng sau:
                    - step: số thứ tự bước (bắt đầu từ 1)
                    - node_id: mã định danh nút (tự tạo, ví dụ: generated_node_1, generated_node_2, ...)
                    - sanitized_concept: tên khái niệm
                    - context: ngữ cảnh (nếu không có thì dùng '{context}' hoặc 'general')
                    - definition: định nghĩa ngắn gọn
                    - example: ví dụ minh họa
                    - learning_objective: mục tiêu học tập của bước
                    - skill_level: mức độ kỹ năng (dùng '{level}' cho bước 1, tăng dần như 'Understand', 'Apply' cho các bước tiếp theo)
                    - difficulty: độ khó (STANDARD cho bước 1, tăng dần như INTERMEDIATE, ADVANCED)
                    - time_estimate: thời gian ước tính (số phút, mặc định 30, tăng dần như 45, 60 cho các bước tiếp theo)
                    Trả về danh sách 3 bước dưới dạng JSON, không giải thích thêm.
                    """
                )
                query_engine_fallback = kg_index.as_query_engine(response_synthesizer=TreeSummarize())
                llm_fallback_response = query_engine_fallback.query(
                    fallback_prompt.format(start_node=start_node, student_goal=student_goal or "general learning", level=level, context=context or "general")
                )
                logger.info(f"LLM fallback response: {llm_fallback_response}")

                if hasattr(llm_fallback_response, 'response'):
                    response_text = str(llm_fallback_response.response)
                    try:
                        path_data = json.loads(response_text)
                        if not isinstance(path_data, list):
                            raise ValueError("Phản hồi LLM không đúng định dạng danh sách JSON")
                        learning_path = []
                        for idx, step in enumerate(path_data, 1):
                            if not isinstance(step, dict):
                                raise ValueError(f"Bước {idx} không phải dictionary")
                            standardized_step = {
                                "step": step.get("step", idx),
                                "node_id": step.get("node_id", f"generated_node_{idx}"),
                                "sanitized_concept": step.get("sanitized_concept", "Unknown Concept"),
                                "context": step.get("context", context or "general"),
                                "definition": step.get("definition", "Không có định nghĩa"),
                                "example": step.get("example", "Không có ví dụ"),
                                "learning_objective": step.get("learning_objective", f"Học {step.get('sanitized_concept', 'Unknown Concept')}"),
                                "skill_level": step.get("skill_level", level),
                                "difficulty": step.get("difficulty", "STANDARD"),
                                "time_estimate": step.get("time_estimate", 30)
                            }
                            learning_path.append(standardized_step)
                    except Exception as e:
                        logger.warning(f"LLM fallback thất bại: {str(e)}. Trả về lộ trình mặc định với 1 bước.")
                        learning_path = [
                            {
                                "step": 1,
                                "node_id": start_node,
                                "sanitized_concept": "Not Available",
                                "context": context or "general",
                                "definition": "Tổ chức dữ liệu dựa trên một tiêu chí chung.",
                                "example": "Not Available",
                                "learning_objective": f"Học tri thức liên quan đến {start_node} và để đạt được mục tiêu {student_goal or 'general learning'}.",
                                "skill_level": level,
                                "difficulty": "STANDARD",
                                "time_estimate": 30
                            }
                        ]
                else:
                    logger.warning("LLM fallback response không hợp lệ (không có thuộc tính response).")
                    learning_path = [
                        {
                            "step": 1,
                            "node_id": start_node,
                            "sanitized_concept": "Unknown Concept",
                            "context": context or "general",
                            "definition": "Không có định nghĩa",
                            "example": "Không có ví dụ",
                            "learning_objective": f"Học {start_node}",
                            "skill_level": level,
                            "difficulty": "STANDARD",
                            "time_estimate": 30
                        }
                    ]
                return {"path": learning_path, "status": "success", "error_message": None}

        else:
            goal_tags = get_goal_tags(driver, goal_node)
            calculate_and_store_similarity(driver, goal_tags)
            path = a_star_custom(start_node, goal_node, driver, goal_tags, dynamic_weights, Config, context)
            if not path:
                return {"path": [], "status": "error", "error_message": "Không có lộ trình"}

        # Xây dựng lộ trình học tập
        learning_path = []
        for idx, node_id in enumerate(path):
            query = f"""
            MATCH (n {{{Config.PROPERTY_ID}: $node_id}})
            RETURN properties(n) AS props
            """
            result = execute_cypher_query(driver, query, params={"node_id": node_id})
            if result:
                node_props = result[0]["props"]
                standardized_step_data = {key.lower().replace(" ", "_"): value for key, value in node_props.items()}
                standardized_step_data["step"] = idx + 1
                learning_path.append(standardized_step_data)

        if not learning_path:
            logger.warning("Không tìm thấy lộ trình học tập hợp lệ")
            return {"path": [], "status": "error", "error_message": "Không tìm thấy lộ trình học tập hợp lệ"}

        return {"path": learning_path, "status": "success", "error_message": None}

    except Exception as e:
        logger.error(f"Lỗi trong generate_learning_path: {str(e)}")
        return {"path": [], "status": "error", "error_message": str(e)}

In [ ]:
def a_star_custom(start_node: str, goal_node: str, driver, goal_tags: List[str], dynamic_weights: Dict, Config, context: str) -> List[str]:
    heuristic_cache = {}  # Bộ nhớ đệm heuristic
    open_set = []
    heapq.heappush(open_set, (0, start_node))
    came_from = {}
    g_score = {start_node: 0}
    f_score = {start_node: heuristic(start_node, driver, goal_tags, dynamic_weights, Config, context, heuristic_cache)}

    while open_set:
        current = heapq.heappop(open_set)[1]

        if current == goal_node:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start_node)
            return path[::-1]

        query = f"""
        MATCH (n {{{Config.PROPERTY_ID}: $node_id}})-[r]->(neighbor)
        RETURN neighbor.{Config.PROPERTY_ID} AS id, r.Weight AS weight
        """
        neighbors = execute_cypher_query(driver, query, params={"node_id": current})

        for neighbor in neighbors:
            tentative_g_score = g_score[current] + neighbor["weight"]
            neighbor_id = neighbor["id"]
            if neighbor_id not in g_score or tentative_g_score < g_score[neighbor_id]:
                came_from[neighbor_id] = current
                g_score[neighbor_id] = tentative_g_score
                f_score[neighbor_id] = tentative_g_score + heuristic(neighbor_id, driver, goal_tags, dynamic_weights, Config, context, heuristic_cache)
                heapq.heappush(open_set, (f_score[neighbor_id], neighbor_id))

    return None




In [ ]:
def extract_cypher_query(response_text: str) -> str:
    """Trích xuất truy vấn Cypher từ phản hồi của LLM."""
    match = re.search(r'\b(MATCH|CALL)\b.*', response_text, re.DOTALL | re.IGNORECASE)
    if match:
        return match.group(0).strip()
    logger.warning("Không tìm thấy truy vấn Cypher hợp lệ trong phản hồi LLM")
    return ""

In [ ]:
generate_learning_path(student_id="stu001",start_node='concept:transactions', level="Remember",context="", learning_style="reading_writing",student_goal="sql basics")

{'path': [],
 'status': 'error',
 'error_message': "Nút khởi đầu 'concept:transactions' không tồn tại trong Knowledge Graph."}

# calculate_dynamic_weights

In [ ]:
# Hàm tính trọng số động
def calculate_dynamic_weights(learning_data_file: str = Config.LEARNING_DATA_FILE) -> Dict:
    """Tính trọng số động cho thuật toán A* dựa trên dữ liệu học tập.

    Args:
        learning_data_file (str): Đường dẫn đến file dữ liệu học tập.

    Returns:
        Dict: Trọng số động cho heuristic A*.
    """
    try:
        if not os.path.exists(learning_data_file):
            logger.info(f"File dữ liệu học tập {learning_data_file} không tồn tại, sử dụng trọng số mặc định")
            return Config.ASTAR_HEURISTIC_WEIGHTS

        df = pd.read_csv(learning_data_file)
        if df.empty:
            logger.info("File dữ liệu học tập rỗng, sử dụng trọng số mặc định")
            return Config.ASTAR_HEURISTIC_WEIGHTS

        node_stats = df.groupby("NodeID").agg({
            "Score": "mean",
            "TimeSpent": "mean"
        }).to_dict()

        dynamic_weights = Config.ASTAR_HEURISTIC_WEIGHTS.copy()
        for node_id in node_stats["Score"]:
            avg_score = node_stats["Score"][node_id]
            avg_time = node_stats["TimeSpent"][node_id]
            if avg_score > 80 and avg_time < 30:
                dynamic_weights["priority"] += 0.5
                logger.info(f"Tăng trọng số priority cho node {node_id} do hiệu suất cao")
            elif avg_score < 50:
                dynamic_weights["difficulty_advanced"] += 1.0
                logger.info(f"Tăng trọng số difficulty_advanced cho node {node_id} do hiệu suất thấp")

        return dynamic_weights

    except Exception as e:
        logger.error(f"Lỗi trong calculate_dynamic_weights: {str(e)}")
        return Config.ASTAR_HEURISTIC_WEIGHTS


# adjust_heuristic_weights()
Adaptive E-learning system based on learner portraits and knowledge graph" của Chen và Huang (2023), được công bố trong IEEE 3rd International Conference on Information Technology Cosa, Big Data and Artificial Intelligence. Bài báo này nhấn mạnh việc sử dụng dữ liệu đa chiều (hiệu suất, độ khó, thời gian học, v.v.) để cá nhân hóa lộ trình học tập, phù hợp với mục tiêu của bạn.

In [ ]:
def adjust_heuristic_weights(feedback_data: Dict) -> None:
    """Điều chỉnh trọng số heuristic dựa trên phản hồi học sinh.

    Args:
        feedback_data (Dict): Dữ liệu phản hồi với các khóa 'performance' và 'difficulty'.
    """
    try:
        # Ánh xạ performance và difficulty thành điểm số
        performance_map = {"low": -1, "medium": 0, "high": 1}
        difficulty_map = {"STANDARD": 1, "ADVANCED": 2}

        performance = feedback_data.get("performance", "medium")
        difficulty = feedback_data.get("difficulty", "STANDARD")

        performance_score = performance_map.get(performance, 0)
        difficulty_factor = difficulty_map.get(difficulty, 1)

        # Công thức điều chỉnh đơn giản
        adjustment = performance_score * 0.1 * difficulty_factor

        # Cập nhật trọng số heuristic
        if difficulty == "STANDARD":
            Config.ASTAR_HEURISTIC_WEIGHTS["difficulty_standard"] += adjustment
        elif difficulty == "ADVANCED":
            Config.ASTAR_HEURISTIC_WEIGHTS["difficulty_advanced"] += adjustment

        logger.info(f"Đã điều chỉnh trọng số heuristic: {adjustment} cho {difficulty}")
    except Exception as e:
        logger.error(f"Lỗi trong adjust_heuristic_weights: {str(e)}")

# generate_learning_content

In [ ]:
def generate_learning_content(node: Dict, student_id: str = None, student_file: str = Config.STUDENT_FILE, student_goal: str = None) -> Dict:
    """Sinh nội dung học tập tùy chỉnh cho một nút trên lộ trình.

    Args:
        node (Dict): Thông tin nút (sanitized_concept, definition, example, learning_objective, skill_level, node_id, context, difficulty, time_estimate, semantic_tags, v.v.).
        student_id (str, optional): ID của học sinh để lấy phong cách học tập.
        student_file (str): Đường dẫn đến file hồ sơ học sinh.

    Returns:
        Dict: Dictionary chứa 'content', 'status', và 'error_message'.
    """
    try:
        # Bước 1: Kiểm tra đầu vào
        if not isinstance(node, dict):
            raise ValueError("Nút phải là một dictionary")
        required_fields = ["node_id", "sanitized_concept"]
        missing_fields = [field for field in required_fields if field not in node or not node[field]]
        if missing_fields:
            raise ValueError(f"Nút thiếu các trường bắt buộc: {missing_fields}")

        # Bước 2: Xác định phong cách học tập
        learning_style = node.get("learning_style_preference")
        if not learning_style and student_id:
            profile = load_student_profile(student_id)
            learning_style = profile.get("learning_style_preference", None)
        learning_style = learning_style or "reading_writing"

        # Bước 3: Truy vấn thông tin subconcepts (nếu có)
        query = f"""
        MATCH (n {{{Config.PROPERTY_ID}: $node_id}})<-[:{Config.RELATIONSHIP_IS_SUBCONCEPT_OF}]-(sub)
        RETURN sub.{Config.PROPERTY_DEFINITION} AS definition, sub.{Config.PROPERTY_EXAMPLE} AS example
        """
        subconcepts = execute_cypher_query(driver, query, params={"node_id": node["node_id"]})
        subconcept_content = ""
        if subconcepts:
            subconcept_content = "\n".join([f"Khái niệm nền: {sub['definition']} (Ví dụ: {sub['example']})" for sub in subconcepts])

        # Bước 4: Chuẩn bị các phần thông tin cho prompt từ tất cả thuộc tính của node và subconcepts
        node_info = "\n".join([f"- {key.replace('_', ' ').title()}: {value if value else 'N/A'}" for key, value in node.items()])
        if subconcept_content:
            node_info += f"\n\nĐể hiểu rõ hơn, bạn cần nắm các khái niệm nền sau:\n{subconcept_content}"

        # Bước 5: Tạo prompt cho LLM với tất cả thông tin từ node và subconcepts
        # prompt_template = PromptTemplate(f"""
        # # MISSION
        # Bạn là một chuyên gia giáo dục, nhiệm vụ của bạn là tạo ra nội dung học tập **bằng tiếng Việt** chi tiết, dễ hiểu và phù hợp với học sinh dựa trên thông tin sau:

        # - **Mục tiêu học tập của học sinh**: {student_goal or 'N/A'}
        # - **Thông tin khái niệm**: {node_info}

        # # METHODOLOGY
        # - Đóng vai là một chuyên gia hiểu rõ về mục tiêu học tập của học sinh: {student_goal}
        # - **Tập trung giải thích khái niệm chính**: Khái niệm chính là "{node['sanitized_concept']}". Hãy giải thích nó một cách rõ ràng, dễ hiểu, và liên kết với các thông tin khác như định nghĩa, ví dụ, và mục tiêu học tập.
        # - **Cung cấp ví dụ thực tế**: Đưa ra ít nhất một ví dụ cụ thể để minh họa khái niệm chính.
        # - **Liên kết với mục tiêu học tập**: Giải thích cách khái niệm này giúp học sinh đạt được mục tiêu học tập "{node.get('learning_objective', 'N/A')}" ở trình độ kỹ năng "{node.get('skill_level', 'N/A')}".
        # - **Tích hợp kỹ thuật Socratic & Thought-Provoking**: Đặt ít nhất một câu hỏi hoặc tình huống giả định để khuyến khích học sinh suy nghĩ sâu hơn. Ví dụ: "Nếu bạn thay đổi X thành Y, điều gì sẽ xảy ra? Hãy khám phá lý do."
        # - **Điều chỉnh theo phong cách học tập**: Nội dung phải phù hợp với phong cách học tập "{learning_style}". Ví dụ:
        #   - Nếu là "visual", hãy mô tả bằng hình ảnh hoặc biểu đồ.
        #   - Nếu là "auditory", hãy giải thích bằng ví dụ âm thanh hoặc câu chuyện.
        #   - Nếu là "reading_writing", tập trung vào văn bản và ghi chú.
        #   - Nếu là "kinesthetic", đưa ra hoạt động thực hành hoặc ví dụ thực tiễn.
        # - **Nếu có khái niệm nền**, trình bày theo cấu trúc ba bước: xác định, giải thích, minh họa.
        # - **Nếu thông tin nào đó không có (N/A)**, hãy tự tạo nội dung phù hợp dựa trên kiến thức chung.
        # - Trả về văn bản thuần túy.

        # # SUBCONCEPTS GUIDANCE
        # Nếu có khái niệm nền, trình bày riêng biệt từng khái niệm theo cấu trúc ba bước sau:
        # 1. **Xác định**: Nêu rõ khái niệm nền là gì.
        # 2. **Giải thích**: Mô tả ngắn gọn ý nghĩa và vai trò của nó.
        # 3. **Minh họa**: Đưa ra một ví dụ đơn giản để làm rõ khái niệm.

        # # OUTPUT
        # - Phần nội dung mà học sinh sẽ dùng để học tập.
        # - Lưu ý trả về dạng văn bản học tập, bỏ qua các đại từ nhân xưng.
        # """)
        prompt_template = PromptTemplate(f"""
        # MISSION
        Tạo nội dung học tập **bằng tiếng Việt** chi tiết, mang tính học thuật, dễ hiểu, và phù hợp với học sinh, đạt độ dài tối thiểu **3.000-4.000 từ**, dựa trên thông tin sau:
        - **Mục tiêu học tập của học sinh**: {student_goal or 'N/A'}
        - **Thông tin khái niệm**: {node_info}

        # METHODOLOGY
        - Đóng vai chuyên gia giáo dục am hiểu mục tiêu học tập: {student_goal}.
        - **Giải thích khái niệm chính**: Khái niệm chính là "{node['sanitized_concept']}". Trình bày chi tiết qua các phần:
          - **Giới thiệu**: Đặt khái niệm trong bối cảnh thực tế, giải thích tầm quan trọng, và trình bày bối cảnh lịch sử/phát triển của khái niệm (300-400 từ).
          - **Giải thích chi tiết**: Mô tả ý nghĩa, cơ chế hoạt động, và vai trò của khái niệm, liên kết với lý thuyết giáo dục (ví dụ: Bloom’s Taxonomy, VARK) và các mô hình học tập liên quan (700-1.000 từ).
          - **Ví dụ thực tế**: Đưa ra ít nhất bốn ví dụ cụ thể, đa dạng, minh họa ứng dụng trong các tình huống khác nhau (ví dụ: học thuật, nghề nghiệp, đời sống) (400-600 từ).
          - **Ứng dụng thực tế**: Phân tích cách áp dụng khái niệm trong ít nhất ba kịch bản thực tế (học tập, công việc, hoặc xã hội), bao gồm lợi ích và thách thức (400-600 từ).
          - **So sánh và phân biệt**: So sánh khái niệm với ít nhất hai khái niệm liên quan để làm rõ sự khác biệt và điểm tương đồng, nhấn mạnh ưu/nhược điểm (300-400 từ).
          - **Phân tích nâng cao**: Thảo luận các khía cạnh phức tạp, tranh luận học thuật, hoặc ứng dụng nâng cao của khái niệm (300-400 từ).
          - **Tóm tắt**: Nhấn mạnh ý chính, lợi ích lâu dài, và giá trị của việc hiểu khái niệm (150-200 từ).
        - **Liên kết với mục tiêu học tập**: Giải thích cách khái niệm giúp đạt được mục tiêu "{node.get('learning_objective', 'N/A')}" ở trình độ kỹ năng "{node.get('skill_level', 'N/A')}", liên hệ với các cấp độ tư duy của Bloom (200-300 từ).
        - **Tích hợp kỹ thuật Socratic**: Đặt ít nhất bốn câu hỏi hoặc tình huống giả định để khuyến khích tư duy phản biện, ví dụ:
          - "Điều gì xảy ra nếu không áp dụng khái niệm này trong tình huống X?"
          - "Làm thế nào để tối ưu hóa khái niệm trong bối cảnh Y?"
          - "Tại sao khái niệm này quan trọng hơn khái niệm Z trong trường hợp cụ thể?"
          - "Nếu thay đổi yếu tố A thành B, kết quả sẽ thay đổi thế nào?" (200-300 từ).
          - **Đáp án mẫu**: Cung cấp hướng dẫn trả lời hoặc đáp án mẫu cho mỗi câu hỏi, giải thích lý do và cách tư duy để đạt câu trả lời (300-400 từ tổng cộng).
        - **Điều chỉnh theo phong cách học tập**: Tùy chỉnh nội dung theo "{learning_style}":
          - **Visual**: Mô tả cách sử dụng sơ đồ tư duy, biểu đồ, hoặc hình ảnh minh họa chi tiết, kèm hướng dẫn vẽ sơ đồ (100-150 từ).
          - **Auditory**: Đề xuất câu chuyện, kịch bản âm thanh, hoặc gợi ý tổ chức thảo luận nhóm (100-150 từ).
          - **Reading/Writing**: Cung cấp văn bản chi tiết, gợi ý ghi chú, bài tập viết phản ánh, hoặc tóm tắt bằng văn bản (100-150 từ).
          - **Kinesthetic**: Đưa ra thí nghiệm, dự án thực hành, hoặc hoạt động mô phỏng, kèm hướng dẫn thực hiện (100-150 từ).
        - **Tăng tính tương tác**: Bao gồm:
          - **Năm câu hỏi trắc nghiệm**: Mỗi câu có 4 đáp án, kèm giải thích chi tiết cho đáp án đúng và lý do các đáp án sai là không phù hợp (300-400 từ).
          - **Hai bài tập thực hành**: Một bài tập cơ bản và một bài tập nâng cao, liên quan đến khái niệm, với hướng dẫn thực hiện (200-300 từ).
          - **Tài nguyên bổ sung**: Gợi ý ít nhất năm tài nguyên (sách, bài báo, video, khóa học trực tuyến) với mô tả ngắn về nội dung và cách sử dụng (200-300 từ).
        - **Xử lý khái niệm nền**: Nếu có, trình bày chi tiết ít nhất ba khái niệm nền theo ba bước:
          1. **Xác định**: Khái niệm nền là gì.
          2. **Giải thích**: Ý nghĩa, vai trò, và mối liên hệ với khái niệm chính.
          3. **Minh họa**: Ví dụ thực tế hoặc tình huống cụ thể (250-350 từ/khái niệm).
        - **Trường hợp thiếu thông tin (N/A)**: Tự tạo nội dung hợp lý dựa trên kiến thức chung, đảm bảo liên quan đến giáo dục và mục tiêu học tập.
        - Trả về văn bản thuần túy, không dùng đại từ nhân xưng, ngôn ngữ chính xác, khách quan, phù hợp với tài liệu học thuật.

        # SUBCONCEPTS GUIDANCE
        Với khái niệm nền, trình bày rõ mối liên hệ với khái niệm chính, đảm bảo mỗi khái niệm nền được giải thích đầy đủ, minh họa bằng ví dụ cụ thể, và liên kết với mục tiêu học tập.

        # OUTPUT
        - Nội dung học tập hoàn chỉnh, đạt độ dài **3.000-4.000 từ**, có cấu trúc logic từ tổng quan đến chi tiết, dễ theo dõi, mang tính học thuật với các phần được phân chia rõ ràng (giới thiệu, giải thích, ví dụ, ứng dụng, so sánh, phân tích, tóm tắt, câu hỏi, bài tập, tài nguyên). Bao gồm đáp án mẫu và giải thích cho câu hỏi tự kiểm tra và Socratic.
""")

        # Bước 6: Gọi LLM để sinh nội dung
        content = Settings.llm.complete(prompt_template.format()).text.strip()

        if not content:
            raise ValueError("LLM trả về nội dung rỗng")

        return {
            "content": content,
            "status": "success",
            "error_message": None
        }

    except Exception as e:
        logger.error(f"Lỗi trong generate_learning_content: {str(e)}")
        return {
            "content": "",
            "status": "error",
            "error_message": str(e)
        }

# generate_quiz

In [ ]:
def generate_quiz(node: Dict, output_dir: str = Config.QUIZ_OUTPUT_DIR) -> Dict:
    """Tạo bài kiểm tra với 20 câu hỏi trắc nghiệm cho một nút.

    Args:
        node (Dict): Thông tin nút (node_id, sanitized_concept, context, definition, example, learning_objective, skill_level, difficulty).
        output_dir (str): Thư mục lưu file bài kiểm tra.

    Returns:
        Dict: Dictionary chứa 'student_file', 'answer_file', 'status', và 'error_message'.
    """
    try:
        # Bước 1: Kiểm tra đầu vào
        if not isinstance(node, dict):
            raise ValueError("Nút phải là một dictionary")

        # Ghi log nội dung node để debug
        logger.debug(f"Nội dung node: {node}")

        required_fields = [
            Config.PROPERTY_ID.lower(),  # node_id
            Config.PROPERTY_SANITIZED_CONCEPT.lower(),  # sanitized_concept
            Config.PROPERTY_CONTEXT.lower(),  # context
            Config.PROPERTY_DEFINITION.lower(),  # definition
            Config.PROPERTY_EXAMPLE.lower(),  # example
            Config.PROPERTY_LEARNING_OBJECTIVE.lower(),  # learning_objective
            Config.PROPERTY_SKILL_LEVEL.lower(),  # skill_level
            Config.PROPERTY_DIFFICULTY.lower()  # difficulty
        ]
        missing_fields = [field for field in required_fields if field not in node or not node[field]]
        if missing_fields:
            raise ValueError(f"Nút thiếu các trường bắt buộc: {missing_fields}")

        # Bước 2: Tạo prompt cho LLM
        common_errors = node.get('common_errors', [])
        common_errors_str = ", ".join(common_errors) if isinstance(common_errors, list) else common_errors if isinstance(common_errors, str) else ""

        prompt_parts = [
            "Dựa trên metadata của nút:",
            f"- Khái niệm: {node.get('sanitized_concept', '')}",
            f"- Ngữ cảnh: {node.get('context', '')}",
            f"- Định nghĩa: {node.get('definition', '')}",
            f"- Ví dụ: {node.get('example', '')}",
            f"- Mục tiêu học tập: {node.get('learning_objective', '')}",
            f"- Trình độ kỹ năng: {node.get('skill_level', '')}",
            f"- Độ khó: {node.get('difficulty', '')}",
            f"- Lỗi thường gặp: {common_errors_str}" if common_errors_str else "Không có lỗi thường gặp"
        ]

        prompt = PromptTemplate("\n".join(part for part in prompt_parts) + f"""
        Tạo chính xác {Config.QUIZ_NUM_QUESTIONS} câu hỏi trắc nghiệm **bằng tiếng Việt**, mỗi câu có 4 lựa chọn, dựa trên thông tin nút ở trên:
        - {int(Config.QUIZ_DISTRIBUTION['basic'] * 100)}% câu hỏi cơ bản (kiến thức nền, khái niệm cơ bản).
        - {int(Config.QUIZ_DISTRIBUTION['intermediate'] * 100)}% câu hỏi trung cấp (ứng dụng đơn giản, ví dụ thực tế).
        - {int(Config.QUIZ_DISTRIBUTION['advanced'] * 100)}% câu hỏi nâng cao (phân tích, suy luận).
        - Đảm bảo câu hỏi liên quan chặt chẽ đến khái niệm chính, sử dụng định nghĩa, ví dụ, và mục tiêu học tập để tạo nội dung.
        - Nếu có lỗi thường gặp, đưa chúng vào các đáp án sai để kiểm tra sự hiểu biết của học sinh.
        - Mỗi câu hỏi phải có lời giải thích chi tiết (explanation) để học sinh hiểu lý do chọn đáp án đúng.
        - Lưu ý:
            + Phải tạo chính xác số lượng {Config.QUIZ_NUM_QUESTIONS} câu hỏi.
            + Câu trả lời chính xác luôn luôn ở vị trí index 0, hay tương ứng là vị trí "Option 1".
        Trả về JSON:
        {{
            "questions": [
                {{
                    "text": "Question text",
                    "options": ["Option 1", "Option 2", "Option 3", "Option 4"],
                    "correct_index": 0-3,
                    "explanation": "Explanation text",
                    "level": "basic|intermediate|advanced"
                }}
            ]
        }}
        """)

        # Bước 3: Gọi LLM để sinh bài kiểm tra
        for attempt in range(2):  # Thử tối đa 2 lần
            try:
                quiz_json = Settings.llm.complete(
                    prompt.format(
                        num_questions=Config.QUIZ_NUM_QUESTIONS,
                        basic_pct=int(Config.QUIZ_DISTRIBUTION["basic"] * 100),
                        intermediate_pct=int(Config.QUIZ_DISTRIBUTION["intermediate"] * 100),
                        advanced_pct=int(Config.QUIZ_DISTRIBUTION["advanced"] * 100)
                    )
                ).text
                cleaned_quiz_str = clean_json_response(quiz_json)
                quiz = json.loads(cleaned_quiz_str)
                if not isinstance(quiz, dict) or "questions" not in quiz or not isinstance(quiz["questions"], list):
                    raise ValueError("Phản hồi LLM không hợp lệ")
                break
            except (json.JSONDecodeError, ValueError) as e:
                if attempt == 1:
                    raise ValueError(f"Không thể tạo bài kiểm tra sau 2 lần thử: {str(e)}")
                logger.warning(f"Lỗi khi gọi LLM: {str(e)}, thử lại...")

        # Bước 4: Kiểm tra tỷ lệ câu hỏi
        total_pct = Config.QUIZ_DISTRIBUTION["basic"] + Config.QUIZ_DISTRIBUTION["intermediate"] + Config.QUIZ_DISTRIBUTION["advanced"]
        if abs(total_pct - 1.0) > 0.01:
            raise ValueError("Tổng tỷ lệ câu hỏi phải bằng 100%")

        # Bước 5: Lưu bài kiểm tra vào file CSV
        os.makedirs(output_dir, exist_ok=True)
        student_file = f"{output_dir}/student_quiz_{node['node_id']}.csv"
        answer_file = f"{output_dir}/answer_key_{node['node_id']}.csv"

        with open(student_file, "w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(["Question_Number", "Question_Text", "Option_1", "Option_2", "Option_3", "Option_4", "Level"])
            for i, q in enumerate(quiz["questions"], 1):
                writer.writerow([i, q["text"]] + q["options"] + [q["level"]])

        with open(answer_file, "w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(["Question_Number", "type", "correct", "explanation", "options"])
            for i, q in enumerate(quiz["questions"], 1):
                options_str = ",".join(q["options"])  # Chuyển danh sách options thành chuỗi
                writer.writerow([i, "multiple_choice", q["correct_index"], q["explanation"], options_str])

        return {
            "student_file": student_file,
            "answer_file": answer_file,
            "status": "success",
            "error_message": None
        }

    except Exception as e:
        logger.error(f"Lỗi trong generate_quiz: {str(e)}")
        return {
            "student_file": None,
            "answer_file": None,
            "status": "error",
            "error_message": str(e)
        }

# evaluate_quiz

In [ ]:
def evaluate_quiz(student_file: str, answer_file: str) -> Dict:
    """Đánh giá bài kiểm tra của học sinh dựa trên file đáp án.

    Args:
        student_file (str): Đường dẫn đến file CSV chứa câu trả lời của học sinh.
        answer_file (str): Đường dẫn đến file CSV chứa đáp án đúng.

    Returns:
        Dict: Dictionary chứa 'score', 'total_questions', 'correct_count', 'feedback', 'status', và 'error_message'.
    """
    try:
        # Đọc file đáp án
        with open(answer_file, "r", encoding="utf-8") as f:
            answer_reader = csv.DictReader(f)
            answer_data = {row["Question_Number"]: row for row in answer_reader}

        # Đọc file câu trả lời của học sinh
        with open(student_file, "r", encoding="utf-8") as f:
            student_reader = csv.DictReader(f)
            student_data = {row["Question_Number"]: row["Student_Answer"].strip().lower() for row in student_reader}

        total_questions = len(answer_data)
        correct_count = 0
        feedback = []

        # Đánh giá từng câu hỏi
        for q_num, answer in answer_data.items():
            if q_num not in student_data:
                feedback.append(f"Câu hỏi {q_num}: Thiếu câu trả lời từ học sinh.")
                continue

            question_type = answer["type"]
            correct_answer = answer["correct"]
            explanation = answer["explanation"]
            student_answer = student_data[q_num]

            # Xử lý câu hỏi trắc nghiệm (multiple_choice)
            if question_type == "multiple_choice":
                options = [opt.strip().lower() for opt in answer["options"].split(",")]
                try:
                    correct_index = int(correct_answer)
                    correct_option = options[correct_index]
                except (ValueError, IndexError):
                    logger.error(f"Lỗi định dạng đáp án đúng '{correct_answer}' cho câu hỏi {q_num}")
                    feedback.append(f"Câu hỏi {q_num}: Lỗi định dạng đáp án đúng trong file đáp án.")
                    continue

                if student_answer == correct_option:
                    correct_count += 1
                else:
                    feedback.append(f"Câu hỏi {q_num}: Sai. Đáp án đúng là '{correct_option}'. Giải thích: {explanation}")
            else:
                # Trường hợp không mong đợi: ghi log và giả định là trắc nghiệm
                logger.warning(f"Loại câu hỏi không mong đợi '{question_type}' cho câu hỏi {q_num}. Giả định là multiple_choice.")
                options = [opt.strip().lower() for opt in answer["options"].split(",")]
                try:
                    correct_index = int(correct_answer)
                    correct_option = options[correct_index]
                except (ValueError, IndexError):
                    logger.error(f"Lỗi định dạng đáp án đúng '{correct_answer}' cho câu hỏi {q_num}")
                    feedback.append(f"Câu hỏi {q_num}: Lỗi định dạng đáp án đúng trong file đáp án.")
                    continue

                if student_answer == correct_option:
                    correct_count += 1
                else:
                    feedback.append(f"Câu hỏi {q_num}: Sai. Đáp án đúng là '{correct_option}'. Giải thích: {explanation}")

        # Tính điểm
        score = (correct_count / total_questions) * 100 if total_questions > 0 else 0.0

        return {
            "score": f"{score:.0f}",
            "total_questions": total_questions,
            "correct_count": correct_count,
            "feedback": feedback,
            "status": "success",
            "error_message": None
        }

    except FileNotFoundError as e:
        logger.error(f"Không tìm thấy file: {str(e)}")
        return {
            "score": "0.00",
            "total_questions": 0,
            "correct_count": 0,
            "feedback": [],
            "status": "error",
            "error_message": f"Không tìm thấy file: {str(e)}"
        }
    except Exception as e:
        logger.error(f"Lỗi trong evaluate_quiz: {str(e)}")
        return {
            "score": "0.00",
            "total_questions": 0,
            "correct_count": 0,
            "feedback": [],
            "status": "error",
            "error_message": str(e)
        }

# update_student_profile

In [ ]:
def update_student_profile(student_id: str, node: Dict, score: float, time_spent: float) -> Dict:
    """Cập nhật hồ sơ học sinh dựa trên hiệu suất học tập trong Neo4j.

    Args:
        student_id (str): ID của học sinh.
        node (Dict): Thông tin nút (node_id, time_estimate, skill_level).
        score (float): Điểm bài kiểm tra (0-100).
        time_spent (float): Thời gian dành cho nút (phút).

    Returns:
        Dict: Dictionary chứa 'status' và 'error_message'.
    """
    try:
        # Bước 1: Kiểm tra đầu vào
        if not isinstance(student_id, str) or not student_id:
            raise ValueError("student_id phải là chuỗi không rỗng")
        if not isinstance(node, dict):
            raise ValueError("Nút phải là một dictionary")
        required_node_fields = [Config.PROPERTY_ID, "time_estimate", "skill_level"]
        missing_fields = [field for field in required_node_fields if field not in node]
        if missing_fields:
            raise ValueError(f"Nút thiếu các trường bắt buộc: {missing_fields}")
        if not isinstance(score, (int, float)) or score < 0 or score > 100:
            raise ValueError("Điểm phải là số từ 0 đến 100")
        if not isinstance(time_spent, (int, float)) or time_spent < 0:
            raise ValueError("Thời gian dành phải là số không âm")

        # Bước 2: Tải hồ sơ học sinh từ Neo4j
        profile = load_student_profile(student_id)
        if profile is None:
            raise ValueError(f"Không tìm thấy hồ sơ học sinh với ID: {student_id}")
        learning_history = profile.get("learning_history", [])
        node_id = node[Config.PROPERTY_ID]
        if node_id not in learning_history:
            learning_history.append(node_id)

        # Bước 3: Cập nhật chi tiết hiệu suất
        perf_detail = f"{node_id}:{score}:{node['time_estimate']},{node['skill_level']}"
        perf_details = [d for d in profile.get("performance_details", []) if not d.startswith(node_id)] + [perf_detail]

        # Bước 4: Tính lại trình độ hiện tại
        total_score = 0
        total_weight = 0
        for d in perf_details:
            try:
                parts = d.split(":")
                if len(parts) != 3:
                    logger.warning(f"Định dạng chi tiết hiệu suất không hợp lệ: {d}. Bỏ qua.")
                    continue
                score_value = float(parts[1])
                weight = 1 if "STANDARD" in parts[2] else 2
                total_score += score_value * weight
                total_weight += weight
            except (ValueError, IndexError) as e:
                logger.warning(f"Lỗi khi xử lý chi tiết hiệu suất {d}: {str(e)}. Bỏ qua.")
                continue
        current_level = round(total_score / total_weight, 2) if total_weight > 0 else 0

        # Bước 5: Gửi thông báo nếu thay đổi trình độ đáng kể
        prev_level = profile.get("current_level", 0)
        if abs(current_level - prev_level) > 20:
            notification = f"Học sinh {student_id} có thay đổi trình độ từ {prev_level} lên {current_level} vào {datetime.now().isoformat()}\n"
            with open(Config.TEACHER_NOTIFICATION_FILE, "a", encoding="utf-8") as f:
                f.write(notification)
            logger.info(f"Đã gửi thông báo: {notification.strip()}")

        # Bước 6: Cập nhật vào Neo4j
        last_updated = datetime.now().isoformat()
        query = """
        MATCH (s:Student {StudentID: $student_id})
        MATCH (n {Node_ID: $node_id})
        MERGE (s)-[r:MASTERED]->(n)
        SET r.progress = $score,
            r.timestamp = datetime(),
            r.time_spent = $time_spent,
            s.learning_history = $learning_history,
            s.current_level = $current_level,
            s.performance_details = $performance_details,
            s.LastUpdated = $last_updated
        """
        params = {
            "student_id": student_id,
            "node_id": node_id,
            "score": score,
            "time_spent": time_spent,
            "learning_history": ",".join(learning_history),
            "current_level": current_level,
            "performance_details": ";".join(perf_details),
            "last_updated": last_updated
        }
        execute_cypher_query(driver, query, params)

        return {"status": "success", "error_message": None}

    except Exception as e:
        logger.error(f"Lỗi trong update_student_profile: {str(e)}")
        return {
            "status": "error",
            "error_message": str(e)
        }

# update_learning_path

In [ ]:
# Hàm cập nhật lộ trình học tập
def update_learning_path(student_id: str, node: Dict, current_path: List[Dict], score: float, max_attempts: int = 2) -> Dict:
    """Cập nhật lộ trình học tập dựa trên hiệu suất học sinh.

    Args:
        student_id (str): ID của học sinh.
        node (Dict): Thông tin nút hiện tại.
        current_path (List[Dict]): Lộ trình học tập hiện tại.
        score (float): Điểm bài kiểm tra (0-100).
        max_attempts (int): Số lần thử tối đa trước khi chuyển lộ trình thay thế.

    Returns:
        Dict: Dictionary chứa 'path', 'status', và 'error_message'.
    """
    try:
        # Bước 1: Kiểm tra đầu vào
        if not isinstance(student_id, str) or not student_id:
            raise ValueError("student_id phải là chuỗi không rỗng")
        if not isinstance(node, dict):
            raise ValueError("Nút phải là một dictionary")
        required_fields = [Config.PROPERTY_ID, "difficulty", "context"]
        missing_fields = [field for field in required_fields if field not in node]
        if missing_fields:
            raise ValueError(f"Nút thiếu các trường bắt buộc: {missing_fields}")
        if not isinstance(current_path, list):
            raise ValueError("Lộ trình hiện tại phải là một danh sách")
        if not isinstance(score, (int, float)) or score < 0 or score > 100:
            raise ValueError("Điểm phải là số từ 0 đến 100")

        # Bước 2: Kiểm tra nút có trong lộ trình không
        node_id = node.get(Config.PROPERTY_ID)
        if node_id is None:
            raise ValueError("Nút thiếu trường bắt buộc: Config.PROPERTY_ID")

        current_node = next((n for n in current_path if n.get(Config.PROPERTY_ID) == node_id), None)
        if current_node is None:
            logger.warning(f"Nút {node_id} không có trong lộ trình hiện tại. Không cập nhật.")
            return {
                "path": current_path,
                "status": "success",
                "error_message": None
            }

        # Bước 3: Tải hồ sơ và kiểm tra số lần thử
        profile = load_student_profile(student_id)
        if profile is None:
            raise ValueError(f"Không tìm thấy hồ sơ học sinh với ID: {student_id}")
        performance_details = profile.get("performance_details", [])
        node_attempts = sum(1 for detail in performance_details if detail.startswith(node_id))

        # Bước 4: Xử lý nếu điểm thấp hơn ngưỡng
        if score < Config.QUIZ_PASSING_SCORE:
            if score <= 10:
                with open(Config.TEACHER_NOTIFICATION_FILE, "a", encoding="utf-8") as f:
                        f.write(f"Học sinh {student_id} thất bại tại {node_id} sau {node_attempts} lần. Mất căn bản tri thức này. Cần chú ý.\n")
            if node_attempts >= max_attempts:
                # Tìm lộ trình thay thế
                alt_node = _find_alternative_node(node_id, node["difficulty"], node["context"])
                if alt_node:
                    with open(Config.TEACHER_NOTIFICATION_FILE, "a", encoding="utf-8") as f:
                        f.write(f"Học sinh {student_id} thất bại tại {node_id} sau {node_attempts} lần. Cần chú y! Hệ thống đã tự động chuyển sang lộ trình thay thế.\n")
                    logger.info(f"Chuyển sang lộ trình thay thế: {alt_node['id']}")
                    current_path[current_path.index(current_node)] = _construct_node(alt_node, current_node)

                else:
                    logger.warning(f"Không tìm thấy lộ trình thay thế cho {node_id}.")
                    with open(Config.TEACHER_NOTIFICATION_FILE, "a", encoding="utf-8") as f:
                        f.write(f"Học sinh {student_id} thất bại tại {node_id} sau {node_attempts} lần. Không có lộ trình thay thế.\n")
            else:
                # Tìm nút khắc phục
                remedy_node = _find_remedy_node(node_id, node["difficulty"], node["context"])
                if remedy_node:
                    logger.info(f"Thêm nút khắc phục: {remedy_node['id']}")
                    current_path.insert(current_path.index(current_node), _construct_node(remedy_node, current_node))
                else:
                    logger.warning(f"Không tìm thấy nút khắc phục cho {node_id}. Gửi thông báo.")
                    with open(Config.TEACHER_NOTIFICATION_FILE, "a", encoding="utf-8") as f:
                        f.write(f"Học sinh {student_id} cần nút khắc phục cho {node_id} sau điểm thấp.\n")

        return {"path": current_path, "status": "success", "error_message": None}

    except Exception as e:
        logger.error(f"Lỗi trong update_learning_path: {str(e)}")
        return {"path": current_path, "status": "error", "error_message": str(e)}




# _find_alternative_node

In [ ]:
def _find_alternative_node(node_id: str, difficulty: float, context: str) -> Dict:
    """Tìm nút thay thế từ Neo4j."""
    query = f"""
    MATCH (current {{{Config.PROPERTY_ID}: $node_id}})-[r:{Config.RELATIONSHIP_HAS_ALTERNATIVE_PATH}]->(alt)
    WHERE alt.{Config.PROPERTY_DIFFICULTY} <= $difficulty
    RETURN alt.{Config.PROPERTY_ID} AS id, alt.{Config.PROPERTY_SANITIZED_CONCEPT} AS sanitized_concept,
           alt.{Config.PROPERTY_DIFFICULTY} AS difficulty, alt.{Config.PROPERTY_CONTEXT} AS context,
           r.Weight AS weight, r.Dependency AS dependency
    """
    result = execute_cypher_query(driver, query, params={"node_id": node_id, "difficulty": difficulty})
    if result:
        for row in result:
            row["context_match"] = 1 if row["context"] == context else 0
        result.sort(key=lambda x: (-x["weight"], -x["dependency"], -x["context_match"]))
        return result[0]
    return None




# _find_remedy_node

In [ ]:
def _find_remedy_node(node_id: str, difficulty: float, context: str) -> Dict:
    """Tìm nút khắc phục từ Neo4j."""
    query = f"""
    MATCH (current {{{Config.PROPERTY_ID}: $node_id}})-[r:{Config.RELATIONSHIP_REMEDIATES}]->(remedy)
    WHERE remedy.{Config.PROPERTY_DIFFICULTY} <= $difficulty
    RETURN remedy.{Config.PROPERTY_ID} AS id, remedy.{Config.PROPERTY_SANITIZED_CONCEPT} AS sanitized_concept,
           remedy.{Config.PROPERTY_DIFFICULTY} AS difficulty, remedy.{Config.PROPERTY_CONTEXT} AS context,
           r.Weight AS weight, r.Dependency AS dependency
    """
    result = execute_cypher_query(driver, query, params={"node_id": node_id, "difficulty": difficulty})
    if result:
        for row in result:
            row["context_match"] = 1 if row["context"] == context else 0
        result.sort(key=lambda x: (-x["weight"], -x["dependency"], -x["context_match"]))
        return result[0]
    return None




# _construct_node

In [ ]:
def _construct_node(query_result: Dict, original_node: Dict) -> Dict:
    """Xây dựng thông tin nút từ kết quả truy vấn."""
    return {
        "node_id": query_result["id"],
        "sanitized_concept": query_result["sanitized_concept"],
        "difficulty": query_result["difficulty"],
        "context": query_result["context"],
        "learning_objective": original_node.get("learning_objective", ""),
        "skill_level": original_node.get("skill_level", ""),
        "time_estimate": original_node.get("time_estimate", 30),
        "step": original_node["step"]
    }

# suggest_prerequisites

In [ ]:
def suggest_prerequisites(goal_node: str, student_id: str, context: str = None) -> Dict:
    """Gợi ý các kiến thức tiên quyết cần học trước khi tiếp cận mục tiêu.

    Args:
        goal_node (str): ID của nút mục tiêu.
        student_id (str): ID của học sinh.
        context (str, optional): Ngữ cảnh học tập.

    Returns:
        Dict: Dictionary chứa 'prerequisites' (danh sách kiến thức tiên quyết), 'status', và 'error_message'.
    """
    try:
        # Bước 1: Kiểm tra đầu vào
        if not isinstance(goal_node, str) or not goal_node:
            raise ValueError("goal_node phải là chuỗi không rỗng")
        if not isinstance(student_id, str) or not student_id:
            raise ValueError("student_id phải là chuỗi không rỗng")

        # Bước 2: Truy vấn các nút tiên quyết chưa được học sinh hoàn thành
        query = f"""
        MATCH (goal {{{Config.PROPERTY_ID}: $goal_node}})<-[:{Config.RELATIONSHIP_IS_PREREQUISITE_OF}]-(prereq)
        WHERE NOT EXISTS {{ MATCH (s:Student {{{Config.PROPERTY_ID}: $student_id}})-[:MASTERED]->(prereq) }}
        RETURN prereq.{Config.PROPERTY_ID} AS id, prereq.{Config.PROPERTY_SANITIZED_CONCEPT} AS sanitized_concept,
              prereq.{Config.PROPERTY_CONTEXT} AS context, prereq.{Config.PROPERTY_PRIORITY} AS priority,
              prereq.{Config.PROPERTY_DIFFICULTY} AS difficulty, prereq.{Config.PROPERTY_SKILL_LEVEL} AS skill_level,
              prereq.{Config.PROPERTY_TIME_ESTIMATE} AS time_estimate
        UNION
        MATCH (goal {{{Config.PROPERTY_ID}: $goal_node}})-[:{Config.RELATIONSHIP_REQUIRES}]->(prereq)
        WHERE NOT EXISTS {{ MATCH (s:Student {{{Config.PROPERTY_ID}: $student_id}})-[:MASTERED]->(prereq) }}
        RETURN prereq.{Config.PROPERTY_ID} AS id, prereq.{Config.PROPERTY_SANITIZED_CONCEPT} AS sanitized_concept,
              prereq.{Config.PROPERTY_CONTEXT} AS context, prereq.{Config.PROPERTY_PRIORITY} AS priority,
              prereq.{Config.PROPERTY_DIFFICULTY} AS difficulty, prereq.{Config.PROPERTY_SKILL_LEVEL} AS skill_level,
              prereq.{Config.PROPERTY_TIME_ESTIMATE} AS time_estimate
        """
        result = execute_cypher_query(driver, query, params={"goal_node": goal_node, "student_id": student_id})

        # Thêm: Nếu không có kết quả, lấy các nút IS_SUBCONCEPT_OF
        if not result:
            subconcept_query = f"""
            MATCH (goal {{{Config.PROPERTY_ID}: $goal_node}})<-[:{Config.RELATIONSHIP_IS_SUBCONCEPT_OF}]-(prereq)
            WHERE NOT EXISTS {{ MATCH (s:Student {{{Config.PROPERTY_ID}: $student_id}})-[:MASTERED]->(prereq) }}
            RETURN prereq.{Config.PROPERTY_ID} AS id, prereq.{Config.PROPERTY_SANITIZED_CONCEPT} AS sanitized_concept,
                  prereq.{Config.PROPERTY_CONTEXT} AS context, prereq.{Config.PROPERTY_PRIORITY} AS priority,
                  prereq.{Config.PROPERTY_DIFFICULTY} AS difficulty, prereq.{Config.PROPERTY_SKILL_LEVEL} AS skill_level,
                  prereq.{Config.PROPERTY_TIME_ESTIMATE} AS time_estimate
            """
            result = execute_cypher_query(driver, subconcept_query, params={"goal_node": goal_node, "student_id": student_id})

        # Bước 3: Sắp xếp kết quả
        prerequisites = []
        if result:
            if context:
                # Ưu tiên nút có ngữ cảnh phù hợp
                for row in result:
                    row["context_match"] = 1 if row["context"] == context else 0
                result.sort(key=lambda x: (-x["priority"], -x["context_match"]))
            else:
                result.sort(key=lambda x: -x["priority"])

            prerequisites = [
                {
                    "id": row["id"],
                    "sanitized_concept": row["sanitized_concept"],
                    "context": row["context"],
                    "priority": row["priority"],
                    "difficulty": row["difficulty"],
                    "skill_level": row["skill_level"],
                    "time_estimate": row["time_estimate"]
                }
                for row in result
                if all(key in row for key in ["id", "sanitized_concept", "context", "priority", "difficulty", "skill_level", "time_estimate"])
            ]

        # Bước 4: Ghi log và trả về
        if not prerequisites:
            logger.info(f"Không tìm thấy kiến thức tiên quyết cho goal node {goal_node}")
        else:
            logger.info(f"Tìm thấy {len(prerequisites)} kiến thức tiên quyết cho goal node {goal_node}")

        return {
            "prerequisites": prerequisites,
            "status": "success",
            "error_message": None
        }

    except Exception as e:
        logger.error(f"Lỗi trong suggest_prerequisites: {str(e)}")
        return {
            "prerequisites": [],
            "status": "error",
            "error_message": str(e)
        }

# get_time_spent

In [ ]:
import time

def get_time_spent(start_time=None, autosave=False):
    """Lấy thời gian học, hỗ trợ autosave hoặc nhập thủ công.

    Args:
        start_time (float, optional): Thời gian bắt đầu (epoch time).
        autosave (bool): Nếu True, tính thời gian tự động từ start_time.

    Returns:
        int: Thời gian học (phút), giới hạn từ 1 đến 1440.
    """
    if autosave:
        if start_time is None:
            print("Không có start_time. Chuyển sang nhập thủ công.")
        else:
            time_spent = int((time.time() - start_time) / 60)  # Chuyển từ giây sang phút
            print(f"Thời gian thực tế từ hệ thống: {time_spent} phút.")
            return max(1, min(time_spent, 1440))  # Giới hạn 1-1440 phút

    # Nhập thủ công nếu không dùng autosave hoặc start_time không hợp lệ
    while True:
        try:
            user_input = input("Nhập thời gian học (phút) hoặc nhấn Enter để chọn mặc định (30 phút): ").strip()
            if user_input == "":
                print("Đã chọn thời gian mặc định: 30 phút.")
                return int(30)
            time_spent = int(user_input)
            if 1 <= int(time_spent) <= 1440:
                print(f"Thời gian học được đặt: {time_spent} phút.")
                return time_spent
            else:
                print("Thời gian phải từ 1 đến 1440 phút. Vui lòng nhập lại.")
        except ValueError:
            print("Thời gian phải là số nguyên. Vui lòng nhập lại.")

****

# run_learning_session#

In [ ]:
def run_learning_session(student_id: str, level: str, context: str, learning_style: str = None, student_goal: str = None, student_file: str = Config.STUDENT_FILE, use_llm: bool = False) -> Dict:
    """Thực thi toàn bộ phiên học tập từ xác định lộ trình đến đánh giá và cập nhật.

    Args:
        student_id (str): ID của học sinh.
        level (str): Trình độ hiện tại (e.g., 'Remember', 'Understand').
        context (str): Ngữ cảnh học tập (e.g., 'math', 'programming').
        learning_style (str, optional): Phong cách học tập.
        student_goal (str, optional): Mục tiêu học tập.
        student_file (str): Đường dẫn đến file hồ sơ học sinh.

    Returns:
        Dict: Dictionary chứa 'status' và 'error_message'.
    """
    global global_start_node
    try:
        # Bước 1: Kiểm tra đầu vào
        if not isinstance(student_id, str) or not student_id:
            logger.warning("student_id phải là chuỗi không rỗng")
            raise ValueError("student_id phải là chuỗi không rỗng")

        # Bước 2: Xác định điểm bắt đầu và điểm đích
        start_node_result = determine_start_node(student_id, level, context, student_goal, student_file, skip_quiz=False)
        if start_node_result["status"] != "success":
            logger.warning(f"Lỗi xác định điểm bắt đầu: {start_node_result['error_message']}")
            raise ValueError(f"Lỗi xác định điểm bắt đầu: {start_node_result['error_message']}")
        start_node = start_node_result["start_node"]

        goal_node_result = determine_goal_node(student_id, context, student_goal, start_node, student_file)
        if goal_node_result["status"] != "success":
            logger.warning(f"Lỗi xác định điểm đích: {goal_node_result['error_message']}")
            raise ValueError(f"Lỗi xác định điểm đích: {goal_node_result['error_message']}")
        goal_node = goal_node_result["goal_node"]

        # Bước 3: Gợi ý kiến thức tiên quyết
        prereq_result = suggest_prerequisites(goal_node, student_id, context)
        if prereq_result["status"] != "success":
            logger.warning(f"Lỗi gợi ý kiến thức tiên quyết: {prereq_result['error_message']}")
        prerequisites = prereq_result["prerequisites"]

        # Bước 4: Tạo lộ trình học tập
        path_result = generate_learning_path(student_id=student_id, level=level,start_node=start_node, context=context, learning_style=learning_style, student_goal=student_goal, student_file=student_file, use_llm=use_llm)
        if path_result["status"] != "success":
            logger.warning(f"Lỗi tạo lộ trình học tập: {path_result['error_message']}")
            raise ValueError(f"Lỗi tạo lộ trình học tập: {path_result['error_message']}")
        learning_path = path_result["path"]

        if not learning_path:
            logger.warning("Không thể tạo lộ trình học tập")
            raise ValueError("Không thể tạo lộ trình học tập")

        # Bước 5: Thêm kiến thức tiên quyết vào lộ trình nếu có
        # if prerequisites:
        #     prereq_nodes = [
        #         {
        #             "step": i + 1,
        #             "node_id": prereq.get("id", "Unknown ID"),
        #             "sanitized_concept": prereq.get("sanitized_concept", "Unknown Concept"),
        #             "context": prereq.get("context", "Unknown Context"),
        #             "definition": prereq.get("definition", "Không có định nghĩa"),
        #             "example": prereq.get("example", "Không có ví dụ"),
        #             "learning_objective": f"Học {prereq.get('sanitized_concept', 'Unknown Concept')}",
        #             "skill_level": prereq.get("skill_level", "Remember"),
        #             "difficulty": prereq.get("difficulty", "STANDARD"),
        #             "time_estimate": prereq.get("time_estimate", 30)
        #         }
        #         for i, prereq in enumerate(prerequisites)
        #     ]
        #     learning_path = prereq_nodes + [
        #         {**node, "step": node["step"] + len(prereq_nodes)}
        #         for node in learning_path
        #     ]

        if prerequisites:
            prereq_nodes = [
                {
                    "step": i + 1,
                    "node_id": prereq.get("id", "Unknown ID"),
                    "sanitized_concept": prereq.get("sanitized_concept", "Unknown Concept"),
                    "context": prereq.get("context", "Unknown Context"),
                    "definition": prereq.get("definition", "Không có định nghĩa"),
                    "example": prereq.get("example", "Không có ví dụ"),
                    "learning_objective": f"Học {prereq.get('sanitized_concept', 'Unknown Concept')}",
                    "skill_level": prereq.get("skill_level", "Remember"),
                    "difficulty": prereq.get("difficulty", "STANDARD"),
                    "time_estimate": prereq.get("time_estimate", 30)
                }
                for i, prereq in enumerate(prerequisites)
            ]

            # Loại bỏ các nút trùng lặp trong prereq_nodes nếu đã có trong learning_path
            learning_path_ids = {node["node_id"] for node in learning_path}
            unique_prereq_nodes = [node for node in prereq_nodes if node["node_id"] not in learning_path_ids]

            # Kết hợp unique_prereq_nodes và learning_path
            learning_path = unique_prereq_nodes + [
                {**node, "step": node["step"] + len(unique_prereq_nodes)}
                for node in learning_path
            ]

        logger.info(f"Lộ trình học tập: {[node['node_id'] for node in learning_path]}")
        print(f"Lộ trình học tập: {[node['node_id'] for node in learning_path]}")

        # Bước 6: Thực thi phiên học tập
        current_step = 0
        learning_data = []

        # Kiểm tra số lượng bước trong lộ trình
        if len(learning_path) <= 1:
            logger.warning(f"Lộ trình học tập chỉ có {len(learning_path)} bước, không đủ để tiếp tục.")
            raise ValueError("Lộ trình học tập không đủ bước để tiếp tục.")

        while (current_step < len(learning_path)):
            node = learning_path[current_step]

            # Sinh nội dung học tập
            content_result = generate_learning_content(node, student_id, student_file, student_goal)
            if content_result["status"] != "success":
                logger.warning(f"Lỗi sinh nội dung cho nút {node['node_id']}: {content_result['error_message']}")
                current_step += 1
                continue
            print(f"\nBước {node['step']} - Nội dung học tập:\n{content_result['content']}")

            # Lưu thời gian bắt đầu cho nút hiện tại
            start_time = time.time()

            # Chờ người dùng học
            while True:
                if input('Bạn xác nhận đã học xong? (Y/N)') == 'Y':
                    break
                else:
                    time.sleep(0.3)
                    print('Cố lên! Tiếp tục học nhé!')
                    continue

            # Tạo bài kiểm tra
            quiz_result = generate_quiz(node)
            if quiz_result["status"] != "success":
                logger.warning(f"Lỗi tạo bài kiểm tra cho nút {node['node_id']}: {quiz_result['error_message']}")
                current_step += 1
                continue
            student_file = quiz_result["student_file"]
            answer_file = quiz_result["answer_file"]

            # Đọc câu hỏi từ student_file để hiển thị cho học sinh
            with open(student_file, "r", encoding="utf-8") as f:
                reader = csv.DictReader(f)
                questions = list(reader)

            # Thu thập câu trả lời học sinh
            student_answers = {}
            for q in questions:
                q_num = q["Question_Number"]
                print(f"\nCâu {q_num} ({q['Level']}): {q['Question_Text']}")
                print(f"A. {q['Option_1']}")
                print(f"B. {q['Option_2']}")
                print(f"C. {q['Option_3']}")
                print(f"D. {q['Option_4']}")
                while True:
                    try:
                        answer = input("Chọn đáp án (A B C D): ").strip().upper()
                        if answer in ['A', 'B', 'C', 'D']:
                            option_index = ord(answer) - ord('A')
                            student_answers[q_num] = q[f"Option_{option_index + 1}"]
                            break
                        print("Đầu vào không hợp lệ: Vui lòng chọn A, B, C hoặc D.")
                    except KeyboardInterrupt:
                        print("Đầu vào bị gián đoạn. Chọn đáp án mặc định: A.")
                        student_answers[q_num] = q["Option_1"]
                        break

            # Cập nhật student_file với câu trả lời của học sinh
            updated_questions = []
            for q in questions:
                q["Student_Answer"] = student_answers.get(q["Question_Number"], "")
                updated_questions.append(q)
            with open(student_file, "w", encoding="utf-8", newline='') as f:
                fieldnames = questions[0].keys()
                writer = csv.DictWriter(f, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(updated_questions)

            # Đánh giá bài kiểm tra
            quiz_eval_result = evaluate_quiz(student_file, answer_file)
            if quiz_eval_result["status"] != "success":
                logger.warning(f"Lỗi đánh giá bài kiểm tra: {quiz_eval_result['error_message']}")
                current_step += 1
                continue
            score = quiz_eval_result["score"]
            print(f"Điểm của bạn: {score}%")

            # Hỏi người dùng về autosave
            autosave = input("Bạn có muốn autosave thời gian không? (yes/no): ").lower() == "yes"

            # Lấy thời gian học
            time_spent = get_time_spent(start_time=start_time if autosave else None, autosave=autosave)
            time_spent = time_spent if time_spent else 30

            # Cập nhật hồ sơ học sinh
            update_profile_result = update_student_profile(student_id, node, score, time_spent)
            if update_profile_result["status"] != "success":
                logger.warning(f"Lỗi cập nhật hồ sơ: {update_profile_result['error_message']}")

            # Lưu dữ liệu học tập
            learning_data.append({"node_id": node["node_id"], "score": score, "time_spent": time_spent})

            # Cập nhật lộ trình nếu cần
            if int(score) >= Config.QUIZ_PASSING_SCORE:
                current_step += 1
            else:
                while True:
                    if input('Bạn có muốn cập nhật lộ trình học? (Y/N)') == 'Y':
                        path_result = update_learning_path(student_id, node, learning_path, score)
                        if path_result["status"] != "success":
                            logger.warning(f"Lỗi cập nhật lộ trình: {path_result['error_message']}")
                        else:
                            learning_path = path_result["path"]
                        break
                    else:
                        print('Không đổi lộ trình...Cố lên! Tiếp tục học nhé!')
                        break

        # Learned_node for collaborative_filtering
        learned_nodes = [
            {"node_id": data["node_id"], "score": data["score"], "time_spent": data["time_spent"]}
            for data in learning_data if int(data["score"]) >= Config.QUIZ_PASSING_SCORE
        ]

        # Bước 6.2 Thu thập phản hồi từ học sinh
        performance = input("Đánh giá hiệu suất của bạn (low/medium/high): ").strip().lower()
        difficulty = input("Đánh giá độ khó của nội dung (STANDARD/ADVANCED): ").strip().upper()

        # Kiểm tra giá trị phản hồi hợp lệ
        valid_performance = ["low", "medium", "high"]
        valid_difficulty = ["STANDARD", "ADVANCED"]
        if performance not in valid_performance or difficulty not in valid_difficulty:
            logger.warning("Phản hồi không hợp lệ. Sử dụng giá trị mặc định.\n\
            performance = 'medium'\n\
            difficulty = 'STANDARD'")
            performance = "medium"
            difficulty = "STANDARD"

        feedback_data = {
            "performance": performance,
            "difficulty": difficulty
        }

        # Gọi hàm điều chỉnh trọng số heuristic
        adjust_heuristic_weights(feedback_data)

        # Bước 7: Lưu dữ liệu học tập
        save_result = save_learning_data(student_id, learning_path, learning_data)
        if save_result["status"] != "success":
            logger.warning(f"Lỗi lưu dữ liệu: {save_result['error_message']}")
            raise ValueError(f"Lỗi lưu dữ liệu: {save_result['error_message']}")

        logger.info("Phiên học tập hoàn tất")
        print("\nPhiên học tập hoàn tất!")

        return {
            "status": "success",
            "error_message": None,
            "learned_nodes": learned_nodes
        }

    except Exception as e:
        logger.error(f"Lỗi trong run_learning_session: {str(e)}")
        return {
            "status": "error",
            "error_message": str(e)
        }

# save_learning_data

In [ ]:
def save_learning_data(student_id: str, learning_path: List[Dict], learning_data: List[Dict]) -> Dict:
    """Lưu dữ liệu lộ trình và hiệu suất học tập vào file CSV.

    Args:
        student_id (str): ID của học sinh.
        learning_path (List[Dict]): Lộ trình học tập.
        learning_data (List[Dict]): Dữ liệu hiệu suất học tập.

    Returns:
        Dict: Dictionary chứa 'status' và 'error_message'.
    """
    try:
        # Lưu lộ trình học tập
        lp_file_exists = os.path.exists(Config.LEARNING_PATHS_FILE)
        lp_header_needed = not lp_file_exists or os.stat(Config.LEARNING_PATHS_FILE).st_size == 0

        with open(Config.LEARNING_PATHS_FILE, "a", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            if lp_header_needed:
                writer.writerow(["StudentID", "LearningPath"])
            writer.writerow([student_id, ",".join([node["node_id"] for node in learning_path])])

        # Lưu dữ liệu hiệu suất
        ld_file_exists = os.path.exists(Config.LEARNING_DATA_FILE)
        ld_header_needed = not ld_file_exists or os.stat(Config.LEARNING_DATA_FILE).st_size == 0

        with open(Config.LEARNING_DATA_FILE, "a", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            if ld_header_needed:
                writer.writerow(["StudentID", "NodeID", "Score", "TimeSpent"])
            for data in learning_data:
                if not all(key in data for key in ["node_id", "score", "time_spent"]):
                    raise ValueError("learning_data thiếu các khóa cần thiết")
                writer.writerow([student_id, data["node_id"], data["score"], data["time_spent"]])

        logger.info(f"Lưu dữ liệu học tập thành công cho học sinh {student_id}")
        return {
            "status": "success",
            "error_message": None
        }

    except Exception as e:
        logger.error(f"Lỗi trong save_learning_data: {str(e)}")
        return {
            "status": "error",
            "error_message": str(e)
        }

# collaborative_filtering

In [ ]:
def collaborative_filtering(student_id: str, learning_paths_file: str = Config.LEARNING_PATHS_FILE, learned_nodes: List[str] = [], learning_data_file: str = Config.LEARNING_DATA_FILE) -> Dict:
    """Đề xuất các nút dựa trên sự tương đồng giữa các học sinh.

    Args:
        student_id (str): ID của học sinh.
        learned_nodes (List[str]): Danh sách các nút đã học trong phiên hiện tại.
        learning_paths_file (str): Đường dẫn đến file lộ trình học tập.
        learning_data_file (str): Đường dẫn đến file dữ liệu học tập.

    Returns:
        Dict: Dictionary chứa 'recommended_nodes', 'status', và 'error_message'.
    """
    try:
        # Bước 1: Kiểm tra đầu vào
        if not isinstance(student_id, str) or not student_id:
            raise ValueError("student_id phải là chuỗi không rỗng")
        if not isinstance(learning_paths_file, str) or not isinstance(learning_data_file, str):
            logger.error(f"learning_paths_file hoặc learning_data_file không phải chuỗi: {learning_paths_file}, {learning_data_file}")
            raise ValueError("learning_paths_file và learning_data_file phải là chuỗi")
        if not learning_paths_file or not learning_data_file:
            raise ValueError("Đường dẫn file không được rỗng")

        # Bước 2: Kiểm tra file đầu vào
        if not os.path.exists(learning_paths_file) or not os.path.exists(learning_data_file):
            logger.warning(f"File lộ trình hoặc dữ liệu học tập không tồn tại: {learning_paths_file}, {learning_data_file}")
            return {
                "recommended_nodes": [],
                "status": "success",
                "error_message": "File dữ liệu không tồn tại"
            }

        # Bước 3: Đọc dữ liệu
        try:
            paths_df = pd.read_csv(learning_paths_file, encoding='utf-8', quoting=csv.QUOTE_ALL, error_bad_lines=False)
            data_df = pd.read_csv(learning_data_file, encoding='utf-8', quoting=csv.QUOTE_ALL, error_bad_lines=False)
        except pd.errors.ParserError as e:
            logger.error(f"Lỗi định dạng file CSV: {str(e)}")
            return {
                "recommended_nodes": [],
                "status": "error",
                "error_message": f"Lỗi định dạng file CSV: {str(e)}"
            }

        # Kiểm tra dữ liệu trống
        if paths_df.empty or data_df.empty:
            logger.warning("Một trong các file dữ liệu trống.")
            return {
                "recommended_nodes": [],
                "status": "success",
                "error_message": "Dữ liệu trống"
            }

        # Kiểm tra các cột cần thiết
        required_columns_paths = ["StudentID", "LearningPath"]
        required_columns_data = ["StudentID", "NodeID", "Score"]
        missing_columns_paths = [col for col in required_columns_paths if col not in paths_df.columns]
        missing_columns_data = [col for col in required_columns_data if col not in data_df.columns]
        if missing_columns_paths or missing_columns_data:
            logger.error(f"File CSV thiếu cột: {missing_columns_paths} trong paths_df, {missing_columns_data} trong data_df")
            return {
                "recommended_nodes": [],
                "status": "error",
                "error_message": f"File CSV thiếu cột: {missing_columns_paths}, {missing_columns_data}"
            }

        # Bước 4: Tạo bảng pivot
        pivot_table = data_df.pivot_table(index="StudentID", columns="NodeID", values="Score", fill_value=0)

        # Xử lý cold start
        if student_id not in pivot_table.index or len(pivot_table) <= 1:
            logger.warning(f"Học sinh {student_id} không có trong dữ liệu hoặc dữ liệu không đủ. Dùng đề xuất dựa trên nội dung.")
            profile = load_student_profile(student_id)
            context = profile.get("context", "")

            if context and context.strip():
                query = f"""
                MATCH (n)
                WHERE n.{Config.PROPERTY_CONTEXT} = $context
                AND n.{Config.PROPERTY_PRIORITY} IS NOT NULL
                RETURN n.{Config.PROPERTY_ID} AS id
                ORDER BY n.{Config.PROPERTY_PRIORITY} DESC
                LIMIT 5
                """
                params = {"context": context}
            else:
                query = f"""
                MATCH (n)
                WHERE n.{Config.PROPERTY_CONTEXT} IS NULL
                AND n.{Config.PROPERTY_PRIORITY} IS NOT NULL
                RETURN n.{Config.PROPERTY_ID} AS id
                ORDER BY n.{Config.PROPERTY_PRIORITY} DESC
                LIMIT 5
                """
                params = {}

            result = execute_cypher_query(driver, query, params=params)
            recommended_nodes = [row["id"] for row in result]
            return {"recommended_nodes": recommended_nodes, "status": "success", "error_message": None}

        # Bước 5: Tính ma trận tương đồng
        similarity_matrix = cosine_similarity(pivot_table)
        student_idx = pivot_table.index.get_loc(student_id)
        similarity_scores = similarity_matrix[student_idx]
        min_similarity_threshold = Config.MIN_SIMILARITY_THRESHOLD  # 0.5
        similar_students = [i for i in np.argsort(similarity_scores)[::-1] if similarity_scores[i] > min_similarity_threshold and i != student_idx][:4]

        # Fallback nếu không tìm thấy học sinh tương đồng
        if not similar_students:
            logger.warning(f"Không tìm thấy học sinh tương đồng cho {student_id}. Dùng top học sinh tương đồng nhất.")
            similar_students = [i for i in np.argsort(similarity_scores)[::-1] if i != student_idx][:4]

        # Bước 6: Đề xuất nút
        similar_student_ids = pivot_table.index[similar_students]
        similar_data = data_df[data_df["StudentID"].isin(similar_student_ids)]
        node_scores = similar_data.groupby("NodeID")["Score"].mean()

        # Lọc các nút đã học
        student_nodes = data_df[data_df["StudentID"] == student_id]["NodeID"].unique()
        all_learned_nodes = set(student_nodes).union(set(learned_nodes))
        recommended_nodes = node_scores[(node_scores > 80) & (~node_scores.index.isin(all_learned_nodes))].index.tolist()

        if not recommended_nodes:
            logger.info(f"Không tìm thấy nút phù hợp cho học sinh {student_id}")
            return {"recommended_nodes": [], "status": "success", "error_message": "Không tìm thấy nút phù hợp"}

        logger.info(f"Đề xuất {len(recommended_nodes)} nút cho học sinh {student_id}")
        return {
            "recommended_nodes": recommended_nodes,
            "status": "success",
            "error_message": None
        }

    except Exception as e:
        logger.error(f"Lỗi trong collaborative_filtering: {str(e)}")
        return {
            "recommended_nodes": [],
            "status": "error",
            "error_message": str(e)
        }

In [ ]:
# Hàm khám phá quy tắc liên kết
def apply_apriori(learning_paths_file: str = Config.LEARNING_PATHS_FILE, min_support: float = 0.1, min_threshold: float = 0.5) -> Dict:
    """Khám phá các quy tắc liên kết trong lộ trình học tập bằng thuật toán Apriori.

    Args:
        learning_paths_file (str): Đường dẫn đến file lộ trình học tập.
        min_support (float): Ngưỡng hỗ trợ tối thiểu.
        min_threshold (float): Ngưỡng độ tin cậy tối thiểu.

    Returns:
        Dict: Dictionary chứa 'association_rules', 'status', và 'error_message'.
    """
    try:
        # Bước 1: Kiểm tra file đầu vào
        if not os.path.exists(learning_paths_file):
            logger.warning(f"File lộ trình học tập {learning_paths_file} không tồn tại")
            return {
                "association_rules": {},
                "status": "success",
                "error_message": "File lộ trình không tồn tại"
            }

        # Bước 2: Đọc dữ liệu
        df = pd.read_csv(learning_paths_file)
        if df.empty:
            logger.warning("File lộ trình học tập rỗng")
            return {
                "association_rules": {},
                "status": "success",
                "error_message": "File lộ trình rỗng"
            }

        # Kiểm tra cột 'LearningPath'
        if "LearningPath" not in df.columns:
            logger.error("File CSV thiếu cột 'LearningPath'")
            return {
                "association_rules": {},
                "status": "error",
                "error_message": "File CSV thiếu cột 'LearningPath'"
            }

        # Chuyển đổi thành danh sách giao dịch, loại bỏ giá trị không hợp lệ
        transactions = [path.split(",") for path in df["LearningPath"] if isinstance(path, str) and path.strip() and len(path.split(",")) > 0]
        if not transactions:
            logger.warning("Không có giao dịch hợp lệ trong dữ liệu")
            return {
                "association_rules": {},
                "status": "success",
                "error_message": "Không có giao dịch hợp lệ"
            }

        # Bước 3: Chuyển đổi thành one-hot encoding
        te = TransactionEncoder()
        te_ary = te.fit(transactions).transform(transactions)
        one_hot = pd.DataFrame(te_ary, columns=te.columns_)


        # Bước 4: Áp dụng thuật toán Apriori
        frequent_itemsets = apriori(one_hot, min_support=min_support, use_colnames=True)
        if frequent_itemsets.empty:
            logger.info("Không tìm thấy itemsets thường xuyên")
            return {
                "association_rules": {},
                "status": "success",
                "error_message": "Không tìm thấy itemsets thường xuyên"
            }
        rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_threshold)
        if rules.empty:
            logger.info("Không tìm thấy quy tắc liên kết")
            return {
                "association_rules": {},
                "status": "success",
                "error_message": "Không tìm thấy quy tắc liên kết"
            }

        # Bước 5: Tạo dictionary quy tắc liên kết
        association_dict = {}
        for _, row in rules.iterrows():
            antecedent = list(row["antecedents"])
            consequent = list(row["consequents"])
            confidence = row["confidence"]

            for ant in antecedent:
                if ant not in association_dict:
                    association_dict[ant] = []
                association_dict[ant].append((consequent, confidence)) # Lưu toàn bộ consequents

        logger.info(f"Tìm thấy {len(association_dict)} quy tắc liên kết")
        return {
            "association_rules": association_dict,
            "status": "success",
            "error_message": None
        }

    except Exception as e:
        logger.error(f"Lỗi trong apply_apriori: {str(e)}")
        return {
            "association_rules": {},
            "status": "error",
            "error_message": str(e)
        }

# main()

In [ ]:
# Hàm chính để chạy chương trình
def main() -> Dict:
    """Chạy chương trình chính để khởi tạo và thực thi phiên học tập.

    Returns:
        Dict: Dictionary chứa 'status' và 'error_message'.
    """
    try:
        session_start_time = time.time()

        # Tải nút Student vào đồ thị Neo4j
        student_load_result = check_and_load_students(driver)
        if student_load_result["status"] != "success":
            raise ValueError(f"Lỗi tải nút Student: {student_load_result['error_message']}")
        logger.info("Tải nút Student thành công")


        # Bước 1: Khởi tạo VectorStoreIndex
        init_vector_result = initialize_vector_index()
        if init_vector_result["status"] != "success":
            raise ValueError(f"Lỗi khởi tạo VectorStoreIndex: {init_vector_result['error_message']}")
        logger.info("Khởi tạo VectorStoreIndex thành công")

        # Bước 2: Nhập thông tin học sinh
        student_id = input("Nhập mã học sinh: ").strip() # e.g: "stu001"

        profile = load_student_profile(student_id)
        if not profile or profile.get("student_id") != student_id:
            raise ValueError(f"Student ID '{student_id}' không tìm thấy trong file học sinh")

        while True:
            level = input("Nhập trình độ mong muốn (Remember|Understand|Apply|Analyze|Evaluate|Create): ").strip()
            if level in ["Remember", "Understand", "Apply", "Analyze", "Evaluate", "Create"]:
                break
            if not level:
                confirm = input(f"Bạn có muốn sử dụng trình độ mặc định '{Config.ASTAR_SKILL_LEVELS_LOW[0]}' không? (yes/no): ").strip().lower()
                if confirm == "yes":
                    level = Config.ASTAR_SKILL_LEVELS_LOW[0]
                    print(f"Đã chọn trình độ mặc định: {Config.ASTAR_SKILL_LEVELS_LOW[0]}")
                    break
            else:
                print("Vui lòng nhập đúng theo 1 trong các trình độ sau: Remember|Understand|Apply|Analyze|Evaluate|Create.")

        while True:
            context = input(f"Nhập ngữ cảnh học tập (ví dụ: {Config.DEFAULT_CONTEXT_EXAMPLE}): ").strip()
            if context:
                break
            while True:
                confirm = input(f"Bạn có muốn sử dụng ngữ cảnh mặc định '{Config.DEFAULT_CONTEXT_EXAMPLE}' không? (yes/no): ").strip().lower()
                if confirm == "yes":
                    context = Config.DEFAULT_CONTEXT_EXAMPLE
                    print(f"Đã chọn ngữ cảnh mặc định: {context}")
                    break
                elif confirm == "no":
                    no_context_confirm = input("Bạn có chắc chắn muốn không đặt ngữ cảnh không? (yes/no): ").strip().lower()
                    if no_context_confirm == "yes":
                        context = None
                        print("Đã chọn không đặt ngữ cảnh.")
                        break
                    elif no_context_confirm == "no":
                        break  # Quay lại nhập context
                    else:
                        print("Vui lòng nhập 'yes' hoặc 'no'.")
                        continue
                else:
                    print("Vui lòng nhập 'yes' hoặc 'no'.")
                    continue

            break


        # Nhập phong cách học tập (tùy chọn)
        while True:
            learning_style = input(f"Nhập phong cách học tập (OPTIONAL), e.g.: {', '.join(Config.LEARNING_STYLE_VARK)}): ").strip()
            if learning_style:
                # Xác nhận giá trị đã nhập
                print(f"Bạn đã nhập phong cách học tập: '{learning_style}'")
                confirm = input("Bạn có đồng ý với phong cách này không? (yes/no): ").strip().lower()
                if confirm == "yes":
                    if learning_style in Config.LEARNING_STYLE_VARK:
                        break
                    else:
                        print(f"Phong cách học tập không hợp lệ. Vui lòng nhập một trong: {', '.join(Config.LEARNING_STYLE_VARK)}.")
                elif confirm == "no":
                    print("Vui lòng nhập lại phong cách học tập.")
                else:
                    print("Vui lòng nhập 'yes' hoặc 'no'.")
            else:
                while True:
                    confirm = input(f"Bạn có muốn sử dụng phong cách học tập mặc định '{Config.DEFAULT_LEARNING_STYLE}' không? (yes/no): ").strip().lower()
                    if confirm == "yes":
                        learning_style = Config.DEFAULT_LEARNING_STYLE
                        print(f"Đã chọn phong cách học tập mặc định: {learning_style}")
                        break
                    elif confirm == "no":
                        learning_style = None
                        print("Đã chọn không đặt phong cách học tập.")
                        break
                    else:
                        print("Vui lòng nhập 'yes' hoặc 'no'.")
                break  # Thoát vòng lặp chính sau khi xử lý chuỗi rỗng


        # Nhập mục tiêu học tập (bắt buộc)
        while True:
            student_goal = input("Nhập mục tiêu học tập (REQUIRED): ").strip()
            if not student_goal:
                print("Mục tiêu học tập không được để trống. Vui lòng nhập lại.")
                continue
            # Xác nhận giá trị đã nhập
            print(f"Bạn đã nhập mục tiêu học tập: '{student_goal}'")
            confirm = input("Bạn có đồng ý với mục tiêu này không? (yes/no): ").strip().lower()
            if confirm == "yes":
                break
            elif confirm == "no":
                print("Vui lòng nhập lại mục tiêu học tập.")
            else:
                print("Vui lòng nhập 'yes' hoặc 'no'.")

        # Hỏi người dùng có muốn sử dụng LLM không
        use_llm = False
        while True:
            use_llm_input = input("Bạn có muốn sử dụng AI để tạo lộ trình học tập không? (yes/no): ").strip().lower()
            if use_llm_input in ["yes", "no"]:
                if use_llm_input == "yes":
                    use_llm = True
                    break
                elif use_llm_input == "no":
                    use_llm = False
                    break
            else:
                break
            print("Vui lòng nhập 'yes' hoặc 'no'.")

        # Bước 3: Chạy phiên học tập
        session_result = run_learning_session(student_id, level, context, learning_style, student_goal, use_llm=use_llm)
        if session_result["status"] != "success":
            logger.error(f"Lỗi chạy phiên học tập: {session_result['error_message']}")
            print(session_result)
            logger.warning(f"session_result: {session_result}")
            raise ValueError(f"Lỗi chạy phiên học tập: {session_result['error_message']}")

        # Thêm: Tính tổng thời gian phiên học tập (tùy chọn)
        total_session_time = int((time.time() - session_start_time) / 60)
        logger.info(f"Tổng thời gian phiên học tập: {total_session_time} phút")
        print(f"Tổng thời gian phiên học tập: {total_session_time} phút.")

        # Bước 4: Chạy lọc cộng tác và quy tắc liên kết
        learned_nodes = session_result.get("learned_nodes", [])
        collab_result = collaborative_filtering(student_id, learned_nodes)
        if collab_result["status"] == "success":
            print(f"Các nút được đề xuất: {collab_result['recommended_nodes']}")
        else:
            logger.warning(f"Lỗi lọc cộng tác: {collab_result['error_message']}")

        apriori_result = apply_apriori()
        if apriori_result["status"] == "success":
            print(f"Quy tắc liên kết: {apriori_result['association_rules']}")
        else:
            logger.warning(f"Lỗi quy tắc liên kết: {apriori_result['error_message']}")

        logger.info("Hoàn tất chương trình chính")
        return {
            "status": "success",
            "error_message": None
        }

    except Exception as e:
        logger.error(f"Lỗi trong main: {str(e)}")
        return {
            "status": "error",
            "error_message": str(e)
        }

# **RUN DEMO**

concept:data_task

In [ ]:
if __name__ == "__main__":
    try:
        main_result = main()
        if isinstance(main_result, dict) and "status" in main_result:
            if main_result["status"] == "success":
                print("Chương trình hoàn tất thành công")
            else:
                print(f"Chương trình thất bại: {main_result.get('error_message', 'Lỗi không xác định')}")
        else:
            print("Chương trình thất bại: Kết quả không hợp lệ từ hàm main")
    except Exception as e:
        print(f"Chương trình thất bại: {str(e)}")
# Business Intelligence advanced level

Đã chọn trình độ mặc định: Remember
Đã chọn phong cách học tập mặc định: reading_writing
Bạn đã nhập mục tiêu học tập: 'Business Intelligence advanced level'
Hãy thực hiện bài kiểm tra đầu vào để xác định lộ trình học tập (Chọn 1 trong 4 đáp án A B C D). . .

Câu 1 (STANDARD): What is the primary purpose of data organization in Business Intelligence?
A. To improve data quality
B. To increase data storage capacity
C. To enhance data visualization
D. To optimize data retrieval

Câu 2 (STANDARD): Which of the following is NOT a method of data organization in BI?
A. Data integration
B. Data visualization
C. Data lineage
D. Data preprocessing

Câu 3 (STANDARD): What is the role of data stewardship in data organization?
A. Ensuring data quality and compliance
B. Managing data storage
C. Creating data visualizations
D. Optimizing data retrieval

Câu 4 (STANDARD): Which of the following is a key component of a data warehouse?
A. ETL processes
B. Data visualization tools
C. Data sources
D. Data

In [ ]:
if __name__ == "__main__":
    try:
        main_result = main()
        if isinstance(main_result, dict) and "status" in main_result:
            if main_result["status"] == "success":
                print("Chương trình hoàn tất thành công")
            else:
                print(f"Chương trình thất bại: {main_result.get('error_message', 'Lỗi không xác định')}")
        else:
            print("Chương trình thất bại: Kết quả không hợp lệ từ hàm main")
    except Exception as e:
        print(f"Chương trình thất bại: {str(e)}")
# Business Intelligence advanced level

Nhập mã học sinh: stu001
Nhập trình độ mong muốn (Remember|Understand|Apply|Analyze|Evaluate|Create): 
Bạn có muốn sử dụng trình độ mặc định 'Remember' không? (yes/no): yes
Đã chọn trình độ mặc định: Remember
Nhập ngữ cảnh học tập (ví dụ: e_learning): 
Bạn có muốn sử dụng ngữ cảnh mặc định 'e_learning' không? (yes/no): no
Bạn có chắc chắn muốn không đặt ngữ cảnh không? (yes/no): yes
Đã chọn không đặt ngữ cảnh.
Nhập phong cách học tập (OPTIONAL), e.g.: visual, auditory, reading_writing, kinesthetic): 
Bạn có muốn sử dụng phong cách học tập mặc định 'reading_writing' không? (yes/no): yes
Đã chọn phong cách học tập mặc định: reading_writing
Nhập mục tiêu học tập (REQUIRED): Business Intelligence advanced level
Bạn đã nhập mục tiêu học tập: 'Business Intelligence advanced level'
Bạn có đồng ý với mục tiêu này không? (yes/no): yes
Bạn có muốn sử dụng AI để tạo lộ trình học tập không? (yes/no): no
Hãy thực hiện bài kiểm tra đầu vào để xác định lộ trình học tập (Chọn 1 trong 4 đáp án A B C D)

In [ ]:
if __name__ == "__main__":
    try:
        main_result = main()
        if isinstance(main_result, dict) and "status" in main_result:
            if main_result["status"] == "success":
                print("Chương trình hoàn tất thành công")
            else:
                print(f"Chương trình thất bại: {main_result.get('error_message', 'Lỗi không xác định')}")
        else:
            print("Chương trình thất bại: Kết quả không hợp lệ từ hàm main")
    except Exception as e:
        print(f"Chương trình thất bại: {str(e)}")
# Business Intelligence advanced level

Nhập mã học sinh: stuyyy
Nhập trình độ mong muốn (Remember|Understand|Apply|Analyze|Evaluate|Create): 
Bạn có muốn sử dụng trình độ mặc định 'Remember' không? (yes/no): yes
Đã chọn trình độ mặc định: Remember
Nhập ngữ cảnh học tập (ví dụ: e_learning): 
Bạn có muốn sử dụng ngữ cảnh mặc định 'e_learning' không? (yes/no): no
Bạn có chắc chắn muốn không đặt ngữ cảnh không? (yes/no): yes
Đã chọn không đặt ngữ cảnh.
Nhập phong cách học tập (OPTIONAL), e.g.: visual, auditory, reading_writing, kinesthetic): 
Bạn có muốn sử dụng phong cách học tập mặc định 'reading_writing' không? (yes/no): yes
Đã chọn phong cách học tập mặc định: reading_writing
Nhập mục tiêu học tập (REQUIRED): Business Intelligence advanced level
Bạn đã nhập mục tiêu học tập: 'Business Intelligence advanced level'
Bạn có đồng ý với mục tiêu này không? (yes/no): yes
Bạn có muốn sử dụng AI để tạo lộ trình học tập không? (yes/no): no
Hãy thực hiện bài kiểm tra đầu vào để xác định lộ trình học tập (Chọn 1 trong 4 đáp án A B C D)

KeyboardInterrupt: Interrupted by user

# LOG

In [ ]:
 display_log_content(log_file)


In [ ]:
# generate_learning_path(
#     student_id="stu001",
#     start_node="concept:data_task",
#     level="Remember",
#     context="",
#     learning_style="reading_writing",
#     student_goal="Business Intelligence basics"
# )

In [ ]:
# generate_learning_path(
#     student_id="stu001",
#     start_node="concept:data_task",
#     level="Remember",
#     context="",
#     learning_style="reading_writing",
#     student_goal="Business Intelligence basics"
# )